In [ ]:
%config Completer.use_jedi = False # To make auto-complete faster

#Reloads imported files automatically
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../utils/')

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stat
from scipy.spatial import KDTree
import copy
import time
import os

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
from matplotlib import colormaps as mplcmaps
import matplotlib.cm as cm

from matplotlib_param_funcs import set_matplotlib_params,reset_rcParams
set_matplotlib_params()

In [ ]:
import compute_variables as CV
import miscellaneous_functions as MF
import mixed_plots as MP
import plotting_helpers as PH
import variable_values_and_errors as val_err
import velocity_plot
import load_sim
import load_data
import map_functions as mapf
import coordinates

In [ ]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:88% !important; }</style>"))

In [ ]:
# degree_symbol = '°'
degree_symbol = '^\circ'

mass_density_label = r"$\Sigma \hspace{0.3} [\rm M_\odot kpc^{-2}]$"

In [ ]:
coolwarm = mplcmaps['coolwarm']
red = coolwarm(0.95)
blue = coolwarm(0.05)
green = 'darkgreen'
grey = 'lightgrey'

plt.rcParams['font.size'] = 20

In [ ]:
#CHOOSE

x_variable = "l"
y_variable = "b"

vel_x_variable = 'r'
vel_y_variable = 'l'

In [ ]:
symbol_dict = mapf.get_kinematic_symbols_dict(x_variable=x_variable,
                                                         y_variable=y_variable,
                                                         vel_x_variable=vel_x_variable,
                                                         vel_y_variable=vel_y_variable)

units_dict = mapf.get_kinematic_units_dict(position_variables=x_variable+y_variable,
                                                     vel_x_variable=vel_x_variable,
                                                     vel_y_variable=vel_y_variable,
                                                     degree_symbol=degree_symbol)

pos_symbols_dict,pos_units_dict = mapf.get_position_symbols_and_units_dict(degree_symbol="°")

In [ ]:
symbol_dict["mean_b"] = r"$\langle |b| \rangle$"
units_dict["mean_b"] = r"$[^\circ]$"

In [ ]:
full_map_string_list,divergent_map_list = mapf.get_map_string_lists()

all_maps = False
full_map_string_list = [map_string for map_string in full_map_string_list if "spherical" not in map_string]

print(full_map_string_list)

# Functions

In [ ]:
def get_variable_ticks(map_string, map_array):
    if map_string == "vertex_abs":
        max_value = 45
        min_value = -max_value
    elif map_string == "vertex":
        max_value = 90
        min_value = -max_value
    elif map_string == "error":
        min_value = 0
        max_value = np.nanmax(map_array)
    elif map_string == "varr" or map_string == "varl":
        #min_value = 0
        min_value = np.nanmin([all_varr[norm_index], all_varl[norm_index]])
        max_value = np.nanmax([all_varr, all_varl])
    elif map_string == "mean_vr" or map_string == "mean_vl":
        mini = np.nanmin([map_dict["mean_vr"],map_dict["mean_vl"]])
        maxi = np.nanmax([map_dict["mean_vr"],map_dict["mean_vl"]])

        limits = [mini, maxi]

        lims_factor = 1
        min_value = -lims_factor*np.max(np.abs(limits))
        max_value = lims_factor*np.max(np.abs(limits))
    elif map_string in ["covariance","correlation","anisotropy"]:#divergent_map_list:
        mini = np.nanmin(map_array)
        maxi = np.nanmax(map_array)

        limits = [mini, maxi]

        lims_factor = 1
        min_value = -lims_factor*np.max(np.abs(limits))
        max_value = lims_factor*np.max(np.abs(limits))
    else:
        min_value = np.nanmin(map_array)
        max_value = np.nanmax(map_array)

    ticks = np.linspace(min_value,max_value,5)
    return ticks
def create_map_array_dict(full_map_string_list, all_arrays):
    if len(full_map_string_list) != len(all_arrays):
        raise ValueError("len(full_map_string_list) different to len(all_arrays)")
    map_dict = {}
    for index, map_string in enumerate(full_map_string_list):
        map_dict[map_string] = all_arrays[index]
    return map_dict

# Load

In [ ]:
dummy_df = pd.DataFrame([[1,2,3],[2,3,1],[6,3,4]], columns=['a','b','c'])

In [ ]:
general_path = '/Users/luismi/Desktop/MRes_UCLan/'

In [ ]:
zabs = True
# zabs = False

R0 = 8.1

GSR = True
# GSR = False

## Sim

In [ ]:
sim_choice = "708main" #stuart

rot_angle = -27
axisymmetric = False
pos_scaling = 1.7

In [ ]:
load_chunk = False

if not load_chunk:
    np_path = general_path+f"data/708main_simulation/numpy_arrays/R0_{R0}/"
        
    df0 = load_sim.load_simulation(path=np_path, choice=sim_choice, rot_angle = rot_angle, axisymmetric=axisymmetric, GSR=GSR,zabs=zabs, pos_factor=pos_scaling,R0=R0)
else:
    if sim_choice == "708main" and rot_angle == 27 and not axisymmetric and zabs and sim_scaling == 1.7:
        pickle_name = "df_bulge_zabs.pkl"
        df0 = pd.read_pickle("708main_simulation/"+pickle_name)

In [ ]:
sim_resampled_bool = False

## Observations

### Gibs 

In [ ]:
general_path = "C:/Users/Luismi/JUPYTER_NOTEBOOKS/MRes_UCLan/"

In [ ]:
data = pd.DataFrame(np.genfromtxt(general_path + "Oscar_data/gibs_vvvPMs.dat", names=True))

In [ ]:
data.head()

In [ ]:
delete_columns = data.columns

In [ ]:
columns_to_keep = ['l','b','Vgc','FeH','mul_grs','mub_grs']

In [ ]:
delete_columns = delete_columns.drop(columns_to_keep)

In [ ]:
data.drop(columns=delete_columns, inplace=True)

In [ ]:
data.dropna(how='any',inplace=True)

In [ ]:
fixed_distance = 8 #8.2

In [ ]:
data['vl']=(fixed_distance*3.086e16)*data.mul_grs.values*((np.pi/(180 * 3600))*10**(-3)/(3.1536e7))
data['vb']=(fixed_distance*3.086e16)*data.mub_grs.values*((np.pi/(180 * 3600))*10**(-3)/(3.1536e7))

In [ ]:
data["vr"] = data["Vgc"]
#data["vr"] = np.sqrt(data["Vgc"]**2 - data["vl"]**2)

In [ ]:
data.drop(columns = ['Vgc','mul_grs','mub_grs'],inplace=True)

In [ ]:
data.loc[data.l > 180, 'l'] -= 360

In [ ]:
data.head()

#### Visualise

In [ ]:
save_path = general_path+'708main_simulation/graphs/Oscar/GIBS/'
print("Saving in:",save_path)

save_bool = False

In [ ]:
fig, ax = plt.subplots()

fullrange = True

lim = 1000
if not fullrange:
    ax.set_xlim(-lim,lim)
    bins = np.linspace(-lim,lim,100)
else:
    bins = 100
    ax.text(x=0.03,y=0.06,s=str(len(np.where(data['vl'] < -1000)[0])),color='blue',transform=ax.transAxes)
lw = 3
alpha = 0.5
a_n,a_bins,_ = ax.hist(data['vl'],bins=bins,alpha=alpha, color='blue',label=r'GIBS $v_l$')
ax.plot([np.mean(data['vl']),np.mean(data['vl'])],[0,np.max(a_n)],alpha=1,color='blue',lw=lw,linestyle='-.')

b_n,_,_ = ax.hist(data['vr'],bins=bins if not fullrange else a_bins,alpha=alpha, color='cyan',label=r'GIBS $v_r$')
ax.plot([np.mean(data['vr']),np.mean(data['vr'])],[0,np.max(b_n)],alpha=1,color='cyan',lw=lw,linestyle='-.')

ax.set_xlabel(r"Velocity $[\mathrm{km \hspace{0.3} s^{-1}}]$")
ax.set_ylabel(r"$N$", rotation=0,labelpad=20)
plt.legend(loc='best')
if save_bool:
    filename = 'velocities_fullrange' if fullrange else 'velocities'
    plt.savefig(save_path+filename+'.png',bbox_inches='tight',dpi=150)
    print(save_path+filename+'.png')
plt.show()

### Apogee

In [ ]:
# obs_errors = True
obs_errors = False

data_zabs = True
# data_zabs = False

In [ ]:
data_path = general_path+"data/Observational_data/"

data = load_data.load_and_process_data(data_path=data_path, error_bool=obs_errors, zabs=zabs, R0=R0, GSR=GSR)

## Re-sample sim

In [ ]:
K = 10 # number of nearest neighbours to find

tree = KDTree(df0[["x","y","z"]].values)

already_chosen_indices = set()
nearest_simulation_star_indices = []
all_distances = []

for i, obs_star in enumerate(data[["x","y","z"]].values):
    k = K
    while True:
        distances, indices = tree.query(obs_star, k=k)# + len(already_chosen_indices))
        
        new_indices = [idx for idx in indices if idx not in already_chosen_indices]
        
        if len(new_indices) >= K:
            chosen_indices = new_indices[:K]
            
            nearest_simulation_star_indices.append(chosen_indices)            
            all_distances.append([dist for idx, dist in zip(indices, distances) if idx in chosen_indices])
            
            already_chosen_indices.update(chosen_indices)
            
            break
        else:
            k += K
            
#     clear_output(wait=True)
#     print(f"Iteration: {i+1}/{len(data)}")
            
nearest_simulation_star_indices = np.array(nearest_simulation_star_indices)
all_distances = np.array(all_distances)

In [ ]:
nearest_simulation_star_indices.shape, all_distances.shape

In [ ]:
save_path = f"{general_path}graphs/other_plots/resampled_sim/"

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot distance distribution

fig,ax=plt.subplots()

ax.hist(all_distances.flatten(),bins=300,log=False)
ax.set_xlabel("Nearest neighbour distances [kpc]")
ax.set_ylabel(r"$N$",rotation=0,labelpad=15)

max_distance = max(all_distances.flatten())
ax.axvline(x=max_distance,color="red",label="Maximum distance: %.2f [kpc]"%max_distance)
ax.legend()

if save_bool:
    plt.savefig(save_path + "distance_distribution",dpi=150,bbox_inches="tight")
    print("Saved:",save_path + "distance_distribution")
plt.show()

In [ ]:
if False: # lognormal fit to the distance distribution

    from scipy.stats import lognorm, norm, gaussian_kde

    # fit lognorm in linear scale
    shape, loc, scale = lognorm.fit(all_distances.flatten(), floc=0)
    mu, sigma = np.log(scale), shape

    plt.figure()
    count, bins, ignored = plt.hist(all_distances.flatten(), bins=200, density=True, alpha=0.3, color='b')
    pdf = lognorm.pdf(bins, sigma, scale=np.exp(mu))
    plt.plot(bins, pdf, color='r')
    plt.show()

    # fit gaussian in log scale
    log_distances = np.log(all_distances.flatten())
    mu, std = norm.fit(log_distances)

    plt.figure()
    count, bins, ignored = plt.hist(log_distances, bins=200, density=True, alpha=0.3, color='b')
    pdf = norm.pdf(bins, mu, std)
    plt.plot(bins, pdf, color='r')
    plt.show()

    # check KDE
    values = np.linspace(min(all_distances.flatten()), max(all_distances.flatten()), 1000)
    kde = gaussian_kde(all_distances.flatten())
    kde_values = kde(values)

    plt.figure()
    plt.hist(all_distances.flatten(), bins=200, density=True, alpha=0.3, color='b')
    plt.plot(values, kde_values, color='r')
    plt.show()

In [ ]:
sim_resampled = df0.iloc[nearest_simulation_star_indices.flatten()]

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
cuts_dict = {"R":[0,3.5],"age":[4,10]}#,"z":[0.5,2.2]}

cumulative = True
# cumulative = False

if True: # plot age distribution
    fig,ax=plt.subplots()

    bins = 500 if cumulative else 100

    ax.hist(MF.apply_cuts_to_df(df=df0, cuts_dict=cuts_dict)["age"],bins=bins,label="Original",density=True,cumulative=cumulative)
    ax.hist(MF.apply_cuts_to_df(df=sim_resampled, cuts_dict=cuts_dict)["age"],bins=bins,alpha=0.7,label=f"Resampled (k={K})",density=True,cumulative=cumulative)
    ax.legend()
    ax.set_xlabel("Age [Gyr]")
    ax.set_ylabel("Probability density" if not cumulative else "Fraction")

    filename = "age_distribution"
    filename += "_cumulative" if cumulative else ""

    for cut_variable in cuts_dict:
        if cut_variable != "age":
            l,r = cuts_dict[cut_variable]
            filename += f"_{l}{cut_variable}{r}"

    if save_bool:
        plt.savefig(save_path+filename+".png",dpi=150,bbox_inches="tight")
        print("Saved:",save_path+filename+".png")
    plt.show()

In [ ]:
save_bool = True
# save_bool = False

# density_bool = True
density_bool = False
bins_x = 100

if True: # xy and xz views
    fig,axs=plt.subplots(figsize=(9,11),nrows=2,gridspec_kw={"hspace":0})
    c1 = MP.quick_show_xy(sim_resampled,show=False,density=density_bool,bins_x=bins_x)
    c2 = MP.quick_show_xz(sim_resampled,show=False,zmin=0,density=density_bool,bins_x=bins_x)

    norm = PH.get_norm_from_count_list([c1,c2],log=True)

    MP.quick_show_xy(sim_resampled,ax=axs[0],norm=norm, density=density_bool,bins_x=bins_x)
    MP.quick_show_xz(sim_resampled,ax=axs[1],norm=norm,zmin=0, density=density_bool,bins_x=bins_x)

    cbar = plt.colorbar(cm.ScalarMappable(norm=norm,cmap="viridis"),ax=axs,shrink=0.8)
    cbar.set_label(mass_density_label) if density_bool else cbar.set_label(r"$N$",rotation=0,labelpad=20)

    for ax in axs:
        ax.set_aspect("equal")
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    if True: # filename and saving

        filename = f"resampled_sim_xyxz_k{k}"
        filename += f"_xbins{bins_x}"
        filename += "_N" if not density_bool else "_density"

        print(filename)

        if save_bool:
            print("Saving in:",save_path)

            plt.savefig(save_path+filename+".png", dpi=200,bbox_inches="tight")

        plt.show()

In [ ]:
sim_resampled_bool = True

if sim_resampled_bool:
    df0 = sim_resampled
    
    del sim_resampled

# Save arrays

## Path functions

In [ ]:
def get_base_path(single_variable):
    save_path = general_path+f'graphs/Observations/Apogee/individual_variable/{single_variable}/'
    MF.create_dir(save_path)
    
    return save_path

def get_save_path_cuts(single_variable, cuts_dict):
        
    save_path = get_base_path(single_variable)
    
    for variable in cuts_dict:
        value_tuple = cuts_dict[variable]
        
        save_path += f"{MF.return_int_or_dec(value_tuple[0],2)}{variable}{MF.return_int_or_dec(value_tuple[1],2)}/"
        MF.create_dir(save_path)
    
    return save_path

def get_save_path_data(save_path_spatial, metal_lowcut, metal_lims, binning_str, pop_str):
        
    save_path = save_path_spatial + "data/"
    MF.create_dir(save_path)
    
    save_path += f"metal_lowcut_{metal_lowcut}/"
    MF.create_dir(save_path)
    
    save_path += f"{str(MF.return_int_or_dec(metal_lims[0],2))}metal{str(MF.return_int_or_dec(metal_lims[1],2))}/"
    MF.create_dir(save_path)
    
    save_path += f"{binning_str}/"
    MF.create_dir(save_path)
    
    save_path += f"{pop_str}/"
    MF.create_dir(save_path)
        
    return save_path

def get_save_path_sim(save_path_spatial, bar_angle, resampled_sim_bool, age_lims, binning_str, pop_str):
        
    save_path = save_path_spatial + "sim/"
    MF.create_dir(save_path)
    
    save_path += f"bar_angle_{bar_angle}/"
    MF.create_dir(save_path)
    
    save_path += "resampled_sim/" if resampled_sim_bool else ""
    MF.create_dir(save_path)
    
    save_path += f"{MF.return_int_or_dec(age_lims[0],2)}age{MF.return_int_or_dec(age_lims[1],2)}/"
    MF.create_dir(save_path)
    
    save_path += f"{binning_str}/"
    MF.create_dir(save_path)
    
    save_path += f"{pop_str}/"
    MF.create_dir(save_path)
        
    return save_path

## Spatial cuts

In [ ]:
x_var = "b"

x_min = 1
x_max = 13

if True: # x_label
    if x_var in pos_symbols_dict:
        x_label = pos_symbols_dict[x_var] + f" [{pos_units_dict[x_var]}]"
    else:
        if x_var == "age":
            x_label = "Age [Gyr]"
        elif x_var == "FeH":
            x_label = "[Fe/H]"
        else:
            raise ValueError("x_label not defined for the chosen variable}")
    
MP.show_text(x_label)

In [ ]:
spatial_cuts_dict = {
    x_var: [x_min,x_max],
    "R": [0,3.5],
    "l": [-2,2]
}

for variable in spatial_cuts_dict:
    print(variable, spatial_cuts_dict[variable])

In [ ]:
save_path_spatial = get_save_path_cuts(single_variable=x_var,cuts_dict=spatial_cuts_dict)

print(save_path_spatial)

## Sim

In [ ]:
data_bool = False

In [ ]:
age_lims = [4,7]
# age_lims = [9.5,10]

df = MF.apply_cuts_to_df(df=df0,cuts_dict=[spatial_cuts_dict,{"age":age_lims}])

### Binning

In [ ]:
equal_steps = True

In [ ]:
if equal_steps:
    if x_var == "b":
        n_edges = 7 if x_max == 9 else 8
        n_points = n_edges - 1
    else:
        raise ValueError(f"Number of points not implemented for {x_var} yet.")

    x_edges = np.linspace(x_min,x_max,n_edges)

    x_range_min = x_edges[:-1]
    x_range_max = x_edges[1:]
    
    binning_str = "equal_steps"
    pop_str = f"{n_points}_bins"
    
# Saving the median because upon plotting we can always compute the mean but for the median we need the dataframe
x_range_plot = PH.get_range_medians(df[x_var], x_range_min, x_range_max)

print("Plotting at:",x_range_plot)
print("Number of datapoints:",n_points)

In [ ]:
fig,ax=plt.subplots(figsize=(10,2))

ax.errorbar(x=x_range_plot,xerr=PH.get_xerr(minima=x_range_min,maxima=x_range_max,plot=x_range_plot,frac=1),y=[0]*n_points,fmt="d",capsize=20)
ax.set_xlim(x_min-(x_max-x_min)/30,x_max+(x_max-x_min)/30)
ax.minorticks_on()
ax.set_xlabel(x_label)
ax.set_yticks([])
plt.show()

In [ ]:
# number of stars per bin
stat.binned_statistic(values=None,x=df[x_var].values,bins=x_edges,statistic="count")[0]

In [ ]:
save_path = get_save_path_sim(save_path_spatial, bar_angle=rot_angle,resampled_sim_bool=sim_resampled_bool, age_lims=age_lims,\
                              binning_str=binning_str, pop_str=pop_str)

print(save_path)

## Data

In [ ]:
data_bool = True

In [ ]:
# metal_lowcut = -9999
metal_lowcut = -1

data_trim = data[data['FeH']>=metal_lowcut]

print(f"Chose minimum metallicity of {metal_lowcut}" if metal_lowcut != -9999 else "No minimum metallicity")

if metal_lowcut != -9999:
    print(f"Removed {len(data)-len(data_trim)} ({MF.return_int_or_dec((len(data)-len(data_trim))/len(data)*100,2)}%) stars. {len(data_trim)} left")

In [ ]:
metallicity_median = MF.return_int_or_dec(np.median(data_trim["FeH"]),2)
print(metallicity_median,np.sum(data_trim['FeH']<metallicity_median),np.sum(data_trim['FeH']>metallicity_median))

In [ ]:
metal_lims = [min(data_trim["FeH"]), metallicity_median]
# metal_lims = [metallicity_median, max(data_trim["FeH"])]

df = MF.apply_cuts_to_df(df=data_trim,cuts_dict=[spatial_cuts_dict,{"FeH":metal_lims}])

print(len(df))

### Binning

In [ ]:
# plot

fig, ax = plt.subplots()

alpha=0.7
xmin = data_trim[x_var].min()
xmax = data_trim[x_var].max()
bins = np.linspace(xmin,xmax,50)

ax.hist(data_trim[x_var],bins=bins,label=fr'${str(MF.return_int_or_dec(min(data_trim["FeH"]),2))}\leq$[Fe/H]$\leq{str(MF.return_int_or_dec(max(data_trim["FeH"]),2))}$',\
        alpha=alpha,color='grey')

if len(data_trim) != len(df):
    ax.hist(df[x_var],bins=bins,label=fr'${str(MF.return_int_or_dec(metal_lims[0],2))}\leq$[Fe/H]$\leq{str(MF.return_int_or_dec(metal_lims[1],2))}$',\
            alpha=alpha,color='orange')

ax.set_xlim(xmin,xmax)
ax.axvline(x_min,color="orange");ax.axvline(x_max,color="orange")
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.set_xlabel(x_label)
ax.set_ylabel(r"$N$",rotation=0,labelpad=20)
ax.legend()

plt
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(20,5))
ax.hist(df[x_var],bins=500)
ticksss = np.arange(x_min,x_max,0.2)
ax.set_xticks(ticksss)
ax.set_xticklabels(labels=[str(np.float32(t)) for t in ticksss],size=8)
ax.set_xlabel(x_label)
plt.show()

In [ ]:
equal_number = False # divide in equal-number bins across all b
equal_number_low = True # divide in equal-number bins below |b|<6.61˚
custom_range = False
equal_steps = False # divide in constant latitude steps

plot_median_bool = True
# plot_median_bool = False # mid-point of bin

if x_var == "b":
    n_points = 3
    n_points += 1 if x_max == 13 else 0
else:
    raise ValueError(f"Number of points not implemented for {x_var} yet.")

print(f"{n_points} total points")

In [ ]:
assert equal_number+equal_number_low+custom_range+equal_steps == 1, "Choose a single range selection"

if x_var == "b":
    if equal_number:
        x_edges = PH.get_equal_n_bin_edges(df[x_var], n_points)

        x_range_max = x_edges[1:]
        x_range_min = x_edges[:-1]

        range_path = ""

    if equal_number_low:
        low_max = np.max(df[df[x_var]<6.8][x_var])
        n_points_low = n_points-2 if x_max == 13 else n_points-1
        edges_low = PH.get_equal_n_bin_edges(df[df[x_var]<=low_max][x_var], n_points_low)

    #     print(edges_low)

        high_min = np.min(df[df[x_var]>6.8][x_var])
        high_max = np.max(df[df[x_var]<9][x_var])

        x_range_min = list(edges_low[:-1]) + [high_min]
        x_range_max = list(edges_low[1:]) + [high_max]

    #     print(x_range_min,x_range_max)

        if x_max == 13:
            higher_min = np.min(df[df[x_var]>9][x_var])
            higher_max = np.max(df[x_var])

            x_range_min += [higher_min]
            x_range_max += [higher_max]

        x_range_min = np.array(x_range_min)
        x_range_max = np.array(x_range_max)
    elif custom_range:
        range_dict = {
            '1min': [0.5,  2.5,  4,    7],
            '1max': [2.5,  4,    6.1,  9],

            '1.5min': [0.5,2.5,4,7],
            '1.5max': [2.5,4,6.2,9],

            '2min': [1,2.5,4,7.1],
            '2max': [2.5,4,6.61,9]
        }

        if x_max==13:
            range_dict["2min"] += [10.4]
            range_dict["2max"] += [13]

        x_range_min = range_dict[str(lmax)+'min']
        x_range_max = range_dict[str(lmax)+'max']

        n_points = len(x_range_plot)    
    elif equal_steps:
        x_edges = np.linspace(x_min,x_max,n_points)
        x_range_min = x_edges[:-1]
        x_range_max = x_edges[1:]
else:
    raise ValueError(f"Binning not implemented for {x_var} yet.")

# Saving the median because upon plotting we can always compute the mean but for the median we need the dataframe
x_range_plot = PH.get_range_medians(df[x_var], x_range_min, x_range_max)

binning_str = np.array(["equal_number","equal_number_low","custom_range","equal_steps"])[np.array([equal_number,equal_number_low,custom_range,equal_steps])][0]
pop_str = f"{n_points}_bins"

print("Plotting at:",x_range_plot,"\n")
print(f"{n_points} datapoints")

In [ ]:
save_path = get_save_path_data(save_path_spatial, metal_lowcut, metal_lims, binning_str, pop_str)

print(save_path)

## Get arrays

In [ ]:
MP.visualise_1D_binning(df[x_var],x_range_min,bin_edges_max=x_range_max,save_bool=True,save_path=save_path,xlabel=x_label,log=False,\
                       hist_bins=400 if not data_bool else 100)

if not data_bool:
    MP.visualise_1D_binning(df[x_var],x_range_min,bin_edges_max=x_range_max,save_bool=True,save_path=save_path,xlabel=x_label,log=True,\
                           hist_bins=400)

In [ ]:
vel_hist_bool = True
# vel_hist_bool = False

velhist_bins = 100 if not data_bool else 50

if vel_hist_bool:
    save_path_hist = save_path + "vel_histograms/"
    MF.create_dir(save_path_hist)
    
    save_path_hist += f"{velhist_bins}bins/"
    MF.create_dir(save_path_hist)
    
    print("Saving velocity histograms on\n",save_path_hist)

In [ ]:
if True: # example vel hist
    MP.plot_velocity_histograms_both_stats(df[(df[x_var]>=min(x_range_min))&(df[x_var]<=min(x_range_max))],vel_x_variable,vel_y_variable,\
                                               bins=velhist_bins,colour_var="x",save_bool=False,suffix="example",verbose=True,show=True)

In [ ]:
bootstrap_repeat = 500

min_star_number = 100 if not data_bool else 50

In [ ]:
map_dict = {}
for map_string in full_map_string_list:
    map_dict[map_string] = np.zeros(shape=(len(x_range_min)))

for x_index, (xmin, xmax) in enumerate(zip(x_range_min,x_range_max)):

    print(xmin,xmax)
    
    include_lims = "both" if x_index==len(x_range_min)-1 else "min"
    df_x = MF.apply_cuts_to_df(df, cuts_dict={x_var:[xmin,xmax]}, lims_dict={x_var:include_lims})

    if vel_hist_bool:
        name_suffix = f"{str(MF.return_int_or_dec(xmin,dec=2))}{x_var}{str(MF.return_int_or_dec(xmax,dec=2))}"
        MP.plot_velocity_histograms_both_stats(df_x,vel_x_variable,vel_y_variable,save_bool=True,save_path=save_path_hist,suffix=name_suffix,verbose=x_index==0,bins=velhist_bins)

    values = val_err.get_all_variable_values_and_errors(df_x[f"v{vel_x_variable}"].values,df_x[f"v{vel_y_variable}"].values, full_map_string_list,\
                                                            repeat=bootstrap_repeat, min_number = min_star_number)   

    if len(values) != len(full_map_string_list):
        raise ValueError("The length of the values list does not match the string list!")

    for map_string in full_map_string_list:
        map_dict[map_string][x_index] = values[map_string]
    
del df,df_x
print("Done")

In [ ]:
map_dict["number"]

In [ ]:
# save arrays

array_path = save_path + "arrays/"
MF.create_dir(array_path)

if True: # values as .txt and .npy
            
    with open(array_path+'values.txt','w') as f:
        for key in map_dict:
            f.write(key+'\n')
            np.savetxt(f,map_dict[key],fmt='%.5f')
            f.write('\n')
    
    for map_string in full_map_string_list:
        np.save(array_path+map_string, map_dict[map_string])
        
if True: # plot limits as .txt and .npy

    with open(array_path+'x_ranges.txt','w') as f:
        f.write("x_range_min\n")
        for mini in x_range_min:
            f.write(f"{mini}\t")
        f.write("\n\nx_range_max\n")
        for maxi in x_range_max:
            f.write(f"{maxi}\t")
        f.write("\n\nx_range_plot\n")
        for p in x_range_plot:
            f.write(f"{p}\t")
    
    np.save(array_path+"x_range_min", x_range_min)
    np.save(array_path+"x_range_max", x_range_max)
    np.save(array_path+"x_range_plot", x_range_plot)
    
print("Saved .txt and .npy in",array_path)

### With distance division

Old code

In [ ]:
d_range = np.linspace(dmin,dmax,5)[:-1]
d_step = np.diff(d_range)[0]

# d_range = [6]
# d_step = 4

print(d_range,d_step)

In [ ]:
all_arrays = np.zeros(shape=(7,len(b_range),len(df_ages)))
variables = ["vertex_abs", "anisotropy", "correlation"]
 
for b_index,latitude in enumerate(b_range):
    for age_index,df in enumerate(df_ages):
        df_b = df[(df['b']>latitude)&(df['b']<latitude+b_step)]
        df_b = df[(df['b']>latitude)&(df['b']<latitude+b_step)]
        
        values_d = []
        for d_index, distance in enumerate(d_range):
            vr = df_b[(df_b['d']>distance)&(df_b['d']<distance+d_step)].vr.values
            vl = df_b[(df_b['d']>distance)&(df_b['d']<distance+d_step)].vl.values

            values_d.append(get_all_variable_values_and_errors(vr,vl,bootstrap_repeat=100,min_number=min_number_sim))
        
        values_d = np.array(values_d)
        values = []
        
        for var in variables:
            val_index = np.where(full_map_string_list == var)[0][0]
            err_index = np.where(full_map_string_list == (var+"_error"))[0][0]
            
            mean_variance = 1/np.sum(1/values_d[:,err_index]**2)
            mean = mean_variance*np.sum(values_d[:,val_index]/values_d[:,err_index]**2)
            values.append(mean)
            values.append(np.sqrt(mean_variance))
                
        for index, val in enumerate(values):
            all_arrays[index, b_index, age_index] = val
    
print("Done")

In [ ]:
full_d_variables = []
for i in variables:
    full_d_variables.append(i)
    full_d_variables.append(i+'_error')

d_map_dict = {}
for variable, array in zip(full_d_variables,all_arrays):
    d_map_dict[variable] = array

In [ ]:
var = 'vertex_abs'

In [ ]:
fig, ax = plt.subplots()

if var == 'vertex_abs':
    ax.set_ylim(-45,45)
else:
    max_val = np.max(np.abs(map_dict[var])) + 0.05
#     ax.set_ylim(-max_val,max_val)
    ax.set_ylim(-0.7,0.5)
ax.fill_between(x=b_range_plot,y1=map_dict[var][:,0]-map_dict[var+'_error'][:,0],y2=map_dict[var][:,0]+map_dict[var+'_error'][:,0],color='blue',alpha=0.6,label='Young')
ax.fill_between(x=b_range_plot,y1=map_dict[var][:,1]-map_dict[var+'_error'][:,1],y2=map_dict[var][:,1]+map_dict[var+'_error'][:,1],color='red',alpha=0.6,label='Old')
ax.axhline(y=0,color='black',linestyle='dotted')
ax.set_title(var)
plt.legend(loc='best')
plt.show()

save_path_arrays = save_path + "arrays/"
if not os.path.isdir(save_path_arrays):
    os.mkdir(save_path_arrays)
    
    for map_string in full_map_string_list:
        np.save(save_path_arrays+"708main_"+map_string, map_dict[map_string])
    print("Arrays saved in",save_path_arrays)

## BT repeat
The standard deviation is the deviation from the mean. We are not calculating the vertex deviation's standard deviation from the mean bootstrap vertex, but from the true vertex instead.
Therefore, let's analyse how much these two values, $l_\mathrm{v}$ and $\langle l_{\mathrm{v}}^\mathrm{bootstrap}\rangle$, differ for different choices of bootstrap repetitions.

To run the code below, you first have to change the definition of get_std_bootstrap() and get_vertex_std_bootstrap() so that they return the list of bootstrap values

In [ ]:
min_number = 100

In [ ]:
bootstrap_repeats = np.array([100,500,1000,5000,10000,50000])
ani_diff,corr_diff,vertex_diff = [[] for repeat in bootstrap_repeats],[[] for repeat in bootstrap_repeats],[[] for repeat in bootstrap_repeats]

df = df_ages[1] ###
for b_index,latitude in enumerate(b_range):
    start = time.time()
    #for age_index,df in enumerate(df_ages):
    
    vr = df[(df['b']>latitude)&(df['b']<latitude+b_step)].vr.values
    vl = df[(df['b']>latitude)&(df['b']<latitude+b_step)].vl.values

    number = len(vr)

    if number > min_number:

        cov = np.cov(vr,vl)

        true_anisotropy = 1-cov[1,1]/cov[0,0]
        true_correlation = cov[0,1]/np.sqrt(cov[0,0]*cov[1,1])
        true_vertex = np.degrees(np.arctan2(2.*cov[0,1], cov[0,0]-cov[1,1])/2.)

        for repeat_index, bootstrap_repeat in enumerate(bootstrap_repeats):

            anisotropy_boot_values,_ = get_std_bootstrap(vr,vl,calculate_anisotropy,repeat=bootstrap_repeat)
            correlation_boot_values,_ = get_std_bootstrap(vr,vl,calculate_correlation,repeat=bootstrap_repeat)
            vertex_boot_values,_ = get_vertex_std_bootstrap(vr, vl, repeat=bootstrap_repeat)

            ani_diff[repeat_index].append(np.abs(true_anisotropy-np.mean(anisotropy_boot_values)))
            corr_diff[repeat_index].append(np.abs(true_correlation-np.mean(correlation_boot_values)))
            vertex_diff[repeat_index].append(np.abs(true_vertex-np.mean(vertex_boot_values)))

            print("Done with repeat",bootstrap_repeat)
                
    print("Done with latitude index",b_index)
    end = time.time()
    print("Took",(end-start)/60,"minutes")

### Plot

In [ ]:
diff_variable = "vertex"

In [ ]:
diffs_dict = {
    "anisotropy": ani_diff,
    "correlation": corr_diff,
    "vertex": vertex_diff
}

ylabel_dict = {
    "anisotropy": r"abs($a_{rl} - \langle a_{rl}^{\mathrm{bootstrap}} \rangle$)",
    "correlation": r"abs($\rho_{rl}-\langle \rho_{rl}^{\mathrm{bootstrap}} \rangle$)",
    "vertex": r"abs($l_\mathrm{v}-\langle l_{\mathrm{v}}^{\mathrm{bootstrap}} \rangle$) [°]"
}

diffs = diffs_dict[diff_variable]
ylabel = ylabel_dict[diff_variable]

In [ ]:
for diff_variable in ["anisotropy","correlation","vertex"]:
    
    diffs = diffs_dict[diff_variable]
    ylabel = ylabel_dict[diff_variable]
    
    diffs_mean = [np.mean(np.abs(array)) for array in diffs]
    diffs_95_percentile = [np.percentile(np.abs(array),95) for array in diffs]
    diffs_max = [np.max(np.abs(array)) for array in diffs]

    fig, ax = plt.subplots()

    bar_width = 0.1
    transparency = 0.6

    x_ticks = np.arange(len(bootstrap_repeats))+1
    x_ticklabels = [str(repeat) for repeat in bootstrap_repeats]

    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_ticklabels)

    ax.bar(x_ticks-bar_width,diffs_mean, width=bar_width, label="Mean",alpha=transparency,color='blue')
    ax.bar(x_ticks,diffs_95_percentile, width=bar_width, label="95 percentile",alpha=transparency,color='red')
    ax.bar(x_ticks+bar_width,diffs_max, width=bar_width, label="Maximum",alpha=transparency,color='green')

    linewidth = 0.5
    for index in range(len(bootstrap_repeats)):
        ax.hlines(diffs_mean[index], xmin=0, xmax=x_ticks[index]-bar_width, color='blue',linestyle='-',lw=linewidth)
        ax.hlines(diffs_95_percentile[index], xmin=0, xmax=x_ticks[index], color='red',linestyle='-',lw=linewidth)
        ax.hlines(diffs_max[index], xmin=0, xmax=x_ticks[index]+bar_width, color='green',linestyle='-',lw=linewidth)

    #ax.legend(bbox_to_anchor=[0.535,0.8])
    ax.set_xlabel("Bootstrap repeats")
    ax.set_ylabel(ylabel)
    #ax.set_yticks(np.arange(0,15))
    #ax.set_xticks([100,500,1000])
    ax.set_xlim(0.75,x_ticks[-1]+0.25)
    #ax.set_ylim(0,13)

    #ax.set_aspect(50)
    filename = f"old_{bootstrap_repeats[-1]}_{diff_variable}"
    plt.savefig(general_path+"bootstrap_repeats/"+filename+".png",bbox_inches='tight',dpi=150)
    print("Saved:",filename)
    plt.close()

# Plot

In [ ]:
def LATPLOT_number_bars_sim(sim_number_array, sim_b_range_plot,alpha=1,zorder=0, show_hist=False):
    if show_hist:
        ax_histx.hist(df_extra["b"].values,bins=100,color="k",histtype="step")
    
    ax_histx.bar(sim_b_range_plot-bar_width/2, sim_number_array[:,0], width=bar_width,alpha=alpha,log=True,color=color_y,zorder=zorder)#,hatch=".")
    ax_histx.bar(sim_b_range_plot+bar_width/2, sim_number_array[:,1], width=bar_width,alpha=alpha,log=True,color=color_o,zorder=zorder)#,hatch=".")
    
def LATPLOT_number_bars_data(data_number_array, o_b_range_plot, alpha=1, zorder=0, linewidth=2,show_hist=False):
    if show_hist:
        ax_histx.hist(o_df_extra["b"].values,bins=50,color="grey",histtype="step")
    
    ax_histx.bar(o_b_range_plot-bar_width/2, data_number_array[:,0], width=bar_width,fill=False,log=True,zorder=zorder,linewidth=linewidth,edgecolor=color_y)#,color="k")
    ax_histx.bar(o_b_range_plot+bar_width/2, data_number_array[:,1], width=bar_width,fill=False,log=True,zorder=zorder,linewidth=linewidth,edgecolor=color_o)#,color="k")

def LATPLOT_values_sim(map_array, error_array, sim_b_range_plot, alpha=1, zorder=0, legend=False):
    
    if np.all(error_array == 0):
        linestyle = None
        line_labels = [label_y, label_o] if legend else [None,None]
        area_labels = [None,None]
    else:
        linestyle = "--"
        line_labels = [None,None]
        area_labels = [label_y, label_o] if legend else [None,None]
    
    ax.plot(sim_b_range_plot, map_array[:,0], color=color_y, linestyle=linestyle, lw=1,zorder=zorder,label=line_labels[0])
    ax.plot(sim_b_range_plot, map_array[:,1] , color=color_o, linestyle=linestyle, lw=1,zorder=zorder,label=line_labels[1])

    ax.fill_between(sim_b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha, facecolor=color_y, zorder=zorder,label=area_labels[0])
    ax.fill_between(sim_b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha, facecolor=color_o, zorder=zorder,label=area_labels[1])

    if map_string =='vertex':
        # to make it cyclic, mirror what goes below the plot to above
        displacement = max(yticks) - min(yticks)
        ax.fill_between(sim_b_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha, facecolor=color_o,zorder=zorder)
        ax.fill_between(sim_b_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha, facecolor=color_o,zorder=zorder)
        ax.fill_between(sim_b_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha, facecolor=color_y,zorder=zorder)
        ax.fill_between(sim_b_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha, facecolor=color_y,zorder=zorder)
        
def LATPLOT_values_data(map_array, error_array, o_b_range_min, o_b_range_max, o_b_range_plot, alpha=1, zorder=1, legend=False):
    if False:#map_string == "anisotropy" and not outlier_bool:
        map_array = copy.copy(map_array)
        error_array = copy.copy(error_array)
        map_array[-1,0] = np.nan # COMMENT THIS OUT TO CHECK THE LAST DATAPOINT IS INDEED OFF
        #error_array[np.where(np.abs(error_array)>2)] = 1        
        
    marker_size = 7.5
    
    xerr = PH.get_xerr(o_b_range_min, o_b_range_max, o_b_range_plot, frac=xerr_frac) if x_error_data_bool else None
    
    ax.errorbar(o_b_range_plot, map_array[:,1] , yerr= error_array[:,1], xerr=xerr, color=color_o, alpha=alpha,capsize=capsize_lat,markersize=marker_size,ls="none",marker="$\u25A1$",zorder=zorder,label=label_poor if legend else None)
    ax.errorbar(o_b_range_plot, map_array[:,0], yerr= error_array[:,0], xerr=xerr, color=color_y, alpha=0.9 if alpha==1 else alpha,capsize=capsize_lat,markersize=marker_size,ls="none",marker="$\u25EF$",zorder=zorder,label=label_rich if legend else None)
    if halo_bool:
        ax.errorbar(o_b_range_plot, map_array[:,2] , yerr=error_array[:,2], color='cyan', alpha=0.9 if alpha==1 else alpha,capsize=capsize_lat,markersize=marker_size,ls="none",marker="$\u25B3$",zorder=zorder,label=label_halo if legend else None)
        
def LATPLOT_number_bars_axis_settings():
    ax_histx.tick_params(axis='x',bottom=False,top=False,labelbottom=False)
    ax_histx.tick_params(axis='y',labelleft=False,labelright=True, labelsize=number_bar_labelsize)

    histx_ticks = [10**i for i in range(1,6)]
    ax_histx.set_yticks(histx_ticks)
#     ax_histx.set_yticklabels(['']+[r"$10^%i$"%np.log10(i) for i in histx_ticks[1:]])
    ax_histx.set_ylabel(r"$N$",labelpad=20,rotation=0,size=axis_labelsize)
    ax_histx.yaxis.set_label_position("right")
        
def LATPLOT_xaxis_settings():
    xmax = np.max(np.concatenate([sim_b_range_max, o_b_range_max]))
    ax.set_xlim(bmin,xmax)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    
#     ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    
    ax.set_xlabel(r"$|b|$ $[^\circ]$", size=axis_labelsize)
        
def LATPLOT_yaxis_settings(map_string, hardcode=True):
    if hardcode and map_string in ylim_dict:
        ax.set_ylim(ylim_dict[map_string])
        
    if map_string in ['tilt_abs','vertex_abs']:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
        
#     yticks = get_variable_ticks(map_string, sim_map_array)
#     if 'vertex' in map_string:
#         ax.set_yticks(yticks)
#         ax.set_ylim(min(yticks),max(yticks))
#     else:
#         ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
#         ax.tick_params(which='minor',direction='in',color='black',length=4)
    
#     ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())

    ax.set_ylabel(map_symbol+units_dict[map_string], size=axis_labelsize)
        
def LATPLOT_legend_and_text(legend_bool, extra_string_bool):

    if legend_bool:
        if map_string in legend_loc_dict:
            leg_location = legend_loc_dict[map_string]
            ax.legend(fontsize=legend_fontsize,loc=leg_location)
        else:
            ax.legend(fontsize=legend_fontsize)
    if extra_string_bool:
        l_string = fr"$|l|<{lmax}^\circ$,"
        extra_string = r"${%i}<d /\mathrm{%s}<{%i}$"%(dmin,'kpc',dmax)
        ax.text(x=0.8,y=-0.1,s=l_string+'  '+d_string,size=extra_variables_size,transform=ax.transAxes)

In [ ]:
def plot_number_bar(barax, plot_range, number_array,color,fill_bool=True,alpha=1,zorder=0, bar_width=50,bar_log=True):
    barax.bar(plot_range,number_array,width=bar_width,log=bar_log,color=color,alpha=alpha,zorder=zorder,fill=fill_bool)
        
def plot_values_scatter_with_errors(ax, val_array, err_array, plot_range, min_range, max_range, color, label,line_alpha=1,zorder=0,\
                                    lines_bool=True,x_error_bool=True):
    xerror = PH.get_xerr(min_range,max_range,plot_range)

    ax.errorbar(plot_range,val_array,yerr=err_array,xerr=xerror if x_error_bool else None,capsize=capsize,marker='.',color=color,\
                label=label,linestyle=None if lines_bool else '',alpha=line_alpha,zorder=zorder)

def plot_values_surface(ax, val_array, err_array, plot_range,color,label,line_alpha=1,surface_alpha=0.75,zorder=0):
    ax.plot(plot_range,val_array,color=color,alpha=line_alpha,zorder=zorder)
    ax.fill_between(plot_range,val_array-err_array,val_array+err_array,label=label,color=color,alpha=surface_alpha,linewidth=0,zorder=zorder)
        
def set_number_bar_axis_settings(barax,min_n,max_n,bar_log=True,labels_on=True,min_shift_bool=True,max_shift_bool=True):
    if bar_log:
        exponent_ticks = np.arange(MF.get_exponent(min_n),MF.get_exponent(max_n)+1,1)
        barax.set_yticks([10**i for i in exponent_ticks])
        barax.set_ylim(bottom = 10**min(exponent_ticks) - (min_n/3 if min_shift_bool else 0))
        barax.set_ylim(top = max_n + (10**MF.get_exponent(max_n) if max_shift_bool else 0))
    elif equal_number:
        barax.set_yticks([0,max_n])
            
    barax.yaxis.set_tick_params(which='minor', right=True,left=False)

    barax.tick_params(which='both',labelleft=False,labelright=labels_on)
    barax.tick_params(which='minor',labelright=False)

    if labels_on:
        barax.set_ylabel(r"$N$",rotation=0,labelpad=20)
        barax.yaxis.set_label_position("right")
    
def set_xaxis_settings(ax,xmin,xmax,xlabel,xticks=None,labels_on=True):
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    
    ax.set_xlim(xmin,xmax)
    
    if xticks is not None:
        ax.set_xticks(xticks)
    
    if labels_on:
        ax.set_xlabel(xlabel)
    else:
        ax.set_xticklabels([])

def compute_ylims(map_dict, map_string, error_string, var1_bool=False):
    minimum = np.nanmin(map_dict[map_string]-map_dict[error_string])
    maximum = np.nanmax(map_dict[map_string]+map_dict[error_string])

    if var1_bool:
        minimum1 = np.nanmin(map_dict[var1]-map_dict[err1])
        maximum1 = np.nanmax(map_dict[var1]+map_dict[err1])

        minimum = min([minimum,minimum1])
        maximum = max([maximum,maximum1])
    
    if map_string in yshift_dict:
        minimum -= yshift_dict[map_string]
        maximum += yshift_dict[map_string]

    if symmetric_ylims_bool:
        maxabs = np.nanmax(np.abs([minimum,maximum]))
        ax.set_ylim(-maxabs,maxabs)
    else:
        ax.set_ylim(minimum,maximum)
        
def set_yaxis_settings(ax, map_string, error_string, map_dict=None, labels_on=True,var1_bool=False, set_ylims=True):
    
    if map_string == 'tilt_abs':# or "mean" in map_string or "std" in map_string:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
        #ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))
    elif map_string == 'anisotropy':# or map_string == 'correlation':
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(0.25))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    elif map_string == "correlation":
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.15))
#     elif map_string in ["mean_vx","mean_vy"]:
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(15))
    elif map_string in ["std_vx","std_vy"]:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(25))
    
    if not var1_bool and labels_on:
        ax.set_ylabel(symbol_dict[map_string]+units_dict[map_string],fontsize=ylabel_size)
    
    if set_ylims:
        if hard_coded_ylims_bool and map_string in hard_coded_ylims:
            ax.set_ylim(hard_coded_ylims[map_string])
        else:
            if map_dict is None:
                raise ValueError("Cannot compute ylims if `map_dict` is None.")
            _PLOT1D_compute_ylims(map_dict, map_string, error_string, var1_bool=var1_bool)
        
    if not labels_on:
        ax.set_yticklabels([])
        
def get_label(map_string):
    return symbol_dict[map_string]+units_dict[map_string]

def get_legend_label(var_tuple,variable):
    var_symbol = pos_symbols_dict[variable]
    var_units = pos_units_dict[variable]
    
    if var_tuple[0] == 0:
        return var_symbol + fr"$< {var_tuple[1]}~$"+var_units
    else:
        return r"$%s<$"%str(var_tuple[0]) + var_symbol + r"$/\mathrm{%s}$"%var_units + fr"$<{var_tuple[1]}$"

In [ ]:
def load_values_and_plot_ranges(path,full_map_string_list):
    
    map_dict = {}
    for m in full_map_string_list:
        map_dict[m] = np.load(f"{path}{m}.npy")
    
    min_range = np.load(path + f"pop_min_range.npy")
    max_range = np.load(path + f"pop_max_range.npy")
    plot_range = np.load(path + f"pop_plot_range.npy")
    
    return map_dict, min_range, max_range, plot_range

In [ ]:
if data_bool: #metallicity labels
    if metal_rich_highlim is None:
        label_rich = fr'[Fe/H]-rich ($>{str(metal_rich_lowlim)}$)'
    else:
        label_rich = fr'[Fe/H]-rich (${str(metal_rich_lowlim)}$-${str(metal_rich_highlim)}$)'

    if metal_poor_lowlim is None:
        label_poor = fr'[Fe/H]-poor ($<{str(metal_rich_lowlim)}$)'
    else:
        label_poor = fr'[Fe/H]-poor (${str(metal_poor_lowlim)}$-$<{str(metal_poor_highlim)}$)'

    MP.show_text(label_rich+"\n"+label_poor)

In [ ]:
# fontsizes and legend loc
plt.rcParams.update({'font.size' : 25})
axis_labelsize = "large"
legend_fontsize = "21.5"
number_bar_labelsize = "medium"
extra_variables_size = "x-small"
capsize_lat = 5

legend_loc_dict = {
    'anisotropy':'lower left',
    'correlation':(0.4,0.71),
    'tilt_abs':(0.4,0.71)
}

ylim_dict = {
    "tilt_abs": [-45,3],
    "vertex_abs": [-45,3],
}

bar_width=0.2 if bmax == 9 else 0.3

In [ ]:
# extra_string_bool = True
extra_string_bool = False

number_bool = True
# number_bool = False

x_error_data_bool = True; xerr_frac = 2
# x_error_data_bool = False

In [ ]:
# Some reference values

l_cuts = [-2,2]
y_cuts = MF.return_int_or_dec_for_array([coordinates.ang_to_rect(ang=l_cut,x=coordinates.get_solar_radius()) for l_cut in l_cuts])

b_range_min,b_range_max = PH.get_equal_n_minmax_b_ranges(data_trim)
b_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(b_range_min,b_range_max)]

z_range_min = [coordinates.ang_to_rect(ang=bmin,x=coordinates.get_solar_radius()) for bmin in b_range_min]
z_range_max = [coordinates.ang_to_rect(ang=bmax,x=coordinates.get_solar_radius()) for bmax in b_range_max]
z_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(z_range_min,z_range_max)]

R_variations = [[0,3.5],[0,2]]

print("l",l_cuts)
print("y",y_cuts)
print("b",b_variations)
print("z",z_variations)
print("R",R_variations)

## Kinpop

##### Prepare

In [ ]:
pos_units_dict["b"] = "deg"

In [ ]:
# see https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3

# colors = ["blue","blue","blue"]
colors = ["#1b9e77","#d95f02","#7570b3"]

line_alpha = 0.9
surface_alpha = 0.75
alpha_reduction_factor = 0.25

number_alpha = 0.75
number_alpha_reduction_factor = 0.5

In [ ]:
# sim galactic vs rectangular 3 columns

# binning_type = "equalN"
binning_type = "equalSteps"

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/1.5b3.51/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/0.21z0.5/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/1.5b3.51/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/0.21z0.5/sim/{binning_type}/40_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([1.5,3.51],"b"),get_legend_label([0.21,0.5],"z")],
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/3.51b6.6/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/0.5z0.94/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/3.51b6.6/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/0.5z0.94/sim/{binning_type}/20_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([3.51, 6.6],"b"),get_legend_label([0.5,0.94],"z")],
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/7.13b8.85/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/1.01z1.26/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/7.13b8.85/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/1.01z1.26/sim/{binning_type}/10_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([7.13, 8.85],"b"),get_legend_label([0.5,0.94],"z")],
    },   
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            f"0R3.5_0R2/-2l2_-0.28y0.28/1.5b3.51_0.21z0.5__3.51b6.6_0.5z0.94__7.13b8.85_1.01z1.26/sim/sim_{binning_type}/sim_n40,40_n40,40__n20,20_n20,20__n10,10_n10,10/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,6,7,8]
    all_dicts[key]["colors"] = 2*["blue", "orange"]
    all_dicts[key]["line_alphas"] = 2*[alpha_reduction_factor*line_alpha] + 2*[line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[alpha_reduction_factor*surface_alpha] + 2*[surface_alpha]
    all_dicts[key]["number_alphas"] = 2*[number_alpha_reduction_factor*number_alpha] + 2*[number_alpha]
    all_dicts[key]["zorderNs"] = [6,6,5,5]
    all_dicts[key]["plot_ranges_str"] = nplots*["mean"]
    all_dicts[key]["surface_bools"] = nplots*[True]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [False,False,True,True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["title"] = None
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(5,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# sim normal vs resampled 3 columns

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/1.5b3.51/sim/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/1.5b3.51/sim/equalN/4_datapoints/arrays/",

        ],
        "title": get_legend_label([1.5,3.51],"b"),
        "labels": 4*[None],
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/3.51b6.6/sim/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/3.51b6.6/sim/equalN/4_datapoints/arrays/",
        ],
        "title": get_legend_label([3.51,6.6],"b"),
        "labels": [None,"Model",None,"Resampled model (k=10)"]
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalN/15_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalN/15_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/7.13b8.85/sim/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/7.13b8.85/sim/equalN/3_datapoints/arrays/",
        ],
        "title": get_legend_label([7.13,8.85],"b"),
        "labels": 4*[None]
    },
    
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "sim__resampled_sim/0R3.5_0R2/-2l2/1.5b3.51__3.51b6.6__7.13b8.85/sim_equalN/sim_n20,20_n4,4__n20,20_n4,4__n15,15_n3,3/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,6,7,8]
    all_dicts[key]["zorderNs"] = [5,5,6,6]
    all_dicts[key]["colors"] = ["blue","blue","orange","orange"]
    
    all_dicts[key]["line_alphas"] = 2*[alpha_reduction_factor*line_alpha, line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[alpha_reduction_factor*surface_alpha, surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["plot_ranges_str"] = ["mean","mean","median","median"]
    all_dicts[key]["surface_bools"] = [True,True,False,False]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [False,True,False,True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(5,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# data and sim 2 columns (sim equalN)

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61]
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalN/25_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalN/25_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10]
    }, 
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN/data_n4,4,3_n4,4,3/sim_equalN/sim_n30,30,25_n30,30,25/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
# data and sim 2 columns (sim equalSteps)

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61]
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalSteps/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalSteps/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalSteps/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalSteps/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalSteps/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalSteps/10_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(4,10,1),
        "x_lims": [4,10]
    }, 
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN/data_n4,4,3_n4,4,3/sim_equalSteps/sim_n40,20,10_n40,20,10/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
# data, sim and sim resampled 3 columns

# sim_binning_type = "equalN"
sim_binning_type = "equalSteps"

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61],
        "bar_width": 0.03
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/1.5b3.51/sim/{sim_binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/3.51b6.6/sim/{sim_binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/7.13b8.85/sim/{sim_binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/1.5b3.51/sim/{sim_binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/3.51b6.6/sim/{sim_binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/7.13b8.85/sim/{sim_binning_type}/10_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10],
        "bar_width": 0.05
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/1.5b3.51/sim/{sim_binning_type}/8_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/3.51b6.6/sim/{sim_binning_type}/5_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/7.13b8.85/sim/{sim_binning_type}/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/1.5b3.51/sim/{sim_binning_type}/8_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/3.51b6.6/sim/{sim_binning_type}/6_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/7.13b8.85/sim/{sim_binning_type}/4_datapoints/arrays/",
        ],
        "title": "Resampled model",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10],
        "bar_width": 0.12
    }    
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            f"data__sim__resampled_sim/0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN__sim_{sim_binning_type}/"+\
            "data_n4,4,3_n4,4,3__sim_n40,20,10_n40,20,10__resampledsim_n8,6,4_n8,5,3/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
for key in all_dicts:
    print(key)
    for path in all_dicts[key]["load_paths"]:
        print(path)
        if not os.path.isdir(path):
            raise ValueError("Path did not exist!")

In [ ]:
os.makedirs(save_path,exist_ok=True)

print(save_path)

##### Plot

In [ ]:
capsize = 5

scatter_join_bool = True
# scatter_join_bool = False

In [ ]:
number_bool = True
# number_bool = False

# number_variations_bool = True
number_variations_bool = False

bar_log = True
# bar_log = False

if number_bool: # check there are actually any to be plotted
    
    any_number_bool = False
    for key in all_dicts:
        any_number_bool = any_number_bool or any(all_dicts[key]["number_bools"])
        
    if not any_number_bool:
        number_bool = False
        print("No plot with number bool to draw, setting number_bool=False")

In [ ]:
hard_coded_ylims_bool = True # currently needed to ensure the y axis is shared correctly
# hard_coded_ylims_bool = False

# symmetric_ylims_bool = True
symmetric_ylims_bool = False

hard_coded_ylims = {
#     "tilt_abs": [-45,3],
    "tilt_abs": [-45,45],
}

yshift_dict = {
    "tilt_abs": 2,
    "anisotropy": 0.02,
    "correlation": 0.01,
    "mean_vx": 1,
    "mean_vy": 1,
    "std_vx": 2,
    "std_vy": 2
}

In [ ]:
def get_legend_params(ncols, all_dicts,number_bool):
    
    if ncols != len(all_dicts):
        raise ValueError("Expected the `ncols` to match the length of `all_dicts`.")
    
    legend_cols = []
    for k in all_dicts.keys():
        if any(all_dicts[k]["labels"]):
            legend_cols.append(k)
    
    ncols_leg = len(np.unique(all_dicts[legend_cols[0]]["colors"])) if len(legend_cols) == 1 else 1
    
    if ncols == 2:
        loc_x = -0.75
    elif ncols == 3:
        loc_x = -0.6 if len(legend_cols) == 1 else 0.16
    elif ncols == 1:
        loc_x = -0.25
        
    legend_loc = [loc_x, 1.65]
    
    if not any(any(all_dicts[k]["number_bools"]) for k in all_dicts.keys()):
        legend_loc[1] -= 0.4
        
    if not any(all_dicts[k]["title"] for k in all_dicts.keys()):
        legend_loc[1] -= 0.15
    
    return legend_cols, legend_loc, ncols_leg
            
legend_row = 1
# legend_row = len(map_list)

legend_cols, legend_loc, ncols_leg = get_legend_params(ncols, all_dicts,number_bool)
# legend_loc = "upper right"; ncols_leg=1
# legend_cols = [1]

legend_bool = True
# legend_bool = False

if legend_bool: # check there are actually any labels to be plotted
    
    any_legend_bool = False
    for key in all_dicts:
        any_legend_bool = any_legend_bool or any(all_dicts[key]["labels"])
        
    if not any_legend_bool:
        legend_bool = False
        print("No plot with label, setting legend_bool=False")

In [ ]:
zero_line_color = "grey"
zero_line_alpha = 1

In [ ]:
plt.rcParams["font.size"] = 22
ylabel_size = 24
legend_fontsize = 15.5

In [ ]:
figsize_x = 15

figsize_y = 1.41*figsize_x # aspect ratio of A4

figsize_x /= 2 if ncols == 1 else 1

In [ ]:
map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy","correlation","tilt_abs"]; map_list_string = "all"
# map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "meanstdani"
# map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
# map_list = ["mean_vx","mean_vy"]; map_list_string = "velmeans"

In [ ]:
filename_suffix = ""

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

fig,axs = plt.subplots(figsize=(figsize_x,figsize_y),nrows=len(map_list)+1,ncols=ncols, facecolor='w',\
                       gridspec_kw={'hspace':0,'wspace':0,'height_ratios':[0.35]+len(map_list)*[1]})

if number_bool: # number and title
    bar_n_min,bar_n_max = [10**30],[0]

    for col in all_dicts:
        dic = all_dicts[col]

        axs[0,col].set_title(dic["title"])

        for i in range(len(dic["load_paths"])):
            if dic["number_bools"][i]:
                map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
                
                if dic["plot_ranges_str"][i] == "mean":
                    plot_range = PH.get_range_means(min_range,max_range)

                POPPLOT_number_bar(axs[0,col], plot_range, map_dict["number"],color=dic["colors"][i],alpha=dic["number_alphas"][i],\
                                   zorder=dic["zorderNs"][i], bar_width=dic["bar_width"])

                bar_n_min = min(bar_n_min, np.min(map_dict["number"]))
                bar_n_max = max(bar_n_max, np.max(map_dict["number"]))
    
    for col in all_dicts:
        POPPLOT_number_bar_axis_settings(axs[0,col],min_n=bar_n_min,max_n=bar_n_max,bar_log=bar_log,labels_on=col==ncols-1)        
else:
    for col in all_dicts:
        fig.delaxes(axs[0,col])
        
        axs[1,col].set_title(all_dicts[col])

for row,map_string in enumerate(map_list): # plot
    error_string = map_string+"_error"
    
    ymin,ymax = [float("inf")],[float("-inf")]
    
    for col in all_dicts:
        
        POPPLOT_yaxis_settings(axs[row+1,col],map_string,error_string,set_ylims=False,labels_on=col==0)
        axs[row+1,col].axhline(y=0,linestyle='--',color=zero_line_color,alpha=zero_line_alpha,zorder=0)
        
        dic = all_dicts[col]
        for i in range(len(dic["load_paths"])):
            
            map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
            plot_range = plot_range if dic["plot_ranges_str"][i]=="median" else PH.get_range_means(min_range,max_range)
            label = dic["labels"][i] if row == legend_row-1 else None
            
            if dic["surface_bools"][i]:
                POPPLOT_values_surface(axs[row+1,col],map_dict[map_string],map_dict[error_string],plot_range,color=dic["colors"][i],label=label,\
                                       line_alpha=dic["line_alphas"][i],surface_alpha=dic["surface_alphas"][i],zorder=dic["zorders"][i])
            else:
                POPPLOT_values_scatter(axs[row+1,col],map_dict[map_string],map_dict[error_string],plot_range,min_range,max_range,color=dic["colors"][i],label=label,\
                                       line_alpha=dic["line_alphas"][i],zorder=dic["zorders"][i],lines_bool=scatter_join_bool)
            
            ymin = min(ymin, np.nanmin(map_dict[map_string]-map_dict[error_string]))
            ymax = max(ymax, np.nanmax(map_dict[map_string]+map_dict[error_string]))
    
    if map_string in yshift_dict:
        ymin -= yshift_dict[map_string]
        ymax += yshift_dict[map_string]
    
    if hard_coded_ylims_bool and map_string in hard_coded_ylims:
        ymin,ymax = hard_coded_ylims[map_string]
    
    for col in all_dicts:
        axs[row+1,col].set_ylim(ymin,ymax)

if legend_bool: # legend
    for col in all_dicts:
        if col in legend_cols:
            axs[legend_row,col].legend(loc=legend_loc,fontsize=legend_fontsize,ncols=ncols_leg)#,loc="lower left")
    
if True: # x-axis
        
    for col in all_dicts:
        
        dic = all_dicts[col]
    
        for row in range(len(map_list)+1): # include number bars

            POPPLOT_xaxis_settings(axs[row,col],xmin=dic["x_lims"][0],xmax=dic["x_lims"][1],xlabel=dic["xaxis_label"],xticks=dic["x_ticks"],labels_on=row==len(map_list))

            if dic["invert_xaxis"]:
                axs[row,col].invert_xaxis()
    
    fig.align_labels()
    
if True: # save
    
    filename = "kinpop_" + map_list_string
    
    filename += "_noN" if not number_bool else ""
    
    filename += "_noNvar" if number_bool and not number_variations_bool else ""
    
    filename += "_noLeg" if not legend_bool else ""
    
    if not scatter_join_bool:
        filename += "_noLines"
    
    filename += filename_suffix
    
    print(filename)
    
    if save_bool:
        print("Saving in",save_path)
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

## Older plots

color_dict = {
    "vertex_abs" : ["blue", "red"],
    "vertex" : ["orangered", "deepskyblue"]
}

### Single

In [ ]:
map_string = "tilt_abs"
# map_string = "correlation"
# map_string = "anisotropy"
# map_string = 'mean_vx'

error_string = map_string+"_error"

In [ ]:
# legend_bool = True
legend_bool = True if map_string == "anisotropy" else False
# legend_bool = False

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot
map_symbol = symbol_dict[map_string]
# map_title = title_dict[map_string]

fig, ax = plt.subplots(figsize=(10,10))

divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)

if sim_bool:
    sim_map_array = map_dict[map_string]
    sim_error_array = map_dict[map_string+"_error"]
    
    if number_bool:
        LATPLOT_number_bars_sim(map_dict["number"], sim_b_range_plot,alpha=0.5)
    
    LATPLOT_values_sim(sim_map_array, sim_error_array, sim_b_range_plot,alpha=0.4,legend=True)

if data_bool:
    o_map_array = o_map_dict[map_string]
    o_error_array = o_map_dict[error_string]
        
    if number_bool:
        LATPLOT_number_bars_data(o_map_dict["number"], o_b_range_plot)
    
    LATPLOT_values_data(o_map_array, o_error_array, o_b_range_min, o_b_range_max, o_b_range_plot,legend=True)

ax.plot([bmin,bmax],[0,0],color='black',linestyle='dotted')

LATPLOT_number_bars_axis_settings()
LATPLOT_xaxis_settings()
LATPLOT_yaxis_settings(map_string)

if True: # save
    
    legend_string = '' if legend_bool else '_noLeg'
    
    filename = map_string + legend_string
    print(save_path+filename)
    
    if save_bool:
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

### Grouped

In [ ]:
# map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
# error_list = [m+"_error" for m in map_list]

# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# error_list = ["std_vx","std_vy","zero","zero"]; map_list_string += "err"
# error_list = ["zero","zero","zero","zero"]

# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "velmeanstdani"
# error_list = ["zero","zero","zero","zero","anisotropy_error"]

map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
error_list = ["zero","zero","anisotropy_error"]

# map_list = ["mean_vx","mean_vy"]; map_list_string = "meanvel"
# error_list = ["std_vx","std_vy"]; map_list_string += "err"
# error_list = ["zero","zero"]

# map_list = ["mean_vx","mean_vy","anisotropy"]; map_list_string = "meanvelerrani"
# error_list = ["std_vx","std_vy","anisotropy_error"]

In [ ]:
# legend_row = 0
legend_row = len(map_list)-1
# legend_row = 999

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

nrows = len(map_list)+1

x_figsize = 10

fig, axs = plt.subplots(figsize=(x_figsize,len(map_list)*x_figsize),ncols=1,nrows=nrows,sharex=True,gridspec_kw={"height_ratios":[0.2]+[1]*(nrows-1),"hspace":0})

ax_histx = axs[0]

if number_bool:
    if sim_bool:
        LATPLOT_number_bars_sim(map_dict["number"], sim_b_range_plot,alpha=0.5)
    if data_bool:
        LATPLOT_number_bars_data(o_map_dict["number"], o_b_range_plot)
        
    LATPLOT_number_bars_axis_settings()

for row in range(nrows - 1):
    ax = axs[row+1]
    map_string = map_list[row]
    error_string = error_list[row]
    
    legend_bool = row == legend_row
    
    map_symbol = symbol_dict[map_string]
    # map_title = title_dict[map_string]

    if sim_bool:
        sim_map_array = map_dict[map_string]
        sim_error_array = map_dict[error_string]

        LATPLOT_values_sim(sim_map_array, sim_error_array, sim_b_range_plot,alpha=0.4,legend=True)

    if data_bool:
        o_map_array = o_map_dict[map_string]
        o_error_array = o_map_dict[error_string]

        LATPLOT_values_data(o_map_array, o_error_array, o_b_range_min, o_b_range_max, o_b_range_plot,legend=True)
    
    if True: # zero line
        minima = [np.nanmin(sim_map_array-sim_error_array),np.nanmin(o_map_array-o_error_array)]
        maxima = [np.nanmax(sim_map_array+sim_error_array),np.nanmax(o_map_array+o_error_array)]
        if PH.shall_plot_zero_line(minima,maxima):
            ax.plot([bmin,bmax],[0,0],color='black',linestyle='dotted')
    
    LATPLOT_yaxis_settings(map_string)
    LATPLOT_legend_and_text(legend_bool, extra_string_bool)

LATPLOT_xaxis_settings()

if True: # save
    
    xerr_string = f"_xerr{xerr_frac}" if x_error_data_bool else ""
    
    filename = filename_prefix + xerr_string
    print(save_path+filename)
    
    if save_bool:
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

### Shadowed

The way to use this is:
* Run all the code above for the variations you want, in order to save the arrays
* Before producing the plot with shadows, run all the code above for the variation you'd like to show as main plot (i.e. the brightest ones)
* The code below will load the shadows and create the plot

#### Load

In [ ]:
# Load

current_var = "R"
variation_var = "R"

variations_dict = {
    "Nbins": [[4,6],[5,7]], # [:,0] and [:,1] correspond to data and sim respectively
    "l": [[-2,2],[-1.5,1.5],[-2.5,2.5]],
    "R": [[0,3.5],[0,2]],
    "d": [[6.1,10.1]]
}

def get_variable_path_str(variable,var_tuple):
    if variable == "Nbins":
        return f"{var_tuple[0]}_points_data/{var_tuple[1]}_points_sim/"
    elif variable == "R" or variable == "d":
        return f"extra_{variable}/{var_tuple[0]}{variable}{var_tuple[1]}/"

def get_variation_results(variable, var_tuple, path, array_prefix="",range_prefix="pop_", deconstructed=True, verbose=False):
    """
    Parameters
    ----------
    variable: string
        variable to use
    var_tuple: tuple 
        values of variation
    path: string
    deconstructed: boolean
        if True, `path` is a tuple with the deconstructed save path portions to be completed with the variation values
        if False, `path` is the complete save path with the variation values already added in
        
    Returns
    -------
    map_dict: dict 
        Contains a 1D array entry per kinematic variable in `full_map_string_list`
    range_min: 1D array
    range_max: 1D array
    range_plot: 1D array
    """
    
    if deconstructed:
        var_string = get_variable_path_str(variable,var_tuple)

        variation_path = path[0] + var_string + path[1]
    else:
        variation_path = path
    
    variation_arrays_path = variation_path + "arrays/"
    
    if verbose:
        print("Loading from\n",variation_arrays_path)
    
    map_dict = {}
    
    for m in full_map_string_list:
        map_dict[m] = np.load(f"{variation_arrays_path}{array_prefix}{m}.npy")
    
    min_range = np.load(variation_arrays_path + f"{range_prefix}range_min.npy")
    max_range = np.load(variation_arrays_path + f"{range_prefix}range_max.npy")
    plot_range = np.load(variation_arrays_path + f"{range_prefix}range_plot.npy")
    
    return map_dict, min_range, max_range, plot_range

current_string_tuples_dict = {
    "Npoints": [n_points_data,n_points_sim],
    "l": [lmin,lmax],
    extra_variable: [extra_min,extra_max]
}

current_tuple = current_string_tuples_dict[current_var]

current_string = get_variable_path_str(current_var,current_tuple)

def get_save_path_variation():
    
    variation_string_dict = {
        "Npoints": "Nbins_variation",
        "l": "l_variation",
        "R": "R_variation",
        "d": "d_variation"
    }
    
    save_path_variation = get_save_path() + variation_string_dict[variation_var] + "/"
    MF.create_dir(save_path_variation)
    
    return save_path_variation

save_path_variation = get_save_path_variation()
print("Saving in:",save_path_variation)

In [ ]:
current_save_path = get_save_path()
deconstructed_path = current_save_path.split(current_string)

variations_list = variations_dict[variation_var]

if current_var == variation_var and current_tuple in variations_list:
    variations_list.remove(current_tuple)

In [ ]:
# variations

variations_list = variations_dict[variation_var]

sim_dict_of_map_dicts,sim_b_range_min_dict,sim_b_range_max_dict,sim_b_range_plot_dict = {},{},{},{}
o_dict_of_map_dicts,o_b_range_min_dict,o_b_range_max_dict,o_b_range_plot_dict = {},{},{},{}

for var_tuple in variations_list:
    
    var_key = f"{var_tuple[0]},{var_tuple[1]}"
    print(var_key)
    
    map_sim,min_sim,max_sim,plot_sim = get_variation_results(variation_var,var_tuple,deconstructed_path,array_prefix="sim_",range_prefix="sim_b_",verbose=True)
    map_data,min_data,max_data,plot_data = get_variation_results(variation_var,var_tuple,deconstructed_path,array_prefix="o_",range_prefix="o_b_",verbose=False)
    
    sim_dict_of_map_dicts[var_key] = map_sim
    o_dict_of_map_dicts[var_key] = map_data
    
    for d,v in zip([sim_b_range_min_dict,sim_b_range_max_dict,sim_b_range_plot_dict],[min_sim,max_sim,plot_sim]):
        d[var_key] = v
    
    for d,v in zip([o_b_range_min_dict,o_b_range_max_dict,o_b_range_plot_dict],[min_data,max_data,plot_data]):
        d[var_key] = v

In [ ]:
for variation in variations_list:
    variation_key = f"{variation[0]},{variation[1]}"
        
    PH.add_zero_array_key(sim_dict_of_map_dicts[variation_key])
    PH.add_zero_array_key(o_dict_of_map_dicts[variation_key])

#### Plot

In [ ]:
def set_new_colors(plot_idx=0, totalN = 256):
    mid_idx = totalN//2
    
    frac = (1/3)**(plot_idx)
    
    young_idx = int(mid_idx * (1-frac))
    old_idx = totalN - young_idx
    
    global color_y, color_o
    color_y,color_o = coolwarm(young_idx),coolwarm(old_idx)

In [ ]:
number_variations_bool = True
# number_variations_bool = False

current_first = True
# current_first = False

In [ ]:
main_alpha = 0.75
alpha_reduction_factor = 1

# variation_legend_bool = True # not currently implemented
variation_legend_bool = False

##### Group

In [ ]:
map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
error_list = [m+"_error" for m in map_list]

# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# error_list = ["zero","zero","zero","zero"]
# error_list = ["std_vx","std_vy","zero","zero"]; map_list_string += "err"

# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "velmeanstdani"
# error_list = ["zero","zero","zero","zero","anisotropy_error"]

# map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
# error_list = ["zero","zero","anisotropy_error"]

# map_list = ["mean_vx","mean_vy"]; map_list_string = "meanvel"
# error_list = ["std_vx","std_vy"]; map_list_string += "err"
# error_list = ["zero","zero"]

# map_list = ["mean_vx","mean_vy","anisotropy"]; map_list_string = "meanvelerrani"
# error_list = ["std_vx","std_vy","anisotropy_error"]

In [ ]:
legend_row = 0
# legend_row = len(map_list)-1
# legend_row = 999

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

x_figsize = 10

fig, axs = plt.subplots(figsize=(x_figsize,len(map_list)*x_figsize),ncols=1,nrows=len(map_list)+1,sharex=True,gridspec_kw={"height_ratios":[0.2]+[1]*len(map_list),"hspace":0})

ax_histx = axs[0]

if number_bool:
    alpha = main_alpha
    plot_idx = 0
    set_new_colors(plot_idx)
    
    if sim_bool:
        LATPLOT_number_bars_sim(map_dict["number"], sim_b_range_plot,alpha=alpha,zorder=10 if current_first else None)
    if data_bool:
        LATPLOT_number_bars_data(o_map_dict["number"], o_b_range_plot,alpha=alpha,zorder=10 if current_first else None)
        
    if number_variations_bool:
        
        for k in sim_dict_of_map_dicts: # assumes the same variation are applied to model and data
            alpha *= alpha_reduction_factor
            plot_idx += 1
            set_new_colors(plot_idx)

            if sim_bool:
                variation_sim_map_dict = sim_dict_of_map_dicts[k]
                variation_sim_b_range_plot = sim_b_range_plot_dict[k]
                
                LATPLOT_number_bars_sim(variation_sim_map_dict["number"],variation_sim_b_range_plot)#,alpha=alpha)
                
            if data_bool:
                variation_o_map_dict = o_dict_of_map_dicts[k]
                variation_o_b_range_plot = o_b_range_plot_dict[k]
                
                LATPLOT_number_bars_data(variation_o_map_dict["number"],variation_o_b_range_plot)#,alpha=alpha)
        
    LATPLOT_number_bars_axis_settings()

for row in range(len(map_list)):
    ax = axs[row+1]
    map_string = map_list[row]
    error_string = error_list[row]
    
    alpha = main_alpha
    plot_idx = 0
    set_new_colors(plot_idx)
    
    legend_bool = row == legend_row
    
    map_symbol = symbol_dict[map_string]
    # map_title = title_dict[map_string]

    if sim_bool:
        LATPLOT_values_sim(map_dict[map_string], map_dict[error_string], sim_b_range_plot,alpha=alpha,zorder=10 if current_first else None,legend=True)
    if data_bool:
        LATPLOT_values_data(o_map_dict[map_string], o_map_dict[error_string], o_b_range_min, o_b_range_max, o_b_range_plot,\
                            zorder=10 if current_first else None,legend=True)
        
    for k in sim_dict_of_map_dicts: # assumes the same variations are applied to model and data
        alpha *= alpha_reduction_factor
        plot_idx += 1
        set_new_colors(plot_idx)

        if sim_bool:
            variation_sim_map_dict = sim_dict_of_map_dicts[k]

            LATPLOT_values_sim(variation_sim_map_dict[map_string],variation_sim_map_dict[error_string],sim_b_range_plot_dict[k],alpha=alpha)

        if data_bool:
            variation_o_map_dict = o_dict_of_map_dicts[k]

            LATPLOT_values_data(variation_o_map_dict[map_string],variation_o_map_dict[error_string],\
                                o_b_range_min_dict[k],o_b_range_max_dict[k],o_b_range_plot_dict[k])

    if True: # zero line
        minima = [np.nanmin(d[map_string]-d[error_string]) for d in [map_dict,o_map_dict,variation_sim_map_dict,variation_o_map_dict]]
        maxima = [np.nanmax(d[map_string]+d[error_string]) for d in [map_dict,o_map_dict,variation_sim_map_dict,variation_o_map_dict]]
        
        if PH.shall_plot_zero_line(minima,maxima):
            ax.plot([bmin,bmax],[0,0],color='black',linestyle='dotted')
    
    LATPLOT_yaxis_settings(map_string)
    LATPLOT_legend_and_text(legend_bool = row == legend_row, extra_string_bool=extra_string_bool)

LATPLOT_xaxis_settings()

if True: # save
    
    variation_string = f"_{variation_var}Var"
    for var_list in variations_list:
        variation_string += f"_{var_list[0]}-{var_list[1]}"
        
    number_variation_string = "_noNvar" if not number_variations_bool else ""
    legend_string = "_noLeg" if legend_row >= len(map_string) else ""
    variation_legend_string = "_varLegend" if variation_legend_bool else ""
    
    filename = "lat_" + map_list_string + variation_string + number_variation_string + legend_string + variation_legend_string
    print(save_path_variation+filename)
    
    if save_bool:
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path_variation+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

##### Single

In [ ]:
# save_bool = True
save_bool = False

In [ ]:
# plot
map_symbol = symbol_dict[map_string]
# map_title = title_dict[map_string]

fig, ax = plt.subplots(figsize=(10,10))

divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)

if sim_bool:
    sim_map_array = map_dict[map_string]
    sim_error_array = map_dict[map_string+"_error"]
    
    alpha = 0.75
    plot_idx = 0
    set_new_colors(plot_idx)
    
    if number_bool:
        LATPLOT_number_bars_sim(map_dict["number"], sim_b_range_plot, alpha=alpha, zorder=10 if current_first else None)
    LATPLOT_values_sim(sim_map_array, sim_error_array, sim_b_range_plot, alpha=alpha, legend=True, zorder=10 if current_first else None)
    
    for k in sim_dict_of_map_dicts:
        alpha *= alpha_reduction_factor
        plot_idx += 1
        set_new_colors(plot_idx)
        
        variation_sim_map_dict = sim_dict_of_map_dicts[k]
        variation_sim_b_range_plot = sim_b_range_plot_dict[k]
        
        if number_variations_bool:
            LATPLOT_number_bars_sim(variation_sim_map_dict["number"],variation_sim_b_range_plot,alpha)
        LATPLOT_values_sim(variation_sim_map_dict[map_string],variation_sim_map_dict[error_string],variation_sim_b_range_plot,alpha)

if data_bool:
    o_map_array = o_map_dict[map_string]
    o_error_array = o_map_dict[error_string]
    
    alpha = 0.75
    plot_idx = 0
    set_new_colors(plot_idx)
    
    if number_bool:        
        LATPLOT_number_bars_data(o_map_dict["number"], o_b_range_plot,alpha=alpha,zorder=11 if current_first else None)
    LATPLOT_values_data(o_map_array, o_error_array, o_b_range_min, o_b_range_max, o_b_range_plot,alpha=alpha,legend=True,zorder=11 if current_first else None)
    
    for k in o_dict_of_map_dicts:
        alpha *= alpha_reduction_factor
        plot_idx += 1
        set_new_colors(plot_idx)
        
        variation_o_map_dict = o_dict_of_map_dicts[k]
        variation_o_b_range_plot = o_b_range_plot_dict[k]
        variation_o_b_range_min = o_b_range_min_dict[k]
        variation_o_b_range_max = o_b_range_max_dict[k]
                
        if number_variations_bool:
            LATPLOT_number_bars_data(variation_o_map_dict["number"],variation_o_b_range_plot,alpha=alpha)
        LATPLOT_values_data(variation_o_map_dict[map_string],variation_o_map_dict[error_string],variation_o_b_range_min,variation_o_b_range_max,variation_o_b_range_plot,alpha=alpha)

ax.plot([bmin,bmax],[0,0],color='black',linestyle='dotted')

LATPLOT_number_bars_axis_settings()
LATPLOT_xaxis_settings()
LATPLOT_yaxis_settings()

if True: # save
    
    xerr_string = f"_xerr{xerr_frac}" if x_error_data_bool and xerr_frac!=2 else ""
    
    variation_string = f"_{variation_var}Var"
    for var_list in variations_list:
        variation_string += f"_{var_list[0]}-{var_list[1]}"
        
    number_variation_string = "_nBarPlotVar" if number_variations_bool else ""
    
    outlier_string = '_NoOutlier' if map_string == "anisotropy" and not outlier_bool else ''
    legend_string = '' if legend_bool else '_noLeg'
    
    filename = map_string + variation_string + number_variation_string + xerr_string + outlier_string + legend_string
    print(save_path_variation+filename)
    
    if save_bool:
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path_variation+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

#### Illustrate bulge cuts variation

Currently only implemented for radius variation (and hard-coded)

In [ ]:
save_bool = True
# save_bool = False

if True: # illustrate bulge cut variation
    
    fig,axs=plt.subplots(figsize=(15,10),nrows=2,sharex=True,gridspec_kw={"hspace":-0.43})
    
    MP.visualise_bulge_selection(given_axs=axs,cuts={"lmax":lmax,"bmax":bmax,"dmax":10.1,"R0":R0},y_max_plot=3)

#     MP.plot_circle(radius=2.5,ax=axs[0],linestyle="--",label=r"$R_\mathrm{GC}\leq2.5$ kpc")
#     axs[1].axvline(-2.5,color="k",linestyle="--")
#     axs[1].axvline(2.5,color="k",linestyle="--")

    axs[0].scatter(pd.concat(df_metals)["x"],pd.concat(df_metals)["y"],color="grey",s=1)
    axs[1].scatter(pd.concat(df_metals)["x"],pd.concat(df_metals)["z"],color="grey",s=1)
    
    for m,M in zip(o_b_range_min,o_b_range_max):
        MP.plot_angled_line(axs[1],xmin=-R0,ymin=0,xmax=4,angle=m,color="red",linestyle="--")
        MP.plot_angled_line(axs[1],xmin=-R0,ymin=0,xmax=4,angle=M,color="blue",linestyle="--")

    _ = [ax.legend(fontsize="x-small") for ax in axs]
    
    fig.delaxes(axs[0])
    
    axs[1].set_xlim(-R0-0.1)

    filename = "illustrate_cuts_data"
    print(filename)
    if save_bool:
        print("Saving in:",save_path)
        plt.savefig(save_path+filename+".png",dpi=200,bbox_inches="tight")

In [ ]:
# young_bool = True
young_bool = False

save_bool = True
# save_bool = False

if True: # illustrate bulge cut variation for the simulation
    
    fig,axs=plt.subplots(figsize=(15,10),nrows=2,sharex=True,gridspec_kw={"hspace":-0.43})
    
    if young_bool:
        df = df0[(df0["age"]>=young_min)&(df0["age"]<=young_max)]
    else:
        df = df0[(df0["age"]>=old_min)&(df0["age"]<=old_max)]
        
    MP.quick_show_xy_xz(df=df,bmin=bmin,axs=axs)
    
    MP.visualise_bulge_selection(given_axs=axs,cuts={"lmax":lmax,"bmax":bmax,"dmax":10.1,"R0":R0},y_max_plot=3)

#     MP.plot_circle(radius=2.5,ax=axs[0],linestyle="--",label=r"$R_\mathrm{GC}\leq2.5$ kpc")
#     axs[1].axvline(-2.5,color="k",linestyle="--")
#     axs[1].axvline(2.5,color="k",linestyle="--")
    
    for m,M in zip(sim_b_range_min,sim_b_range_max):
        MP.plot_angled_line(axs[1],xmin=-R0,ymin=0,xmax=4,angle=m,color="red",linestyle="--")
        MP.plot_angled_line(axs[1],xmin=-R0,ymin=0,xmax=4,angle=M,color="blue",linestyle="--")

    _ = [ax.legend(fontsize="x-small") for ax in axs]
    
    fig.delaxes(axs[0])
    
    axs[1].set_xlim(-R0-0.1)
    
    filename = "illustrate_cut_sim%s"%("Young" if young_bool else "Old")

    print(filename)
    if save_bool:
        print("Saving in:",save_path)
        plt.savefig(save_path+filename+".png",dpi=200,bbox_inches="tight")
        
    del df

### Even older

#### Plot pairs

##### Standard & abs

In [ ]:
color_dict_comp = {
    0 : ["blue", "red"],
    1 : ["deepskyblue", "gold"]
}

In [ ]:
transparency = 0.5
alpha_area = 0.4

fig, ax = plt.subplots()
divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)
ax_histx.xaxis.set_tick_params(labelbottom=False)

map_string="vertex"
error_string = "vertex_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string]
map_symbol = symbol_dict["vertex"]
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)
label_y = r"$5-9.8$ Gyr"
label_o = r"$9.8-10$ Gyr"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

bar_width=0.2
ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha_area, facecolor=color_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha_area, facecolor=color_o)
ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha_area, facecolor=color_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha_area, facecolor=color_o)
ax.set_yticks(yticks)
ax.set_ylim(min(yticks),max(yticks))


map_string = "vertex_abs"
error_string = "vertex_error_abs"
map_array = map_dict[map_string]
error_array = map_dict[error_string]
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$5-9.8$ Gyr abs"
label_o = r"$9.8-10$ Gyr abs"

bar_width=0.2
ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

handles, labels = ax.get_legend_handles_labels()
# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, labels, loc="upper left")

ax_histx.set_yticks([100,1000,10000,100000])
ax_histx.set_ylabel(r"$N$",labelpad=5)
ax_histx.yaxis.set_label_position("right")
ax.yaxis.set_minor_locator(AutoMinorLocator(2))
ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
ax.set_xlim(0,-10)
ax.set_xticks(np.arange(0,-10-1,-1))

ax.set_xlabel(r"$b$ [°]")
ax.set_ylabel(map_symbol)

ax.set_aspect('auto')
plt.savefig(save_path+"vertex_comparison.png",bbox_inches='tight',dpi=300)

plt.show()

##### velocities

In [ ]:
transparency = 0.5
alpha_area = 0.4
error_factor = 100

fig, ax = plt.subplots()
divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)
ax_histx.xaxis.set_tick_params(labelbottom=False)

map_string="mean_vr"
error_string = "varr"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$\langle v_r \rangle$ $5-9.8$ Gyr"
label_o = r"$\langle v_r \rangle$ $9.8-10$ Gyr"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

bar_width=0.2
ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)


map_string = "mean_vl"
error_string = "varl"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\langle v_l \rangle$ $5-9.8$ Gyr"
label_o = r"$\langle v_l \rangle$ $9.8-10$ Gyr"

ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
ax.legend(loc='lower right',ncol=2,fontsize=16)

ax_histx.set_yticks([100,1000,10000,100000])
ax_histx.set_ylabel(r"$N$",labelpad=5)
ax_histx.yaxis.set_label_position("right")
ax.yaxis.set_minor_locator(AutoMinorLocator(2))
ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
ax.set_xlim(0,-10)
ax.set_xticks(np.arange(0,-10-1,-1))

ylim = 26
yticks = np.linspace(-ylim,ylim,5)
ax.set_yticks(yticks)
ax.set_ylim(min(yticks),max(yticks))

ax.set_xlabel(r"$b$ [°]")
ax.set_ylabel("Mean velocity")

ax.set_aspect('auto')

if error_factor != 1:
    ax.text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor))

error_factor_string = '_error'+str(error_factor) if error_factor != 1 else ''
plt.savefig(save_path+f"mean_velocities{error_factor_string}.png",bbox_inches='tight',dpi=300)
plt.show()

##### corr & anis

In [ ]:
o_b_range_max

In [ ]:
color_dict_comp = {
    0 : ["blue", "red"],
    1 : ["deepskyblue", "gold"]
}

In [ ]:
transparency = 0.5
alpha_area = 0.4
error_factor = 10

fig, ax = plt.subplots()
divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)
ax_histx.xaxis.set_tick_params(labelbottom=False)

map_string="anisotropy"
error_string = "anisotropy_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$1-\sigma_{l}/\sigma_r$ ($5-9.8$ Gyr)"
label_o = r"$1-\sigma_{l}/\sigma_r$ ($9.8-10$ Gyr)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

bar_width=0.2
ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#OBSERVATIONS-----------------------------------------------------------------------------------------------
o_map_array = o_map_dict[map_string]
o_error_array = o_map_dict[error_string]

#ax.errorbar(o_b_range_plot, o_map_array[:,0], yerr= o_error_array[:,0], color=color_y, fmt='o')
#ax.errorbar(o_b_range_plot, o_map_array[:,1] , yerr= o_error_array[:,1], color=color_o, fmt='o')
ax.scatter(o_b_range_plot, o_map_array[:,0], color=color_y)
ax.scatter(o_b_range_plot, o_map_array[:,1] , color=color_o)

#------------------------------------------------------------------------------------------------------------

map_string = "correlation"
error_string = "correlation_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\rho_{rl}$ ($5-9.8$ Gyr)"
label_o = r"$\rho_{rl}$ ($9.8-10$ Gyr)"

ax_histx.bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)

ax.plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
ax.plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

ax.fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
ax.fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#OBSERVATIONS-----------------------------------------------------------------------------------------------
o_map_array = o_map_dict[map_string]
o_error_array = o_map_dict[error_string]

ax.errorbar(o_b_range_plot, o_map_array[:,0], yerr= o_error_array[:,0], color=color_y, fmt='o')
ax.errorbar(o_b_range_plot, o_map_array[:,1] , yerr= o_error_array[:,1], color=color_o, fmt='o')

#------------------------------------------------------------------------------------------------------------

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
ax.legend(loc='best')

ax_histx.set_yticks([100,1000,10000,100000])
ax_histx.set_ylabel(r"$N$",labelpad=15,rotation=0)
ax_histx.yaxis.set_label_position("right")
ax.yaxis.set_minor_locator(AutoMinorLocator(2))
ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
ax.set_xlim(0,-10)
ax.set_xticks(np.arange(0,-10-1,-1))

ax.set_ylim(-1,1)

ax.set_xlabel(r"$b$ [°]")
ax.set_ylabel("Anisotropy & correlation")
#ax.set_yticks(np.array(yticks))
#ax.set_ylim(min(yticks),max(yticks))

ax.set_aspect('auto')

x_error = -0.5
if error_factor != 1:
    ax.text(x_error,-0.85,r"Error $\times 10^{-%i}$" % get_exponent(error_factor), size=15)#error_fontsize)

error_factor_string = '_error'+str(error_factor) if error_factor != 1 else ''
plt.savefig(save_path+f"corr_anis{error_factor_string}.png",bbox_inches='tight',dpi=300)
plt.show()

#### Grouped

##### vertex,anicorr,vel

In [ ]:
transparency = 0.5
alpha_area = 0.4
legend_fontsize = 15
legend_ncol = 2
error_fontsize=15

fig, axs = plt.subplots(4,1,sharex=True,figsize=(7,16),gridspec_kw={"hspace":0, "height_ratios":[0.3,1,1,1]})

for ax in axs:
    if ax.is_last_row():
        ax.set_xlabel(r"$b$ [°]")
    else:
        ax.xaxis.set_tick_params(labelbottom=False)
    ax.set_aspect('auto')
    
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
    ax.set_xlim(0,-10)
    ax.set_xticks(np.arange(0,-10-1,-1))

#------------------------------------------------------------------------------------------------------------------
error_factor_vert = 1

map_string="vertex"
error_string = "vertex_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)
label_y = r"$l_{\mathrm{v}}$ (Y)"
label_o = r"$l_{\mathrm{v}}$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha_area, facecolor=color_o)
axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha_area, facecolor=color_o)
axs[1].set_yticks(yticks)
axs[1].set_ylim(min(yticks),max(yticks))

map_string = "vertex_abs"
error_string = "vertex_error_abs"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (Y)"
label_o = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (O)"

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#handles, labels = axs[1].get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#axs[1].legend(handles, labels, loc="upper center", fontsize=legend_fontsize)
axs[1].legend(loc="upper left", fontsize=legend_fontsize,ncol=legend_ncol)

if error_factor_vert != 1:
    if error_factor_vert >= 10:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_vert), size=error_fontsize)
    elif error_factor_vert == 2:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 0.5$", size=error_fontsize)
    else:
        raise ValueError("Unknown error factor in vertex")
        
axs[1].set_ylabel("Vertex deviation")
    
#-------------------------------------------------------------------------------------------------------------------

error_factor_anicorr = 10

map_string="anisotropy"
error_string = "anisotropy_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_anicorr
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$1-\sigma_{l}^2/\sigma_r^2$ (Y)"
label_o = r"$1-\sigma_{l}^2/\sigma_r^2$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

map_string = "correlation"
error_string = "correlation_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_anicorr
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\rho_{rl}$ (Y)"
label_o = r"$\rho_{rl}$ (O)"

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[2].set_ylabel("Anisotropy & correlation")

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
axs[2].legend(loc='upper right', fontsize=legend_fontsize,ncol=legend_ncol)

yticks = np.linspace(-0.4,0.4,5)
axs[2].set_yticks(yticks[:-1])
axs[2].set_ylim(min(yticks),max(yticks))

axs[2].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_anicorr), size=error_fontsize)

#------------------------------------------------------------------------------------------------------------------

error_factor_vel = 100

map_string="mean_vr"
error_string = "varr"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_vel
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$\langle v_r \rangle$ (Y)"
label_o = r"$\langle v_r \rangle$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[3].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[3].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[3].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[3].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

map_string = "mean_vl"
error_string = "varl"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_vel
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\langle v_l \rangle$ (Y)"
label_o = r"$\langle v_l \rangle$ (O)"

axs[3].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[3].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[3].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[3].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[3].set_ylabel("Mean velocity")

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
axs[3].legend(loc='lower right', fontsize=legend_fontsize,ncol=legend_ncol)
ylim = 26
yticks = np.linspace(-ylim,ylim,5)
axs[3].set_yticks(yticks[:-1])
axs[3].set_ylim(min(yticks),max(yticks))

axs[3].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_vel), size=error_fontsize)

#------------------------------------------------------------------------------------------------------------------

bar_width=0.2
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][1])
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][1])
axs[0].yaxis.set_tick_params(labelleft=False,labelright=True)

axs[0].set_yticks([100,1000,10000,100000])
axs[0].set_ylabel(r"$N$",labelpad=13,rotation=0)
axs[0].yaxis.set_label_position("right")

axs[0].text(-7.4,3500,"Y: 5-9.8 Gyr\nO: 9.8-10 Gyr", size=error_fontsize)

error_string_vert = "_errorvert"+str(error_factor_vert) if error_factor_vert != 1 else ''
error_string_anicorr = '_erroranicorr'+str(error_factor_anicorr) if error_factor_anicorr != 1 else ''
error_string_vel = '_errorvel'+str(error_factor_vel) if error_factor_vel != 1 else ''
filename = "group"+error_string_vert+error_string_anicorr+error_string_vel+'.png'
plt.savefig(save_path+filename,bbox_inches='tight',dpi=300)
plt.show()

##### vertex,anicorr

In [ ]:
transparency = 0.5
alpha_area = 0.4
legend_fontsize = 15
error_fontsize=15

fig, axs = plt.subplots(3,1,sharex=True,figsize=(7,12),gridspec_kw={"hspace":0, "height_ratios":[0.3,1,1]})

for ax in axs:
    if ax.is_last_row():
        ax.set_xlabel(r"$b$ [°]")
    else:
        ax.xaxis.set_tick_params(labelbottom=False)
    ax.set_aspect('auto')
    
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
    ax.set_xlim(0,-10)
    ax.set_xticks(np.arange(0,-10-1,-1))

#------------------------------------------------------------------------------------------------------------------
error_factor_vert = 1

map_string="vertex"
error_string = "vertex_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)
label_y = r"$l_{\mathrm{v}}$ (Y)"
label_o = r"$l_{\mathrm{v}}$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha_area, facecolor=color_o)
axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha_area, facecolor=color_o)
axs[1].set_yticks(yticks)
axs[1].set_ylim(min(yticks),max(yticks))

map_string = "vertex_abs"
error_string = "vertex_error_abs"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (Y)"
label_o = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (O)"

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#handles, labels = axs[1].get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#axs[1].legend(handles, labels, loc="upper center", fontsize=legend_fontsize)
axs[1].legend(loc="upper left", fontsize=legend_fontsize, ncol=2)

if error_factor_vert != 1:
    if error_factor_vert >= 10:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_vert), size=error_fontsize)
    elif error_factor_vert == 2:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 0.5$", size=error_fontsize)
    else:
        raise ValueError("Unknown error factor in vertex")
        
axs[1].set_ylabel("Vertex deviation")
    
#-------------------------------------------------------------------------------------------------------------------

error_factor_anicorr = 10

map_string="anisotropy"
error_string = "anisotropy_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_anicorr
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$1-\sigma_{l}^2/\sigma_r^2$ (Y)"
label_o = r"$1-\sigma_{l}^2/\sigma_r^2$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

map_string = "correlation"
error_string = "correlation_error"
map_array = map_dict[map_string]
error_array = np.sqrt(map_dict[error_string])/error_factor_anicorr
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\rho_{rl}$ (Y)"
label_o = r"$\rho_{rl}$ (O)"

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[2].set_ylabel("Anisotropy & correlation")

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
axs[2].legend(loc='lower right', fontsize=legend_fontsize, ncol=2)

yticks = np.linspace(-0.4,0.4,5)
axs[2].set_yticks(yticks[:-1])
axs[2].set_ylim(min(yticks),max(yticks))

if error_factor_anicorr != 1:
    if error_factor_anicorr >= 10:
        axs[2].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_anicorr), size=error_fontsize)
    elif error_factor_anicorr == 2:
        axs[2].text(-0.5,yticks[1]*7/4,r"Error $\times 0.5$", size=error_fontsize)
    else:
        raise ValueError("Unknown error factor in anicorr")

#------------------------------------------------------------------------------------------------------------------

bar_width=0.2
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][1])
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][1])
axs[0].yaxis.set_tick_params(labelleft=False,labelright=True)

axs[0].set_yticks([100,1000,10000,100000])
axs[0].set_ylim(10,500000)
axs[0].set_ylabel(r"$N$",labelpad=13, rotation=0)
axs[0].yaxis.set_label_position("right")

axs[0].text(-7.4,3500,"Y: 5-9.8 Gyr\nO: 9.8-10 Gyr", size=error_fontsize)

error_string_vert = "_errorvert"+str(error_factor_vert) if error_factor_vert != 1 else ''
error_string_anicorr = '_erroranicorr'+str(error_factor_anicorr) if error_factor_anicorr != 1 else ''
filename = "group"+error_string_vert+error_string_anicorr+'.png'
plt.savefig(save_path+filename,bbox_inches='tight',dpi=300)
plt.show()

##### vertex,covdiff

In [ ]:
transparency = 0.5
alpha_area = 0.4
legend_fontsize = 15
error_fontsize=15

fig, axs = plt.subplots(3,1,sharex=True,figsize=(7,12),gridspec_kw={"hspace":0, "height_ratios":[0.3,1,1]})

for ax in axs:
    if ax.is_last_row():
        ax.set_xlabel(r"$b$ [°]")
    else:
        ax.xaxis.set_tick_params(labelbottom=False)
    ax.set_aspect('auto')
    
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    ax.tick_params(which='minor',direction='in',color='black',length=4)#,width=1)
    ax.set_xlim(0,-10)
    ax.set_xticks(np.arange(0,-10-1,-1))

#------------------------------------------------------------------------------------------------------------------
error_factor_vert = 1

map_string="vertex"
error_string = "vertex_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)
label_y = r"$l_{\mathrm{v}}$ (Y)"
label_o = r"$l_{\mathrm{v}}$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha_area, facecolor=color_o)
axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha_area, facecolor=color_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha_area, facecolor=color_o)
axs[1].set_yticks(yticks[1:])
axs[1].set_ylim(min(yticks),max(yticks))

map_string = "vertex_abs"
error_string = "vertex_error_abs"
map_array = map_dict[map_string]
error_array = map_dict[error_string]/error_factor_vert
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (Y)"
label_o = r"$l_{\mathrm{v}}^{\mathrm{abs}}$ (O)"

axs[1].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[1].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[1].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[1].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

#handles, labels = axs[1].get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#axs[1].legend(handles, labels, loc="upper center", fontsize=legend_fontsize)
axs[1].legend(loc="best", fontsize=legend_fontsize,ncol=2)

if error_factor_vert != 1:
    if error_factor_vert >= 10:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_vert), size=error_fontsize)
    elif error_factor_vert == 2:
        axs[1].text(-0.5,yticks[1]*7/4,r"Error $\times 0.5$", size=error_fontsize)
    else:
        raise ValueError("Unknown error factor in vertex")

axs[1].set_ylabel("Vertex deviation")
    
#-------------------------------------------------------------------------------------------------------------------

map_string="var_difference"
error_string = "var_difference_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string] #/error_factor_anicorr
yticks = get_variable_ticks(map_string, map_array)
label_y = r"$\sigma_r^2-\sigma_{l}^2$ (Y)"
label_o = r"$\sigma_r^2-\sigma_{l}^2$ (O)"
color_y, color_o = color_dict_comp[0][0], color_dict_comp[0][1]

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

map_string = "covariance"
error_string = "covariance_error"
map_array = map_dict[map_string]
error_array = map_dict[error_string] #/error_factor_anicorr
color_y, color_o = color_dict_comp[1][0], color_dict_comp[1][1]
label_y = r"$\sigma_{rl}^2$ (Y)"
label_o = r"$\sigma_{rl}^2$ (O)"

axs[2].plot(b_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)
axs[2].plot(b_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)

axs[2].fill_between(b_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)
axs[2].fill_between(b_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)

axs[2].set_ylabel("Dispersion difference & covariance")

#handles, labels = ax.get_legend_handles_labels()
## sort both labels and handles by labels
#labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
#ax.legend(handles, labels, loc="upper left")
axs[2].legend(loc='best', fontsize=legend_fontsize,ncol=2)

yticks = np.concatenate(([0], np.linspace(-2900,5200,5)))
axs[2].set_yticks(yticks)
axs[2].set_ylim(min(yticks),max(yticks))

#axs[2].text(-0.5,-0.17,r"Error $\times 10^{-%i}$" % get_exponent(error_factor_anicorr), size=error_fontsize)

#------------------------------------------------------------------------------------------------------------------

bar_width=0.2
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[0][1])
axs[0].bar(b_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][0])
axs[0].bar(b_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_dict_comp[1][1])
axs[0].yaxis.set_tick_params(labelleft=False,labelright=True)

axs[0].set_yticks([100,1000,10000,100000])
axs[0].set_ylabel(r"$N$",labelpad=13, rotation=0)
axs[0].yaxis.set_label_position("right")

axs[0].text(-7.4,4000,"Y: 5-9.8 Gyr\nO: 9.8-10 Gyr", size=error_fontsize)

#error_string_ = '_erroranicorr'+str(error_factor_anicorr) if error_factor_anicorr != 1 else ''
error_string_vert = "_errorvert"+str(error_factor_vert) if error_factor_vert != 1 else ''
filename = "vardiffcov"+error_string_vert+'.png'
plt.savefig(save_path+filename,bbox_inches='tight',dpi=300)
plt.show()

# Stellar populations

## Kinematics vs age/FeH

In [ ]:
def get_base_POPPLOT_path(resampled_sim_bool=False):
    save_path = general_path+'graphs/Observations/Apogee/individual_variable/age_metal/'
    
    save_path += "resampled_sim/" if resampled_sim_bool else ""
    MF.create_dir(save_path)
    
    return save_path

def get_save_path_POPPLOT_spatial_cuts(resampled_sim_bool,extra_variable,extra_min,extra_max,depth_var,depth_min,depth_max,height_var,height_min,height_max):
    """
    Expected values:
    
    extra_variable = "R" or "d"
    depth_var = "l" or "y"
    height_var = "b" or "z"
    """
    
    save_path = get_base_POPPLOT_path(resampled_sim_bool)

    save_path += f"{MF.return_int_or_dec(extra_min,2)}{extra_variable}{MF.return_int_or_dec(extra_max,2)}/"
    MF.create_dir(save_path)

    save_path += f"{MF.return_int_or_dec(depth_min,2)}{depth_var}{MF.return_int_or_dec(depth_max,2)}/"
    MF.create_dir(save_path)
    
    save_path += f"{MF.return_int_or_dec(height_min,2)}{height_var}{MF.return_int_or_dec(height_max,2)}/"
    MF.create_dir(save_path)
    
    return save_path

def get_save_path_POPPLOT_data(save_path_spatial, binning_str, pop_str):
        
    save_path = save_path_spatial + "data/"
    MF.create_dir(save_path)
    
    save_path += f"{binning_str}/"
    MF.create_dir(save_path)
    
    save_path += f"{pop_str}/"
    MF.create_dir(save_path)
        
    return save_path

def get_save_path_POPPLOT_sim(save_path_spatial, binning_str, pop_str):
        
    save_path = save_path_spatial + "sim/"
    MF.create_dir(save_path)
    
    save_path += f"{binning_str}/"
    MF.create_dir(save_path)
    
    save_path += f"{pop_str}/"
    MF.create_dir(save_path)
        
    return save_path

In [ ]:
# metal_lowcut = -9999
metal_lowcut = -1

try:
    data_trim = data[data['FeH']>=metal_lowcut]
    print(f"Chose minimum metallicity of {metal_lowcut}" if metal_lowcut != -9999 else "No minimum metallicity")
except NameError:
    print("Working with simulation only")

### Latitude ranges

Currently chosen according to the data.

In [ ]:
depth_variable_lims = {
    "l": [-2,2],
    "y": [-0.1,0.1]
}

overall_height_variable_lims = {
    "b": [1.5,9],
    "z": [0.2,2]
}

extra_variable_lims = {
    "d": [6.1,10.1],
    "R": [0,3.5]
}

# CHOOSE
# extra_variable = "d"
extra_variable = "R"
depth_var = "l"
height_var = "b"

if True: # print
    depth_min,depth_max = depth_variable_lims[depth_var]
    overall_height_min,overall_height_max = overall_height_variable_lims[height_var]
    extra_min,extra_max = extra_variable_lims[extra_variable]
    
    print(depth_var,depth_min,depth_max)
    print(height_var,overall_height_min,overall_height_max)
    print(extra_variable,extra_min,extra_max)
    
if True: # assert
    assert depth_min == -2 and depth_max == 2, "Are you sure you don't want to have l between -2 and 2?"
    assert extra_min == 0 and extra_max == 3.5, "Are you sure you don't want to have R between 0 and 3.5?"

In [ ]:
if height_var == "b":
    o_b_range_min,o_b_range_max = PH.get_equal_n_minmax_b_ranges(data_trim, n_points=3,\
                                                                 extra_variable=extra_variable,extra_min=extra_min,extra_max=extra_max,\
                                                                 depth_min=depth_min,depth_max=depth_max,\
                                                                 overall_bmin=overall_height_min,overall_bmax=overall_height_max)

In [ ]:
lat_bin_idx = 0

if height_var == "b":
    bmin = o_b_range_min[lat_bin_idx]
    bmax = o_b_range_max[lat_bin_idx]

    print(bmin, MF.return_int_or_dec(bmin,2))
    print(bmax, MF.return_int_or_dec(bmax,2))

I will apply the same latitude binning (using 0R3.5) for the different radial variations (eg 0R2, for which the equal-number latitude binning would change slightly)

### In bulk

#### Sim

In [ ]:
def get_plot_ranges(df_extra, binning_type="equalN", n_points=20, plot_median_bool=False):
    if binning_type == "manual":

        if old_subplots:
    #         pop_str_sim = "0to9in1_oldSplit"
    #         pop_min_range = np.array([0,4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
    #         pop_max_range = np.array([4,5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])

            pop_str_sim = "4to9in1_oldSplit"
            pop_min_range = np.array([4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
            pop_max_range = np.array([5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])

            max_age_lim = 9

            limit_index = np.where(pop_max_range == max_age_lim)[0][0] + 1
        else:
            pop_str_sim = "4to10in1"
            pop_min_range = np.array([min_age,5,6,7,8, 9])
            pop_max_range = np.array([5,6,7,8,9, max_age])

    elif binning_type == "equalN":
        all_age_bins = PH.get_equal_n_bin_edges(val_array=df_extra["age"].values,n_bins=n_points,pandas_way=True)

        pop_min_range = all_age_bins[:-1]
        pop_max_range = all_age_bins[1:]

        pop_str_sim = f"{n_points}_datapoints"

    elif binning_type == "equalSteps":

        all_age_bins = np.linspace(min_age,max_age,n_points+1)
        pop_min_range = all_age_bins[:-1]
        pop_max_range = all_age_bins[1:]

        pop_str_sim = f"{n_points}_datapoints"

    else:
        raise ValueError("Unknown binning method.")

    assert len(pop_min_range) == len(pop_max_range), "Lengths need to be equal"

    if plot_median_bool:
        """
        If I show the values as a surface (with fill_between) I'd rather plot at the mean (i.e. mid-point of the bin) because otherwise I need to give some other indication
        of the width of each bin, and I don't want to show x-error bars as I imagine they'd overlap in an ugly way with the surface (although I have not tried).
        """

        pop_plot_range = PH.get_range_medians(df_extra["age"],pop_min_range,pop_max_range)
    else:
        pop_plot_range = PH.get_range_means(pop_min_range,pop_max_range)
        
    return pop_min_range,pop_max_range,pop_plot_range,pop_str_sim

In [ ]:
depth_variable_lims = {
    "l": [-2,2],
    "y": [coordinates.ang_to_rect(ang=-2,x=coordinates.get_solar_radius()), coordinates.ang_to_rect(ang=2,x=coordinates.get_solar_radius())]
}

height_variable_lims = {
    "b": [bmin,bmax],
    "z": [coordinates.ang_to_rect(bmin,x=R0), coordinates.ang_to_rect(bmax,x=R0)]
} if "bmin" in globals() and "bmax" in globals() else {
    "b": [1.5,9],
    "z": [0.2,2]
}

z_range_min = [coordinates.ang_to_rect(bmin,x=R0) for bmin in o_b_range_min]
z_range_max = [coordinates.ang_to_rect(bmax,x=R0) for bmax in o_b_range_max]

extra_variable_lims = {
    "d": [6.1,10.1],
    "R": [0,3.5]
}

# CHOOSE
# extra_variable = "d"
extra_variable = "R"
depth_var = "l"
height_var = "b"

if True: # print
    depth_min,depth_max = depth_variable_lims[depth_var]
    height_min,height_max = height_variable_lims[height_var]
    extra_min,extra_max = extra_variable_lims[extra_variable]
    
    print(depth_var,depth_min,depth_max)
    print(height_var,height_min,height_max)
    print(extra_variable,extra_min,extra_max)
    
    print("\nz ranges:")
    print(z_range_min)
    print(z_range_max)

In [ ]:
# do everything up to saving the arrays in a loop

save_arrays_bool = True
# save_arrays_bool = False

min_pop,max_pop = 4,10
pop_var = "age"
xlabel = "Age [Gyr]" # needed in visualise_1D_binning

bootstrap_repeat = 500
min_star_number = 50

vel_hist_bool = True
velhist_bins = 50

binning_type_list = len(n_points_list)*["equalSteps"]
plot_median_list = len(n_points_list)*[True]

cuts_dict_list = [
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[0],o_b_range_max[0]],extra_variable:[0,3.5],pop_var:[min_pop,max_pop]},
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[0],o_b_range_max[0]],extra_variable:[0,2],pop_var:[min_pop,max_pop]},
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[1],o_b_range_max[1]],extra_variable:[0,3.5],pop_var:[min_pop,max_pop]},
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[1],o_b_range_max[1]],extra_variable:[0,2],pop_var:[min_pop,max_pop]},
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[2],o_b_range_max[2]],extra_variable:[0,3.5],pop_var:[min_pop,max_pop]},
    {depth_var:[depth_min,depth_max],height_var:[o_b_range_min[2],o_b_range_max[2]],extra_variable:[0,2],pop_var:[min_pop,max_pop]}
]

n_points_list = [
    40,
    40,
    20,
    20,
    10,
    10
]

n_points_list_resampled = [
    8, # or 7
    8, # or 7
    6,
    5,
    4, # or 5
    3
]

vel_freq_list = [
    4,
    4,
    2,
    2,
    1,
    1
]

n_points_list = n_points_list_resampled if sim_resampled_bool else n_points_list
vel_freq_list = [1]*len(n_points_list) if sim_resampled_bool else vel_freq_list

assert len(cuts_dict_list)==len(n_points_list)==len(vel_freq_list)==len(binning_type_list)==len(plot_median_list), "All lists must have the same length"

for i in range(len(cuts_dict_list)):
    df_extra = MF.apply_cuts_to_df(df0,cuts_dict=cuts_dict_list[i])
    
    pop_min_range,pop_max_range,pop_plot_range,pop_str_sim = get_plot_ranges(df_extra, binning_type=binning_type_list[i],\
                                                                 n_points=n_points_list[i],plot_median_bool=plot_median_list[i])
    
    print(cuts_dict_list[i])
    print(f"{len(df_extra)} total stars")
    print(f"pop_plot_range: {pop_plot_range}")
    print("Star numbers:",stat.binned_statistic(values=None,x=df_extra[pop_var].values,bins=np.union1d(pop_min_range,pop_max_range),statistic="count")[0])
    
    extra_min,extra_max = cuts_dict_list[i][extra_variable]
    depth_min,depth_max = cuts_dict_list[i][depth_var]
    height_min,height_max = cuts_dict_list[i][height_var]
    
    save_path_spatial = get_save_path_POPPLOT_spatial_cuts(sim_resampled_bool,extra_variable,extra_min,extra_max,depth_var,depth_min,depth_max,height_var,height_min,height_max)
    save_path = get_save_path_POPPLOT_sim(save_path_spatial, binning_type_list[i], pop_str_sim)
    
    print("save_path:",save_path)
    
    MP.visualise_1D_binning(df_extra[pop_var].values, pop_min_range, pop_max_range, hist_bins=100, log=True,\
                            save_bool=True,save_path=save_path,filename_prefix=pop_var,xlabel=xlabel,show_bool=True)

    if vel_hist_bool:
        save_path_hist = save_path + "vel_histograms/"
        MF.create_dir(save_path_hist)

        save_path_hist += f"{velhist_bins}bins/"
        MF.create_dir(save_path_hist)

        print("Saving velocity histograms on\n",save_path_hist)
        
    if True: # get arrays
        map_dict = {}
        for map_string in full_map_string_list:
            map_dict[map_string] = np.zeros(shape=(len(pop_min_range)))

        for pop_index, (popmin, popmax) in enumerate(zip(pop_min_range,pop_max_range)):

            print(popmin,popmax,end=";  ")

            include_lims = "both" if pop_index==len(pop_min_range)-1 else "min"
            df_pop = MF.apply_cuts_to_df(df_extra, cuts_dict={pop_var:[popmin,popmax]}, lims_dict={pop_var:include_lims})

            if vel_hist_bool and pop_index % vel_freq_list[i] == 0:
                name_suffix = f"{str(MF.return_int_or_dec(popmin,dec=2))}pop{str(MF.return_int_or_dec(popmax,dec=2))}"
                MP.plot_velocity_histograms_both_stats(df_pop,vel_x_variable,vel_y_variable,save_bool=True,save_path=save_path_hist,suffix=name_suffix,verbose=pop_index==0,bins=velhist_bins)

            values = val_err.get_all_variable_values_and_errors(df_pop[f"v{vel_x_variable}"].values,df_pop[f"v{vel_y_variable}"].values, full_map_string_list,\
                                                                    repeat=bootstrap_repeat, min_number = min_star_number)   

            if len(values) != len(full_map_string_list):
                raise ValueError("The length of the values list does not match the string list!")

            for map_string in full_map_string_list:
                map_dict[map_string][pop_index] = values[map_string]

        del df_pop
        print("Done")
    
    if save_arrays_bool:

        array_path = save_path + "arrays/"
        
        overwrite = False
        if os.path.isdir(array_path):
            overwrite_str = input("There may be files already in this folder, do you want to overwrite them? Y/N\n")
            if overwrite_str.upper() == "Y":
                overwrite = True
        else:
            MF.create_dir(array_path)
            overwrite = True
            
        if overwrite:

            if True: # values as .txt and .npy

                with open(array_path+'values.txt','w') as f:
                    for key in map_dict:
                        f.write(key+'\n')
                        np.savetxt(f,map_dict[key],fmt='%.5f')
                        f.write('\n')

                for map_string in full_map_string_list:
                    np.save(array_path+map_string, map_dict[map_string])

            if True: # plot limits as .txt and .npy

                with open(array_path+'pop_ranges.txt','w') as f:
                    f.write("pop_min_range\n")
                    for mini in pop_min_range:
                        f.write(f"{mini}\t")
                    f.write("\n\npop_max_range\n")
                    for maxi in pop_max_range:
                        f.write(f"{maxi}\t")
                    f.write("\n\npop_plot_range\n")
                    for p in pop_plot_range:
                        f.write(f"{p}\t")

                np.save(array_path+"pop_min_range", pop_min_range)
                np.save(array_path+"pop_max_range", pop_max_range)
                np.save(array_path+"pop_plot_range", pop_plot_range)

            print("Saved .txt and .npy in",array_path)
    else:
        print("Not saving")

    print("\n")

### Sim

In [ ]:
data_bool = False

In [ ]:
depth_variable_lims = {
    "l": [-2,2],
    "y": [coordinates.ang_to_rect(ang=-2,x=coordinates.get_solar_radius()), coordinates.ang_to_rect(ang=2,x=coordinates.get_solar_radius())]
}

height_variable_lims = {
    "b": [bmin,bmax],
    "z": [coordinates.ang_to_rect(bmin,x=R0), coordinates.ang_to_rect(bmax,x=R0)]
} if "bmin" in globals() and "bmax" in globals() else {
    "b": [1.5,9],
    "z": [0.2,2]
}

extra_variable_lims = {
    "d": [6.1,10.1],
    "R": [0,2]
}

# CHOOSE
# extra_variable = "d"
extra_variable = "R"
depth_var = "l"
height_var = "b"

if True: # print
    depth_min,depth_max = depth_variable_lims[depth_var]
    height_min,height_max = height_variable_lims[height_var]
    extra_min,extra_max = extra_variable_lims[extra_variable]
    
    print(depth_var,depth_min,depth_max)
    print(height_var,height_min,height_max)
    print(extra_variable,extra_min,extra_max)

In [ ]:
if False: # xz young-old density plot
    
    # save_bool = True
    save_bool = False

    # projection = "xy"
    projection = "xz"

    plt.rcParams["font.size"] = 16

    if True: # visualise cuts for young and old

        xymax = 3.5

        zmax = 2.2
        zmin = (0 if zabs else -zmax) if projection == "xz" else 0.5

        bins_x=70
        density_bool=False

        young_ages,old_ages = [4,7],[9.5,10]

        if True:

            aspect_ratio = 2*(zmax-zmin)/(1.5*xymax) if projection == "xz" else 1

            fig,axs=plt.subplots(figsize=(10,aspect_ratio*10),nrows=2,gridspec_kw={"hspace":0})

            if True: # quick show & colorbar

                spatial_cut_dict = { depth_var:[depth_min,depth_max] } if projection == "xz" else { "z":[zmin,zmax]}
                young_cut_dict,old_cut_dict = {"age":young_ages}, {"age":old_ages}

                if projection == "xz":
                    c1 = MP.quick_show_xz(MF.apply_cuts_to_df(df0,[spatial_cut_dict,young_cut_dict]),bins_x=bins_x,zmin=zmin,zmax=zmax,xmin=-xymax,xmax=xymax,show=False,density=density_bool)
                    c2 = MP.quick_show_xz(MF.apply_cuts_to_df(df0,[spatial_cut_dict,old_cut_dict]),bins_x=bins_x,zmin=zmin,zmax=zmax,xmin=-xymax,xmax=xymax,show=False,density=density_bool)
                elif projection == "xy":
                    c1 = MP.quick_show_xy(MF.apply_cuts_to_df(df0,[spatial_cut_dict,young_cut_dict]),bins_x=bins_x,ymin=-xymax,ymax=xymax,xmin=-xymax,xmax=xymax,show=False,density=density_bool)
                    c2 = MP.quick_show_xy(MF.apply_cuts_to_df(df0,[spatial_cut_dict,old_cut_dict]),bins_x=bins_x,ymin=-xymax,ymax=xymax,xmin=-xymax,xmax=xymax,show=False,density=density_bool)
                else:
                    raise ValueError("Only xy and xz currently supported")

                norm = PH.get_norm_from_count_list([c1,c2],log=True)

                if projection == "xz":
                    _ = MP.quick_show_xz(MF.apply_cuts_to_df(df0,[spatial_cut_dict,young_cut_dict]),bins_x=bins_x,zmin=zmin,zmax=zmax,xmin=-xymax,xmax=xymax,ax=axs[0],norm=norm,density=density_bool)
                    _ = MP.quick_show_xz(MF.apply_cuts_to_df(df0,[spatial_cut_dict,old_cut_dict]),bins_x=bins_x,zmin=zmin,zmax=zmax,xmin=-xymax,xmax=xymax,ax=axs[1],norm=norm,density=density_bool)
                elif projection == "xy":
                    _ = MP.quick_show_xy(MF.apply_cuts_to_df(df0,[spatial_cut_dict,young_cut_dict]),bins_x=bins_x,ymin=-xymax,ymax=xymax,xmin=-xymax,xmax=xymax,ax=axs[0],norm=norm,density=density_bool)
                    _ = MP.quick_show_xy(MF.apply_cuts_to_df(df0,[spatial_cut_dict,old_cut_dict]),bins_x=bins_x,ymin=-xymax,ymax=xymax,xmin=-xymax,xmax=xymax,ax=axs[1],norm=norm,density=density_bool)
                else:
                    raise ValueError("Only xy and xz currently supported")

                cbar = plt.colorbar(cm.ScalarMappable(norm=norm,cmap="viridis"),ax=axs,shrink=0.8)
                cbar.set_label(mass_density_label) if density_bool else cbar.set_label(r"$N$",rotation=0,labelpad=20)

            if True: # visualise cuts & legend

                cuts_to_visualise = {depth_var:[depth_max],height_var:[height_min,height_max],"R":[2,3.5],"l":[2]}
                if height_var == "z":
                    cuts_to_visualise["b"] = [bmin,bmax]

                filename,_ = MP.visualise_bulge_selection(given_axs=axs[::-1],projection=projection,cuts_dict=cuts_to_visualise,R0=R0)
                _,_ = MP.visualise_bulge_selection(given_axs=axs,projection=projection,cuts_dict=cuts_to_visualise,R0=R0)

                axs[0].legend(loc="upper right" if projection=="xy" else "upper left",framealpha=0.8,ncols=1 if projection=="xy" else 2)

            for i,ax in enumerate(axs): # lims, aspect, titles
                ax.set_xlim(-xymax,xymax)
                ax.set_ylim(zmin,zmax) if projection == "xz" else ax.set_ylim(-xymax,xymax)
                ax.set_aspect("equal")

                if projection == "xy":
                    ax.text(x=0.03,y=0.97,s=["Young","Old"][i],transform=ax.transAxes,bbox={"facecolor":"w","alpha":0.9},ha="left",va="top")
                if projection == "xz":
                    ax.set_title(["Young","Old"][i])

            if True: # filename and saving

                filename += f"_{projection}" if projection != "both" else ""

                filename += f"_{MF.return_int_or_dec(depth_min,2)}{depth_var}{MF.return_int_or_dec(depth_max,2)}" if projection == "xz" \
                            else f"_{MF.return_int_or_dec(zmin,2)}z{MF.return_int_or_dec(zmax,2)}"

                print(filename)

                save_path = f"{general_path}graphs/other_plots/visualise_bulge_cuts/youngold_cbar/{projection}/"
                save_path += "resampled/" if sim_resampled_bool else ""

                if save_bool:
                    print("Saving in:",save_path)
                    plt.savefig(save_path+filename+".png", dpi=200,bbox_inches="tight")

            plt.show()

In [ ]:
min_age,max_age = 4,10

df_extra = MF.apply_cuts_to_df(df0,cuts_dict={depth_var:[depth_min,depth_max],height_var:[height_min,height_max],\
                                              extra_variable:[extra_min,extra_max],"age":[min_age,max_age]})

print(len(df_extra),"stars")

In [ ]:
# sim_plot_median = True
sim_plot_median = False

equal_number = False
equal_steps = True
manual_ages = False

n_points_sim = 7

In [ ]:
# binning

assert manual_ages + equal_steps + equal_number == 1, "Select a single option"
binning_str_sim = np.array(["custom_range","equalSteps","equalN"])[np.array([manual_ages,equal_steps,equal_number])][0]
print("Using",binning_str_sim)

if manual_ages:
    
    if old_subplots:
#         pop_str_sim = "0to9in1_oldSplit"
#         pop_min_range = np.array([0,4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
#         pop_max_range = np.array([4,5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])

        pop_str_sim = "4to9in1_oldSplit"
        pop_min_range = np.array([4,5,6,7,8, 9,   9.5, 9.8,  9.9,   9.925, 9.95, 9.975])
        pop_max_range = np.array([5,6,7,8,9, 9.5, 9.8, 10,   9.925, 9.95,  9.975, 10])
        
        max_age_lim = 9
        
        limit_index = np.where(pop_max_range == max_age_lim)[0][0] + 1
    else:
        pop_str_sim = "4to10in1"
        pop_min_range = np.array([min_age,5,6,7,8, 9])
        pop_max_range = np.array([5,6,7,8,9, max_age])

elif equal_number:
    all_age_bins = PH.get_equal_n_bin_edges(val_array=df_extra["age"].values,n_bins=n_points_sim,pandas_way=True)
    
    pop_min_range = all_age_bins[:-1]
    pop_max_range = all_age_bins[1:]
    
    pop_str_sim = f"{n_points_sim}_datapoints"
    
elif equal_steps:
    
    all_age_bins = np.linspace(min_age,max_age,n_points_sim+1)
    pop_min_range = all_age_bins[:-1]
    pop_max_range = all_age_bins[1:]
    
    pop_str_sim = f"{n_points_sim}_datapoints"
    
else:
    raise ValueError("Unknown binning method.")
    
assert len(pop_min_range) == len(pop_max_range), "Lengths need to be equal"

if sim_plot_median:
    """
    If I show the values as a surface (with fill_between) I'd rather plot at the mean (i.e. mid-point of the bin) because otherwise I need to give some other indication
    of the width of each bin, and I don't want to show x-error bars as I imagine they'd overlap in an ugly way with the surface (although I have not tried).
    """
    
    pop_plot_range = PH.get_range_medians(df_extra["age"],pop_min_range,pop_max_range)
    print("Plotting at the median\n",pop_plot_range)
else:
    pop_plot_range = PH.get_range_means(pop_min_range,pop_max_range)
    print("Plotting at the mean\n",pop_plot_range)

In [ ]:
fig,ax=plt.subplots(figsize=(10,2))

ax.errorbar(x=pop_plot_range,xerr=PH.get_xerr(minima=pop_min_range,maxima=pop_max_range,plot=pop_plot_range,frac=1),y=[0]*n_points_sim,fmt="d",capsize=20)
ax.set_xlim(3.95,10.05)
ax.minorticks_on()
ax.set_yticks([])
ax.invert_xaxis()
plt.show()

In [ ]:
# number of stars per bin
stat.binned_statistic(values=None,x=df_extra["age"].values,bins=all_age_bins,statistic="count")[0]

lengths = []
for individual_age in ages_range:
    age_step = age_step_young if individual_age < age_limit_oldyoung else age_step_old
    
    age_max = individual_age + age_step
    age_min = individual_age
    df = df_extra[(df_extra.age>=age_min)&(df_extra.age<age_max)]
    lengths.append(len(df))
    print("Age interval", np.float16(age_min), "to", np.float16(age_max),"has",len(df),"stars")
print("Total number of stars selected across time:",np.sum(lengths))
print("Which is "+str(np.float16(100*np.sum(lengths)/len(df0)))+"% of the total")
del df

### Data

In [ ]:
data_bool = True

In [ ]:
depth_variable_lims = {
    "l": [-2,2],
    "y": [-0.1,0.1]
}

height_variable_lims = {
    "b": [bmin,bmax],
    "z": [np.tan(bmin*np.pi/180)*R0, np.tan(bmax*np.pi/180)*R0]
} if "bmin" in globals() and "bmax" in globals() else {
    "b": [1.5,9],
    "z": [0.2,2]
}

extra_variable_lims = {
    "d": [6.1,10.1],
    "R": [0,3.5]
}

# CHOOSE
# extra_variable = "d"
extra_variable = "R"
depth_var = "l"
height_var = "b"

if True: # print
    depth_min,depth_max = depth_variable_lims[depth_var]
    height_min,height_max = height_variable_lims[height_var]
    extra_min,extra_max = extra_variable_lims[extra_variable]
    
    print(depth_var,depth_min,depth_max)
    print(height_var,height_min,height_max)
    print(extra_variable,extra_min,extra_max)

In [ ]:
data_extra = MF.apply_cuts_to_df(data_trim,cuts_dict={depth_var:[depth_min,depth_max],height_var:[height_min,height_max],\
                                              extra_variable:[extra_min,extra_max]})

print(len(data_extra))

In [ ]:
equal_number = True # divide in equal-number bins across all metallicities
manual_metal = False
equal_steps = False # divide in constant metallicity steps

plot_median_bool = True
# plot_median_bool = False # mid-point of bin

n_points_data = 3

In [ ]:
assert manual_metal + equal_steps + equal_number == 1, "Select a single option"
binning_str_data = np.array(["custom_range","equalSteps","equalN"])[np.array([manual_metal,equal_steps,equal_number])][0]

if manual_metal:
    min_metal,max_metal = min(data_extra["FeH"]),max(data_extra['FeH'])
    
    pop_str_data = "metal_cuts_equal"
    pop_min_range = np.array([min_metal, -0.55, -0.25, 0.15])
    pop_max_range = np.array([-0.55,-0.25,0.15,max_metal])

#     pop_str_data = "metal_cuts_A"
#     pop_min_range = np.array([min_metal, -0.7, -0.4, -0.1, 0.3])
#     pop_max_range = np.array([-0.7,-0.4,0.1,0.3,max_metal])
    
#     pop_str_data = "metal_cuts_B"
#     pop_min_range = np.array([min_metal, -1, -0.5, 0,0.3])
#     pop_max_range = np.array([-1,-0.5,0,0.3,max_metal])
    
#     pop_str_data = "all_poor"
#     pop_max_range = np.array([-1.3,-1.2,-1.1,-1.0,-0.9,-0.8,-0.7])
#     pop_min_range = np.array([min_metal]*len(pop_max_range))

#     pop_str_data = "axisymmetric"
#     pop_min_range = np.array([-1.2, -1.2, -1.2, -1.1,-1.1, -1,-1,-1])
#     pop_max_range = np.array([-0.9,-0.8,-0.7,-0.9,-0.8,-0.7, -0.6,-0.5])

#     pop_str_data = "all_rich"
#     pop_min_range = np.array([-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0])
#     pop_max_range = np.array([max_metal]*len(pop_min_range))
    
    pass
elif equal_steps:
    min_metal = data_extra['FeH'].min()
    metal_step = 0.1
    pop_min_range = np.arange(min_metal,data_extra['FeH'].max(),metal_step)
    pop_max_range = pop_min_range + metal_step
    
    pop_str_data = f"{metal_step}step"  
elif equal_number:
    
    pop_str_data = f"{n_points_data}_datapoints"
    
    metal_edges = PH.get_equal_n_bin_edges(data_extra.FeH.values, n_points_data)
    pop_max_range = metal_edges[1:]
    pop_min_range = metal_edges[:-1]

if True: # pop_plot_range
    if manual_metal and range_str in ["all_rich","all_poor"]:
        if range_str == "all_rich":
            pop_plot_range = pop_min_range
        if range_str == "all_poor":
            pop_plot_range = pop_max_range
    elif plot_median_bool:
        pop_plot_range = PH.get_range_medians(data_extra.FeH.values, pop_min_range, pop_max_range)
    else:
        pop_plot_range = np.array([np.mean([m,M]) for m,M in zip(pop_min_range,pop_max_range)])
    
assert len(pop_min_range) == len(pop_max_range), "Lengths need to be equal"
print(pop_plot_range)

### Save path

In [ ]:
save_path_spatial = get_save_path_POPPLOT_spatial_cuts(sim_resampled_bool,extra_variable,extra_min,extra_max,depth_var,depth_min,depth_max,height_var,height_min,height_max)

if data_bool:
    save_path = get_save_path_POPPLOT_data(save_path_spatial, binning_str_data, pop_str_data)
else:
    save_path = get_save_path_POPPLOT_sim(save_path_spatial, binning_str_sim, pop_str_sim)
    
print(save_path)

In [ ]:
df = data_extra if data_bool else df_extra
pop_var = "FeH" if data_bool else "age"
xlabel = "[Fe/H]" if data_bool else "Age [Gyr]"

MP.visualise_1D_binning(df[pop_var].values, pop_min_range, pop_max_range, hist_bins=100, log=False,\
                        save_bool=True,save_path=save_path,filename_prefix=pop_var,xlabel=xlabel)

if not data_bool:
    MP.visualise_1D_binning(df[pop_var].values, pop_min_range, pop_max_range, hist_bins=100, log=True,\
                            save_bool=True,save_path=save_path,filename_prefix=pop_var,xlabel=xlabel)

### Get arrays

In [ ]:
bootstrap_repeat = 500
min_star_number = 50 if not data_bool else 50

In [ ]:
vel_hist_bool = True
# vel_hist_bool = False

velhist_bins = 50 if not data_bool else 20

if vel_hist_bool:
    save_path_hist = save_path + "vel_histograms/"
    MF.create_dir(save_path_hist)
    
    save_path_hist += f"{velhist_bins}bins/"
    MF.create_dir(save_path_hist)
    
    print("Saving velocity histograms on\n",save_path_hist)

if vel_hist_bool:
    MP.plot_velocity_histograms_both_stats(df[(df[pop_var]>=min(pop_min_range))&(df[pop_var]<=min(pop_max_range))],vel_x_variable,vel_y_variable,\
                                           bins=velhist_bins,colour_var="x",save_bool=False,suffix="example",verbose=True,show=True)

In [ ]:
map_dict = {}
for map_string in full_map_string_list:
    map_dict[map_string] = np.zeros(shape=(len(pop_min_range)))
    
map_dict["mean_b"] = np.zeros(shape=(len(pop_min_range)))
map_dict["std_b"] = np.zeros(shape=(len(pop_min_range)))

for pop_index, (popmin, popmax) in enumerate(zip(pop_min_range,pop_max_range)):

    print(popmin,popmax)
    
    include_lims = "both" if pop_index==len(pop_min_range)-1 else "min"
    df_pop = MF.apply_cuts_to_df(df, cuts_dict={pop_var:[popmin,popmax]}, lims_dict={pop_var:include_lims})
        
    if vel_hist_bool:
        name_suffix = f"{str(MF.return_int_or_dec(popmin,dec=2))}pop{str(MF.return_int_or_dec(popmax,dec=2))}"
        MP.plot_velocity_histograms_both_stats(df_pop,vel_x_variable,vel_y_variable,save_bool=True,save_path=save_path_hist,suffix=name_suffix,verbose=pop_index==0,bins=velhist_bins)
        
    values = val_err.get_all_variable_values_and_errors(df_pop[f"v{vel_x_variable}"].values,df_pop[f"v{vel_y_variable}"].values, full_map_string_list,\
                                                            repeat=bootstrap_repeat, min_number = min_star_number)   

    if len(values) != len(full_map_string_list):
        raise ValueError("The length of the values list does not match the string list!")

    for map_string in full_map_string_list:
        map_dict[map_string][pop_index] = values[map_string]
        
    map_dict["mean_b"][pop_index] = np.mean(df_pop["b"])
    map_dict["std_b"][pop_index] = np.std(df_pop["b"])
    
del df_pop
print("Done")

In [ ]:
print(map_dict['number'])

In [ ]:
# save arrays

array_path = save_path + "arrays/"
MF.create_dir(array_path)

if True: # values as .txt and .npy
            
    with open(array_path+'values.txt','w') as f:
        for key in map_dict:
            f.write(key+'\n')
            np.savetxt(f,map_dict[key],fmt='%.5f')
            f.write('\n')
    
    for map_string in full_map_string_list:
        np.save(array_path+map_string, map_dict[map_string])
        
if True: # plot limits as .txt and .npy

    with open(array_path+'pop_ranges.txt','w') as f:
        f.write("pop_min_range\n")
        for mini in pop_min_range:
            f.write(f"{mini}\t")
        f.write("\n\npop_max_range\n")
        for maxi in pop_max_range:
            f.write(f"{maxi}\t")
        f.write("\n\npop_plot_range\n")
        for p in pop_plot_range:
            f.write(f"{p}\t")
    
    np.save(array_path+"pop_min_range", pop_min_range)
    np.save(array_path+"pop_max_range", pop_max_range)
    np.save(array_path+"pop_plot_range", pop_plot_range)
    
print("Saved .txt and .npy in",array_path)

### Plot

In [ ]:
# POPPLOT functions

# Takes global: bar_log, bar_width
def POPPLOT_number_bar(barax, plot_range, number_array,color,alpha=1,zorder=0, bar_width=50):
    barax.bar(plot_range,number_array,width=bar_width,log=bar_log,color=color,alpha=alpha,zorder=zorder)
        
def POPPLOT_values_scatter(ax, val_array, err_array, plot_range, min_range, max_range, color, label,line_alpha=1,zorder=0,lines_bool=True,x_error_bool=True):
    xerror = PH.get_xerr(min_range,max_range,plot_range)

    ax.errorbar(plot_range,val_array,yerr=err_array,xerr=xerror if x_error_bool else None,capsize=capsize,marker='.',color=color,\
                label=label,linestyle=None if lines_bool else '',alpha=line_alpha,zorder=zorder)

def POPPLOT_values_surface(ax, val_array, err_array, plot_range,color,label,line_alpha=1,surface_alpha=0.75,zorder=0):
    ax.plot(plot_range,val_array,color=color,alpha=line_alpha,zorder=zorder)
    ax.fill_between(plot_range,val_array-err_array,val_array+err_array,label=label,color=color,alpha=surface_alpha,linewidth=0,zorder=zorder)
        
def POPPLOT_number_bar_axis_settings(barax,min_n,max_n,bar_log=True,labels_on=True,min_shift_bool=True,max_shift_bool=True):
    if bar_log:
        exponent_ticks = np.arange(MF.get_exponent(min_n),MF.get_exponent(max_n)+1,1)
        barax.set_yticks([10**i for i in exponent_ticks])
        barax.set_ylim(bottom = 10**min(exponent_ticks) - (min_n/3 if min_shift_bool else 0))
        barax.set_ylim(top = max_n + (10**MF.get_exponent(max_n) if max_shift_bool else 0))
    elif equal_number:
        barax.set_yticks([0,max_n])
            
    barax.yaxis.set_tick_params(which='minor', right=True,left=False)

    barax.tick_params(which='both',labelleft=False,labelright=labels_on)
    barax.tick_params(which='minor',labelright=False)

    if labels_on:
        barax.set_ylabel(r"$N$",rotation=0,labelpad=20)
        barax.yaxis.set_label_position("right")
    
def POPPLOT_xaxis_settings(ax,xmin,xmax,xlabel,xticks=None,labels_on=True):
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    
    ax.set_xlim(xmin,xmax)
    
    if xticks is not None:
        ax.set_xticks(xticks)
    
    if labels_on:
        ax.set_xlabel(xlabel)
    else:
        ax.set_xticklabels([])

def _POPPLOT_compute_ylims(map_dict, map_string, error_string, var1_bool=False):
    minimum = np.nanmin(map_dict[map_string]-map_dict[error_string])
    maximum = np.nanmax(map_dict[map_string]+map_dict[error_string])

    if var1_bool:
        minimum1 = np.nanmin(map_dict[var1]-map_dict[err1])
        maximum1 = np.nanmax(map_dict[var1]+map_dict[err1])

        minimum = min([minimum,minimum1])
        maximum = max([maximum,maximum1])
    
    if map_string in yshift_dict:
        minimum -= yshift_dict[map_string]
        maximum += yshift_dict[map_string]

    if symmetric_ylims_bool:
        maxabs = np.nanmax(np.abs([minimum,maximum]))
        ax.set_ylim(-maxabs,maxabs)
    else:
        ax.set_ylim(minimum,maximum)
        
def POPPLOT_yaxis_settings(ax, map_string, error_string, map_dict=None, labels_on=True,var1_bool=False, set_ylims=True):
    
    if map_string == 'tilt_abs':# or "mean" in map_string or "std" in map_string:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(20))
        #ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))
    elif map_string == 'anisotropy':# or map_string == 'correlation':
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(0.25))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))
    elif map_string == "correlation":
        ax.yaxis.set_major_locator(ticker.MultipleLocator(0.15))
#     elif map_string in ["mean_vx","mean_vy"]:
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(15))
    elif map_string in ["std_vx","std_vy"]:
        ax.yaxis.set_major_locator(ticker.MultipleLocator(25))
    
    if not var1_bool and labels_on:
        ax.set_ylabel(symbol_dict[map_string]+units_dict[map_string],fontsize=ylabel_size)
    
    if set_ylims:
        if hard_coded_ylims_bool and map_string in hard_coded_ylims:
            ax.set_ylim(hard_coded_ylims[map_string])
        else:
            if map_dict is None:
                raise ValueError("Cannot compute ylims if `map_dict` is None.")
            _POPPLOT_compute_ylims(map_dict, map_string, error_string, var1_bool=var1_bool)
        
    if not labels_on:
        ax.set_yticklabels([])
        
def get_label(map_string):
    return symbol_dict[map_string]+units_dict[map_string]

def get_legend_label(var_tuple,variable):
    var_symbol = pos_symbols_dict[variable]
    var_units = pos_units_dict[variable]
    
    if var_tuple[0] == 0:
        return var_symbol + fr"$< {var_tuple[1]}~$"+var_units
    else:
        return r"$%s<$"%str(var_tuple[0]) + var_symbol + r"$/\mathrm{%s}$"%var_units + fr"$<{var_tuple[1]}$"

#### Single

In [ ]:
capsize = 5
ylabel_size = "medium"

data_xerror_bool = True
# data_xerror_bool = False

# bar_log = True
bar_log = False
# bar_log = not data_bool

data_lines_bool = True
# data_lines_bool = False

# data_surface_bool = True
data_surface_bool = False

sim_surface_bool = True
# sim_surface_bool = False

In [ ]:
exponent_ticks = [1,2,3] if data_bool else [3,4,5]
xlabel = "[Fe/H]" if data_bool else "Age [Gyr]"

line_alpha = 0.75
surface_alpha = 0.75

In [ ]:
var = 'mean_vx';        var1_bool=True
# var = 'anisotropy';     var1_bool=True
# var = "std_vx";         var1_bool=True
# var = 'tilt_abs';       var1_bool=False

err = var + "_error"

print(var,err)

color = 'steelblue'
if var1_bool:
    color = 'purple'
    color1 = 'orange'
    
    if var == 'anisotropy': var1 = 'correlation'
    elif "vx" in var: var1 = var.split("vx")[0] + "vy" + var.split("vx")[1]
    else: raise ValueError("Could not determine var1")
    
    err1 = var1+'_error'
    
    print(var1,err1)

In [ ]:
# hard_coded_ylims_bool = True
hard_coded_ylims_bool = False
# hard_coded_ylims_bool = var == "tilt_abs"

# symmetric_ylims_bool = True
symmetric_ylims_bool = False

hard_coded_ylims = {
    "tilt_abs": [-45,3]
}

yshift_dict = {
    "anisotropy": 0.01,
    "tilt_abs": 5,
    "mean_b": 0.25,
    "std_vx": 1,
    "mean_vx": 1
}

legend_loc = "best"
# legend_loc = "upper left"

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

if True: # fig, bar, axhline
    fig,(barax,ax) = plt.subplots(figsize=(10,10),nrows=2,ncols=1, sharex=True,facecolor='w',gridspec_kw={'hspace':0,'height_ratios':[0.15,1]})

    POPPLOT_number_bar(barax, pop_plot_range, map_dict["number"],color="grey",alpha=line_alpha)
    POPPLOT_number_bar_axis_settings(barax,min_n=np.min(map_dict["number"]),max_n=np.max(map_dict["number"]))

if True: # plot
    
    if data_bool or not sim_surface_bool:
        POPPLOT_values_data(ax, map_dict[var], map_dict[err], pop_plot_range, pop_min_range, pop_max_range,color,get_label(var),alpha=line_alpha)
    else:
        POPPLOT_values_sim(ax,map_dict[var],map_dict[err],pop_plot_range,color,get_label(var),line_alpha=line_alpha,surface_alpha=surface_alpha)

    if var1_bool: # var1, legend
        if data_bool or not sim_surface_bool:
            POPPLOT_values_data(ax, map_dict[var1], map_dict[err1], pop_plot_range, pop_min_range, pop_max_range,color1,get_label(var1),alpha=line_alpha)
        else:
            POPPLOT_values_sim(ax,map_dict[var1],map_dict[err1],pop_plot_range,color1,get_label(var1),line_alpha=line_alpha,surface_alpha=surface_alpha)

        ax.legend(loc=legend_loc)
        
    if True: # zero line
        minima = [np.nanmin(map_dict[var]-map_dict[err])]
        maxima = [np.nanmax(map_dict[var]+map_dict[err])]
        if var1_bool:
            minima.append(np.nanmin(map_dict[var1]-map_dict[err1]))
            maxima.append(np.nanmax(map_dict[var1]+map_dict[err1]))
            
        zero_line_within_lims = False
        if hard_coded_ylims_bool and var in hard_coded_ylims:
            zero_line_within_lims = hard_coded_ylims[var][0] < 0 and hard_coded_ylims[var][1] > 0
            
        zero_line_within_lims_1 = False
        if var1_bool and hard_coded_ylims_bool and var1 in hard_coded_ylims:
            zero_line_within_lims_1 = hard_coded_ylims[var1][0] < 0 and hard_coded_ylims[var1][1] > 0
        
        if zero_line_within_lims or zero_line_within_lims_1 or PH.shall_plot_zero_line(minima,maxima):
            ax.axhline(y=0,linestyle='--',color='grey')

if True: # axis
    POPPLOT_xaxis_settings(ax,xmin=min(pop_min_range),xmax=max(pop_max_range),xlabel=xlabel)
    POPPLOT_yaxis_settings(ax,var,err,map_dict,var1_bool=var1_bool)
    
    if not data_bool:
        ax.invert_xaxis()

if True: # save

    filename = var
    if var1_bool: filename += var1

    if filename == 'anisotropycorrelation': filename = 'anicorr'
    elif filename == 'mean_vxmean_vy': filename = 'vel'

    if data_bool and data_lines_bool:
        filename += "_lines"

    if bootstrap_repeat != 500:
        filename += f'_boot{bootstrap_repeat}'

    print(filename)

    if save_bool:
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting,bbox_inches='tight',dpi=300)
        print("Saved in",save_path)
plt.show()

metal_poor = data_extra[(data_extra['FeH']>-1) & (data_extra['FeH'] < -0.5)]
vr = metal_poor.vr.values
vl = metal_poor.vl.values

velocity_plot.velocity_plot(vr,vl,tilt_abs=True)

##### Optimise cut

In [ ]:
def compute_corr(plim,rlim,lowlim=data_extra['FeH'].min(),highlim=0.3):
    left_pop = data_extra[(data_extra['FeH']>lowlim)&(data_extra['FeH']<plim)]
    centre_pop = data_extra[(data_extra['FeH']>plim)&(data_extra['FeH']<rlim)]
    right_pop = data_extra[(data_extra['FeH']>rlim)&(data_extra['FeH']<highlim)]

    left_corr = CV.calculate_correlation(left_pop.vr.values,left_pop.vl.values)
    centre_corr = CV.calculate_correlation(centre_pop.vr.values,centre_pop.vl.values)
    right_corr = CV.calculate_correlation(right_pop.vr.values,right_pop.vl.values)
    
    return left_corr,centre_corr,right_corr

In [ ]:
lowlim = data_extra['FeH'].min()
highlim = 0.3

# Initial guesses
plim = -0.5
rlim = -0.2

step = 0.1
combinations = [[i,j] for i in [-step,0,step] for j in [-step,0,step]]

l,c,r = compute_corr(plim,rlim) # We want largest l, lowest |c|, lowest r

In [ ]:
iterations = 100

while iterations:    
    
    for comb in combinations:
        new_plim = plim+comb[0]
        new_rlim = rlim+comb[1]
        new_l,new_c,new_r = compute_corr(new_plim,new_rlim)
        
        if new_l > l and abs(new_c) < abs(c) and new_r < r:
            plim,rlim = new_plim,new_rlim
            l,c,r = new_l,new_c,new_r
        elif new_l > l and abs(new_c) < abs(c):
            plim = new_plim
            l,c = new_l,new_c
        elif abs(new_c) < abs(c) and new_r < r:
            rlim = new_rlim
            c,r = new_c,new_r
        
        #print("%.4f\t%.4f"%(plim, rlim))
        
    iterations -= 1

In [ ]:
plim, rlim

In [ ]:
left_pop = data_extra[(data_extra['FeH']>lowlim)&(data_extra['FeH']<plim)]
centre_pop = data_extra[(data_extra['FeH']>plim)&(data_extra['FeH']<rlim)]
right_pop = data_extra[(data_extra['FeH']>rlim)&(data_extra['FeH']<highlim)]

for pop in [left_pop,centre_pop,right_pop]:
    velocity_plot.velocity_plot(pop.vr.values,pop.vl.values)

#### Sim & data

This can't be used the same way as in the latitude plots because currently the code is run mutually exclusive way for data or sim. As a consequence, here we will load everything, including the main plot
* Run all the code above for the variations you want, in order to save the arrays
* The code below will load the main plot and the shadows, and create the plot

Note: currently I can only plot at the median if pop_plot_range.npy is saved at median values, as it requires having the whole array

##### Prepare

In [ ]:
pos_units_dict["b"] = "deg"

In [ ]:
def get_var_tuple_str(variable,var_tuple):
    if len(var_tuple) == 1:
        return f"{variable}{MF.return_int_or_dec(var_tuple[0],2)}"
    elif len(var_tuple) > 2:
        return f"{variable}" + ",".join([str(MF.return_int_or_dec(v,2)) for v in var_tuple])
    else:
        return f"{MF.return_int_or_dec(var_tuple[0],2)}{variable}{MF.return_int_or_dec(var_tuple[1],2)}"

def get_variation_str(var,all_value_tuples):
    var_str = ""

    for val_tuple in all_value_tuples:
        var_str += get_var_tuple_str(var,val_tuple) + "_"
    
    return var_str.removesuffix("_") + "/"

def get_save_path_POPPLOT_both(save_path,extra_var,depth_var,height_var,sim_bool=True,data_bool=True,binning_str_sim=None,binning_str_data=None,pop_str_data=None,\
                               pop_str_sim=None,extra_min=None,extra_max=None,depth_min=None,depth_max=None,height_min=None,height_max=None,all_variations_dict=None):
        
    for var,m,M in zip([extra_var,depth_var,height_var],[extra_min,depth_min,height_min],[extra_max,depth_max,height_max]):
        save_path += get_variation_str(var,all_variations_dict[var]) if var in all_variations_dict else get_var_tuple_str(var,[m,M])+"/"
        MF.create_dir(save_path)
    
    save_path += "both/"
    MF.create_dir(save_path)
    
    if data_bool:
        save_path += f"data_{binning_str_data}/"
        MF.create_dir(save_path)

        save_path += "data_"+get_variation_str("n",all_variations_dict["n_data"]) if "n_data" in all_variations_dict else f"data_{pop_str_data}/"
        MF.create_dir(save_path)
    
    if sim_bool:
        save_path += f"sim_{binning_str_sim}/"
        MF.create_dir(save_path)

        save_path += "sim_"+get_variation_str("n",all_variations_dict["n_sim"]) if "n_sim" in all_variations_dict else f"sim_{pop_str_sim}/"
        MF.create_dir(save_path)

    return save_path

In [ ]:
def load_values_and_plot_ranges(path,full_map_string_list):
    
    map_dict = {}
    for m in full_map_string_list:
        map_dict[m] = np.load(f"{path}{m}.npy")
    
    min_range = np.load(path + f"pop_min_range.npy")
    max_range = np.load(path + f"pop_max_range.npy")
    plot_range = np.load(path + f"pop_plot_range.npy")
    
    return map_dict, min_range, max_range, plot_range

In [ ]:
depth_variable_lims = {
    "l": [-2,2],
    "y": [coordinates.ang_to_rect(ang=-2,x=coordinates.get_solar_radius()), coordinates.ang_to_rect(ang=2,x=coordinates.get_solar_radius())]
}

overall_height_variable_lims = {
    "b": [1.5,9],
    "z": [coordinates.ang_to_rect(ang=1.5,x=coordinates.get_solar_radius()), coordinates.ang_to_rect(ang=9,x=coordinates.get_solar_radius())]
}

extra_variable_lims = {
    "d": [6.1,10.1],
    "R": [0,3.5]
}

# CHOOSE variables for selection
# extra_variable = "d"
extra_variable = "R"
depth_var = "l"
height_var = "b"

if True: # print
    depth_min,depth_max = depth_variable_lims[depth_var]
    overall_height_min,overall_height_max = overall_height_variable_lims[height_var]
    extra_min,extra_max = extra_variable_lims[extra_variable]
    
    print(depth_var,depth_min,depth_max)
    print(height_var,overall_height_min,overall_height_max)
    print(extra_variable,extra_min,extra_max)

In [ ]:
# see https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3

# colors = ["blue","blue","blue"]
colors = ["#1b9e77","#d95f02","#7570b3"]

line_alpha = 0.9
surface_alpha = 0.75
alpha_reduction_factor = 0.25

number_alpha = surface_alpha
number_alpha_reduction_factor = 0.5

In [ ]:
binning_str_data = "equalN"
binning_str_sim = "equalN"

In [ ]:
current_first = True
# current_first = False

# current_first_N = True
current_first_N = False

In [ ]:
data_bool = True
# data_bool = False

sim_bool = True
# sim_bool = False

In [ ]:
if True: # height_variations

    height_range_min,height_range_max = PH.get_equal_n_minmax_b_ranges(data_trim, n_points=3,\
                                                                 extra_variable=extra_variable,extra_min=extra_min,extra_max=extra_max,\
                                                                 depth_min=depth_variable_lims["l"][0],depth_max=depth_variable_lims["l"][1],\
                                                                 overall_bmin=overall_height_variable_lims["b"][0],overall_bmax=overall_height_variable_lims["b"][1])
    
    if height_var == "z":
        height_range_min = [coordinates.ang_to_rect(ang=bmin,x=coordinates.get_solar_radius()) for bmin in height_range_min]
        height_range_max = [coordinates.ang_to_rect(ang=bmax,x=coordinates.get_solar_radius()) for bmax in height_range_max]
    
    if True: # choose parameters
        extra_variations = [[0,3.5],[0,2]]
        height_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(height_range_min,height_range_max)]

        n_extra_var,n_height_var = len(extra_variations),len(height_variations)

        if data_bool:
            n_data = 4
            n_variations_data = n_extra_var*[[n_data,n_data,n_data-1]]
            metal_str_variations = [[f"{x}_datapoints" for x in n_variations_data[i]] for i in range(len(n_variations_data))]

        if sim_bool:
            if sim_resampled_bool:
                n_variations_sim = n_variations_data
            else:
                n_sim = 20
                n_variations_sim = n_extra_var*[[n_sim,n_sim,n_sim-5]]
            age_str_variations = [[f"{x}_datapoints" for x in n_variations_sim[i]] for i in range(len(n_variations_sim))]

        all_colors = n_extra_var*[colors]
        line_alphas = [n_height_var*[line_alpha], n_height_var*[line_alpha*alpha_reduction_factor]]
        surface_alphas = [n_height_var*[surface_alpha], n_height_var*[surface_alpha*alpha_reduction_factor]]
        number_alphas = [n_height_var*[number_alpha], n_height_var*[number_alpha*number_alpha_reduction_factor]]

        z_orders = [[len(extra_variations)*len(height_variations)-i + (-j if current_first else j) for j in range(len(height_variations))]\
                     for i in np.arange(0,len(extra_variations)*len(height_variations),len(height_variations))]
        z_orders_N = [[len(extra_variations)*len(height_variations)-i + (-j if current_first_N else j) for j in range(len(height_variations))]\
                       for i in np.arange(0,len(extra_variations)*len(height_variations),len(height_variations))]
    
    all_dicts = []
    
    for i,(em,eM) in enumerate(extra_variations):
        for j,(hm,hM) in enumerate(height_variations):
            
            current_dict = {
                "color":all_colors[i][j],
                "line_alpha":line_alphas[i][j],
                "surface_alpha":surface_alphas[i][j],
                "number_alpha":number_alphas[i][j],
                "zorder": z_orders[i][j],
                "zorderN": z_orders_N[i][j],
                "label_sim": get_legend_label([hm,hM],height_var) if i==0 else None,
                "label_data": None
            }
            
            if data_bool:
                spatial_path = get_save_path_POPPLOT_spatial_cuts(resampled_sim_bool=False,\
                                                                  extra_variable=extra_variable,extra_min=em,extra_max=eM,\
                                                                  depth_var=depth_var,depth_min=depth_min,depth_max=depth_max,\
                                                                  height_var=height_var,height_min=hm,height_max=hM)
                
                load_path_data = get_save_path_POPPLOT_data(save_path_spatial=spatial_path, binning_str=binning_str_data,\
                                                        pop_str=metal_str_variations[i][j]) + "arrays/"
                current_dict["load_path_data"] = load_path_data
                
            if sim_bool:
                spatial_path = get_save_path_POPPLOT_spatial_cuts(resampled_sim_bool=sim_resampled_bool,\
                                                                  extra_variable=extra_variable,extra_min=em,extra_max=eM,\
                                                                  depth_var=depth_var,depth_min=depth_min,depth_max=depth_max,\
                                                                  height_var=height_var,height_min=hm,height_max=hM)
                
                load_path_sim = get_save_path_POPPLOT_sim(save_path_spatial=spatial_path, binning_str=binning_str_sim,\
                                                      pop_str=age_str_variations[i][j]) + "arrays/"
                current_dict["load_path_sim"] = load_path_sim
            
            all_dicts.append(current_dict)
            
    all_variations_dict = {extra_variable:extra_variations,height_var:height_variations}
    if data_bool:
        all_variations_dict["n_data"] = n_variations_data
    if sim_bool:
        all_variations_dict["n_sim"] = n_variations_sim
    
    save_path = get_save_path_POPPLOT_both(save_path=get_base_path(sim_resampled_bool),\
                                           extra_var=extra_variable,depth_var=depth_var,height_var=height_var,\
                                           binning_str_sim=binning_str_sim,binning_str_data=binning_str_data,\
                                           depth_min=depth_min,depth_max=depth_max,\
                                           all_variations_dict=all_variations_dict,
                                           data_bool=data_bool,sim_bool=sim_bool)

print(save_path)

In [ ]:
if sim_bool:
    for dicts in all_dicts:
        print(dicts["load_path_sim"])

if data_bool:
    print("\n")
    for dicts in all_dicts:
        print(dicts["load_path_data"])

##### Plot

In [ ]:
sim_surface_bool = True
# sim_surface_bool = False

data_surface_bool = True
# data_surface_bool = False

In [ ]:
# everything below only takes effect if surface_bool is False

capsize = 5

data_xerror_bool = True
# data_xerror_bool = False
data_lines_bool = True
# data_lines_bool = False

sim_xerror_bool = True
# sim_xerror_bool = False
sim_lines_bool = True
# sim_lines_bool = False

In [ ]:
# sim_title = "Model"
# data_title = "Data"

sim_title = "Galactic spatial cuts"
data_title = "Rectangular spatial cuts"


sim_xlabel = r"Age [Gyr]"
# data_xlabel = "[Fe/H]"
data_xlabel = r"Age [Gyr]"

invert_sim_xaxis = True
invert_data_xaxis = True

In [ ]:
# To plot at the median we can only get the plotting locations from pop_plot_range.npy (assuming it was saved at the median) as they require having the full array

data_plot_range_str = "mean"
# data_plot_range_str = "median"

sim_plot_range_str = "mean"
# sim_plot_range_str = "median"

In [ ]:
number_bool = True
# number_bool = False

bar_log = True
# bar_log = False

# number_variations_bool = True
number_variations_bool = False

idx_variations_begin = 3

In [ ]:
hard_coded_ylims_bool = True # currently needed to ensure the y axis is shared correctly
# hard_coded_ylims_bool = False

# symmetric_ylims_bool = True
symmetric_ylims_bool = False

hard_coded_ylims = {
    "tilt_abs": [-45,3],
}

yshift_dict = {
    "tilt_abs": 2,
    "anisotropy": 0.02,
    "correlation": 0.01,
    "mean_vx": 1,
    "mean_vy": 1,
    "std_vx": 2,
    "std_vy": 2
}

In [ ]:
legend_row = 1
# legend_row = len(map_list)

# legend_loc = "best"; ncols_leg=1
# legend_loc = "lower left"; ncols_leg=1

if sim_bool and data_bool:
    legend_loc = (0.25,1.25 if not number_bool else 1.65); ncols_leg=len(colors)
else:
    legend_loc = (-0.25,1.05 if not number_bool else 1.45); ncols_leg=len(colors)

# legend_bool = True
legend_bool = False

# legend_data_bool = True
legend_data_bool = False

In [ ]:
zero_line_color = "grey"
zero_line_alpha = 1

In [ ]:
plt.rcParams["font.size"] = 22
ylabel_size = 24
legend_fontsize = 15.5

In [ ]:
figsize_x = 15

# figsize_y = 14/3*0.9*len(map_list)
figsize_y = 1.41*figsize_x # aspect ratio of A4

figsize_x /= 2 if sim_bool+data_bool == 1 else 1

In [ ]:
map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy","correlation","tilt_abs"]; map_list_string = "all"
# map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "meanstdani"
# map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
# map_list = ["mean_vx","mean_vy"]; map_list_string = "velmeans"

In [ ]:
filename_suffix = ""

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
save_path = "/Users/luismi/Desktop/MRes_UCLan/graphs/Observations/Apogee/individual_variable/age_metal/0R3.5_0R2/-2l2__-0.28y0.28/"+\
            "1.5b3.51_3.51b6.6_7.13b8.85__0.21z0.5_0.5z0.94_1.01z1.26/sim/sim_equalN/sim_n20,20,15_n20,20,15__n20,20,15_n20,20,15/"

os.makedirs(save_path,exist_ok=True)

In [ ]:
save_path

In [ ]:
# plot

fig,axs = plt.subplots(figsize=(figsize_x,figsize_y),nrows=len(map_list)+1,ncols=sim_bool+data_bool, facecolor='w',\
                       gridspec_kw={'hspace':0,'wspace':0,'height_ratios':[0.35]+len(map_list)*[1]})

if True: # select axes
    if sim_bool and data_bool: 
        sim_axes,data_axes = axs[:,0], axs[:,1]
    elif sim_bool:
        sim_axes = axs
    elif data_bool:
        data_axes = axs
    else:
        raise ValueError("Both sim_bool and data_bool were set to False...")

if number_bool: # title and number bars
    if sim_bool and data_bool:
        sim_axes[0].set_title(sim_title)
        data_axes[0].set_title(data_title)
    
    bar_n_min,bar_n_max = [10**30],[0]
    
    for i,d in enumerate(all_dicts):
        if not number_variations_bool and i >= idx_variations_begin:
            break
        
        if sim_bool:
            map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(d["load_path_sim"],full_map_string_list)
            plot_range = PH.get_range_means(min_range,max_range)
            POPPLOT_number_bar(sim_axes[0], plot_range, map_dict["number"],color=d["color"],alpha=d["number_alpha"],zorder=d["zorderN"])
            
            bar_n_min = min(bar_n_min, np.min(map_dict["number"]))
            bar_n_max = max(bar_n_max, np.max(map_dict["number"]))
        
        if data_bool:
            o_map_dict,o_min_range,o_max_range,o_plot_range = load_values_and_plot_ranges(d["load_path_data"],full_map_string_list)
            o_plot_range = o_plot_range if data_plot_range_str=="median" else PH.get_range_means(o_min_range,o_max_range)
            POPPLOT_number_bar(data_axes[0], o_plot_range, o_map_dict["number"],color=d["color"],alpha=d["number_alpha"],zorder=d["zorderN"])
    
            bar_n_min = min(bar_n_min, np.min(o_map_dict["number"]))
            bar_n_max = max(bar_n_max, np.max(o_map_dict["number"]))
    
    if sim_bool: POPPLOT_number_bar_axis_settings(sim_axes[0],min_n=bar_n_min,max_n=bar_n_max,bar_log=bar_log,min_shift_bool=False,labels_on=not data_bool)
    if data_bool: POPPLOT_number_bar_axis_settings(data_axes[0],min_n=bar_n_min,max_n=bar_n_max,bar_log=bar_log,min_shift_bool=False,labels_on=data_bool)
else:
    if sim_bool: fig.delaxes(sim_axes[0])
    if data_bool: fig.delaxes(data_axes[0])
    
    if sim_bool and data_bool:
        sim_axes[1].set_title("Model")
        data_axes[1].set_title("Data")

for row,map_string in enumerate(map_list): # plot
    error_string = map_string+"_error"
    
    ymin,ymax = [float("inf")],[float("-inf")]
    
    for i,d in enumerate(all_dicts):
        
        if sim_bool:
            map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(d["load_path_sim"],full_map_string_list)
            plot_range = plot_range if sim_plot_range_str=="median" else PH.get_range_means(min_range,max_range)
            label = d["label_sim"] if row == legend_row-1 else None
            
            if sim_surface_bool:
                POPPLOT_values_surface(sim_axes[row+1],map_dict[map_string],map_dict[error_string],plot_range,\
                               color=d["color"],label=label,line_alpha=d["line_alpha"],surface_alpha=d["surface_alpha"],zorder=d["zorder"])
            else:
                POPPLOT_values_scatter(sim_axes[row+1],map_dict[map_string],map_dict[error_string],plot_range,min_range,max_range,\
                           color=d["color"],label=label,line_alpha=d["line_alpha"],zorder=d["zorder"],lines_bool=sim_lines_bool,x_error_bool=sim_xerror_bool)
            
            ymin = min(ymin, np.nanmin(map_dict[map_string]-map_dict[error_string]))
            ymax = max(ymax, np.nanmax(map_dict[map_string]+map_dict[error_string]))
            
            POPPLOT_yaxis_settings(sim_axes[row+1],map_string,error_string,set_ylims=False)
            
            sim_axes[row+1].axhline(y=0,linestyle='--',color=zero_line_color,alpha=zero_line_alpha,zorder=0)
            
        if data_bool:
            o_map_dict,o_min_range,o_max_range,o_plot_range = load_values_and_plot_ranges(d["load_path_data"],full_map_string_list)
            o_plot_range = o_plot_range if data_plot_range_str=="median" else PH.get_range_means(o_min_range,o_max_range)
            o_label = d["label_data"] if row == legend_row-1 else None
            
            if data_surface_bool:
                POPPLOT_values_surface(data_axes[row+1],o_map_dict[map_string],o_map_dict[error_string],o_plot_range,\
                               color=d["color"],label=o_label,line_alpha=d["line_alpha"],surface_alpha=d["surface_alpha"],zorder=d["zorder"])
            else:
                POPPLOT_values_scatter(data_axes[row+1],o_map_dict[map_string],o_map_dict[error_string],o_plot_range,o_min_range,o_max_range,\
                           color=d["color"],label=o_label,line_alpha=d["line_alpha"],zorder=d["zorder"],lines_bool=data_lines_bool,x_error_bool=data_xerror_bool)
        
            ymin = min(ymin, np.nanmin(o_map_dict[map_string]-o_map_dict[error_string]))
            ymax = max(ymax, np.nanmax(o_map_dict[map_string]+o_map_dict[error_string]))
    
            POPPLOT_yaxis_settings(data_axes[row+1],map_string,error_string,labels_on=False,set_ylims=False)

            data_axes[row+1].axhline(y=0,linestyle='--',color=zero_line_color,alpha=zero_line_alpha,zorder=0)
    
    if map_string in yshift_dict:
        ymin -= yshift_dict[map_string]
        ymax += yshift_dict[map_string]
    
    if hard_coded_ylims_bool and map_string in hard_coded_ylims:
        ymin,ymax = hard_coded_ylims[map_string]
    
    if sim_bool: sim_axes[row+1].set_ylim(ymin,ymax)
    if data_bool: data_axes[row+1].set_ylim(ymin,ymax)

if legend_bool: # legend
    if sim_bool: 
        sim_axes[legend_row].legend(loc=legend_loc,fontsize=legend_fontsize,ncols=ncols_leg)#,loc="lower left")
    if data_bool and legend_data_bool: 
        data_axes[legend_row].legend(loc=legend_loc,fontsize=legend_fontsize,ncols=ncols_leg)
    
if True: # x-axis
    
    if sim_bool: 
        _,min_range,max_range,_ = load_values_and_plot_ranges(all_dicts[0]["load_path_sim"],full_map_string_list)
    if data_bool:
        _,o_min_range,o_max_range,_ = load_values_and_plot_ranges(all_dicts[0]["load_path_data"],full_map_string_list)
    
    for row in range(len(map_list)+1): # include number bars
        
        x_labels_on = row == len(map_list)
        
        if sim_bool:
            age_ticks = np.arange(min(min_range)+1,max(max_range)+1,1)
            POPPLOT_xaxis_settings(sim_axes[row],xmin=min(min_range),xmax=max(max_range),xlabel=sim_xlabel,\
                                   xticks=age_ticks,labels_on=x_labels_on)
            if invert_sim_xaxis:
                sim_axes[row].invert_xaxis()
        
        if data_bool:
#             metal_ticks = np.arange(-0.9,0.6+0.3,0.3)
            metal_ticks = np.arange(min(o_min_range)+1,max(o_max_range)+1,1)
            POPPLOT_xaxis_settings(data_axes[row],xmin=min(o_min_range),xmax=max(o_max_range),xlabel=data_xlabel,xticks=metal_ticks,labels_on=x_labels_on)
            
            if invert_data_xaxis:
                data_axes[row].invert_xaxis()
    
    fig.align_labels()
    
if True: # save
    
    filename = "kinpop_" + map_list_string
    
    filename += "_noN" if not number_bool else ""
    
    filename += "_noNvar" if number_bool and not number_variations_bool else ""
    
    filename += "_noLeg" if not legend_bool else ""
    
    if not data_lines_bool:
        filename += "_noDataLines"
    
    filename += filename_suffix
    
    print(filename)
    
    if save_bool:
        print("Saving in",save_path)
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

#### Custom

##### Prepare

In [ ]:
pos_units_dict["b"] = "deg"

In [ ]:
def load_values_and_plot_ranges(path,full_map_string_list):
    
    map_dict = {}
    for m in full_map_string_list:
        map_dict[m] = np.load(f"{path}{m}.npy")
    
    min_range = np.load(path + f"pop_min_range.npy")
    max_range = np.load(path + f"pop_max_range.npy")
    plot_range = np.load(path + f"pop_plot_range.npy")
    
    return map_dict, min_range, max_range, plot_range

In [ ]:
# Some reference values

l_cuts = [-2,2]
y_cuts = MF.return_int_or_dec_for_array([coordinates.ang_to_rect(ang=l_cut,x=coordinates.get_solar_radius()) for l_cut in l_cuts])

b_range_min,b_range_max = PH.get_equal_n_minmax_b_ranges(data_trim)
b_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(b_range_min,b_range_max)]

z_range_min = [coordinates.ang_to_rect(ang=bmin,x=coordinates.get_solar_radius()) for bmin in b_range_min]
z_range_max = [coordinates.ang_to_rect(ang=bmax,x=coordinates.get_solar_radius()) for bmax in b_range_max]
z_variations = [[MF.return_int_or_dec(m,2),MF.return_int_or_dec(M,2)] for (m,M) in zip(z_range_min,z_range_max)]

R_variations = [[0,3.5],[0,2]]

print("l",l_cuts)
print("y",y_cuts)
print("b",b_variations)
print("z",z_variations)
print("R",R_variations)

In [ ]:
# see https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3

# colors = ["blue","blue","blue"]
colors = ["#1b9e77","#d95f02","#7570b3"]

line_alpha = 0.9
surface_alpha = 0.75
alpha_reduction_factor = 0.25

number_alpha = 0.75
number_alpha_reduction_factor = 0.5

In [ ]:
# sim galactic vs rectangular 3 columns

# binning_type = "equalN"
binning_type = "equalSteps"

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/1.5b3.51/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/0.21z0.5/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/1.5b3.51/sim/{binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/0.21z0.5/sim/{binning_type}/40_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([1.5,3.51],"b"),get_legend_label([0.21,0.5],"z")],
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/3.51b6.6/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/0.5z0.94/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/3.51b6.6/sim/{binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/0.5z0.94/sim/{binning_type}/20_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([3.51, 6.6],"b"),get_legend_label([0.5,0.94],"z")],
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/7.13b8.85/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-0.28y0.28/1.01z1.26/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/7.13b8.85/sim/{binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-0.28y0.28/1.01z1.26/sim/{binning_type}/10_datapoints/arrays/"
        ],
        "labels": [None,None,get_legend_label([7.13, 8.85],"b"),get_legend_label([0.5,0.94],"z")],
    },   
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            f"0R3.5_0R2/-2l2_-0.28y0.28/1.5b3.51_0.21z0.5__3.51b6.6_0.5z0.94__7.13b8.85_1.01z1.26/sim/sim_{binning_type}/sim_n40,40_n40,40__n20,20_n20,20__n10,10_n10,10/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,6,7,8]
    all_dicts[key]["colors"] = 2*["blue", "orange"]
    all_dicts[key]["line_alphas"] = 2*[alpha_reduction_factor*line_alpha] + 2*[line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[alpha_reduction_factor*surface_alpha] + 2*[surface_alpha]
    all_dicts[key]["number_alphas"] = 2*[number_alpha_reduction_factor*number_alpha] + 2*[number_alpha]
    all_dicts[key]["zorderNs"] = [6,6,5,5]
    all_dicts[key]["plot_ranges_str"] = nplots*["mean"]
    all_dicts[key]["surface_bools"] = nplots*[True]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [False,False,True,True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["title"] = None
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(5,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# sim normal vs resampled 3 columns

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/1.5b3.51/sim/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/1.5b3.51/sim/equalN/4_datapoints/arrays/",

        ],
        "title": get_legend_label([1.5,3.51],"b"),
        "labels": 4*[None],
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalN/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/3.51b6.6/sim/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/3.51b6.6/sim/equalN/4_datapoints/arrays/",
        ],
        "title": get_legend_label([3.51,6.6],"b"),
        "labels": [None,"Model",None,"Resampled model (k=10)"]
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalN/15_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalN/15_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R2/-2l2/7.13b8.85/sim/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + "0R3.5/-2l2/7.13b8.85/sim/equalN/3_datapoints/arrays/",
        ],
        "title": get_legend_label([7.13,8.85],"b"),
        "labels": 4*[None]
    },
    
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "sim__resampled_sim/0R3.5_0R2/-2l2/1.5b3.51__3.51b6.6__7.13b8.85/sim_equalN/sim_n20,20_n4,4__n20,20_n4,4__n15,15_n3,3/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,6,7,8]
    all_dicts[key]["zorderNs"] = [5,5,6,6]
    all_dicts[key]["colors"] = ["blue","blue","orange","orange"]
    
    all_dicts[key]["line_alphas"] = 2*[alpha_reduction_factor*line_alpha, line_alpha]
    all_dicts[key]["surface_alphas"] = 2*[alpha_reduction_factor*surface_alpha, surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["plot_ranges_str"] = ["mean","mean","median","median"]
    all_dicts[key]["surface_bools"] = [True,True,False,False]
    all_dicts[key]["scatter_join_bools"] = nplots*[True]
    all_dicts[key]["number_bools"] = [False,True,False,True]
    
    all_dicts[key]["invert_xaxis"] = True
    all_dicts[key]["xaxis_label"] = "Age [Gyr]"
    all_dicts[key]["x_ticks"] = np.arange(5,11,1) #np.arange(-0.9,0.6+0.3,0.3)
    all_dicts[key]["x_lims"] = [4,10]

In [ ]:
# data and sim 2 columns (sim equalN)

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61]
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalN/25_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalN/30_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalN/25_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10]
    }, 
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN/data_n4,4,3_n4,4,3/sim_equalN/sim_n30,30,25_n30,30,25/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
# data and sim 2 columns (sim equalSteps)

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61]
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/sim/equalSteps/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/sim/equalSteps/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/sim/equalSteps/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/sim/equalSteps/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/sim/equalSteps/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/sim/equalSteps/10_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(4,10,1),
        "x_lims": [4,10]
    }, 
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            "0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN/data_n4,4,3_n4,4,3/sim_equalSteps/sim_n40,20,10_n40,20,10/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
# data, sim and sim resampled 3 columns

# sim_binning_type = "equalN"
sim_binning_type = "equalSteps"

all_dicts = {
    0: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R2/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/1.5b3.51/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/3.51b6.6/data/equalN/4_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + "0R3.5/-2l2/7.13b8.85/data/equalN/3_datapoints/arrays/",
        ],
        "title": "Data",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": False,
        "xaxis_label": "[Fe/H]",
        "x_ticks": np.arange(-0.9,0.6+0.3,0.3),
        "x_lims": [-1,0.61],
        "bar_width": 0.03
    },
    1: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/1.5b3.51/sim/{sim_binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/3.51b6.6/sim/{sim_binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R2/-2l2/7.13b8.85/sim/{sim_binning_type}/10_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/1.5b3.51/sim/{sim_binning_type}/40_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/3.51b6.6/sim/{sim_binning_type}/20_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=False) + f"0R3.5/-2l2/7.13b8.85/sim/{sim_binning_type}/10_datapoints/arrays/",

        ],
        "title": "Model",
        "labels": 3*[None] + [get_legend_label([1.5,3.51],"b"), get_legend_label([3.51,6.6],"b"), get_legend_label([7.13,8.85],"b")],
        "plot_ranges_str": 6*["mean"],
        "surface_bools": 6*[True],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10],
        "bar_width": 0.05
    },
    2: {
        "load_paths": [
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/1.5b3.51/sim/{sim_binning_type}/8_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/3.51b6.6/sim/{sim_binning_type}/5_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R2/-2l2/7.13b8.85/sim/{sim_binning_type}/3_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/1.5b3.51/sim/{sim_binning_type}/8_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/3.51b6.6/sim/{sim_binning_type}/6_datapoints/arrays/",
            get_base_POPPLOT_path(resampled_sim_bool=True) + f"0R3.5/-2l2/7.13b8.85/sim/{sim_binning_type}/4_datapoints/arrays/",
        ],
        "title": "Resampled model",
        "labels": 6*[None],
        "plot_ranges_str": 6*["median"],
        "surface_bools": 6*[False],
        "invert_xaxis": True,
        "xaxis_label": "Age [Gyr]",
        "x_ticks": np.arange(5,10,1),
        "x_lims": [4,10],
        "bar_width": 0.12
    }    
}

save_path = get_base_POPPLOT_path(resampled_sim_bool=False) + \
            f"data__sim__resampled_sim/0R3.5_0R2/-2l2/1.5b3.51_3.51b6.6_7.13b8.85/both/data_equalN__sim_{sim_binning_type}/"+\
            "data_n4,4,3_n4,4,3__sim_n40,20,10_n40,20,10__resampledsim_n8,6,4_n8,5,3/"

ncols = len(all_dicts)

for key in all_dicts: # this loop sets features/settings that share the same values across the columns
    
    nplots = len(all_dicts[key]["load_paths"])
    
    all_dicts[key]["zorders"] = [5,4,3,8,7,6]
    all_dicts[key]["zorderNs"] = [5,5,5,6,7,8]
    all_dicts[key]["colors"] = 2*colors
    
    all_dicts[key]["line_alphas"] = 3*[alpha_reduction_factor*line_alpha] + 3*[line_alpha]
    all_dicts[key]["surface_alphas"] = 3*[alpha_reduction_factor*surface_alpha] + 3*[surface_alpha]
    all_dicts[key]["number_alphas"] = nplots*[number_alpha]
    
    all_dicts[key]["number_bools"] = 3*[False] + 3*[True]

In [ ]:
for key in all_dicts:
    print(key)
    for path in all_dicts[key]["load_paths"]:
        print(path)
        if not os.path.isdir(path):
            raise ValueError("Path did not exist!")

In [ ]:
os.makedirs(save_path,exist_ok=True)

print(save_path)

##### Plot

In [ ]:
capsize = 5

scatter_join_bool = True
# scatter_join_bool = False

In [ ]:
number_bool = True
# number_bool = False

# number_variations_bool = True
number_variations_bool = False

bar_log = True
# bar_log = False

if number_bool: # check there are actually any to be plotted
    
    any_number_bool = False
    for key in all_dicts:
        any_number_bool = any_number_bool or any(all_dicts[key]["number_bools"])
        
    if not any_number_bool:
        number_bool = False
        print("No plot with number bool to draw, setting number_bool=False")

In [ ]:
hard_coded_ylims_bool = True # currently needed to ensure the y axis is shared correctly
# hard_coded_ylims_bool = False

# symmetric_ylims_bool = True
symmetric_ylims_bool = False

hard_coded_ylims = {
#     "tilt_abs": [-45,3],
    "tilt_abs": [-45,45],
}

yshift_dict = {
    "tilt_abs": 2,
    "anisotropy": 0.02,
    "correlation": 0.01,
    "mean_vx": 1,
    "mean_vy": 1,
    "std_vx": 2,
    "std_vy": 2
}

In [ ]:
def get_legend_params(ncols, all_dicts,number_bool):
    
    if ncols != len(all_dicts):
        raise ValueError("Expected the `ncols` to match the length of `all_dicts`.")
    
    legend_cols = []
    for k in all_dicts.keys():
        if any(all_dicts[k]["labels"]):
            legend_cols.append(k)
    
    ncols_leg = len(np.unique(all_dicts[legend_cols[0]]["colors"])) if len(legend_cols) == 1 else 1
    
    if ncols == 2:
        loc_x = -0.75
    elif ncols == 3:
        loc_x = -0.6 if len(legend_cols) == 1 else 0.16
    elif ncols == 1:
        loc_x = -0.25
        
    legend_loc = [loc_x, 1.65]
    
    if not any(any(all_dicts[k]["number_bools"]) for k in all_dicts.keys()):
        legend_loc[1] -= 0.4
        
    if not any(all_dicts[k]["title"] for k in all_dicts.keys()):
        legend_loc[1] -= 0.15
    
    return legend_cols, legend_loc, ncols_leg
            
legend_row = 1
# legend_row = len(map_list)

legend_cols, legend_loc, ncols_leg = get_legend_params(ncols, all_dicts,number_bool)
# legend_loc = "upper right"; ncols_leg=1
# legend_cols = [1]

legend_bool = True
# legend_bool = False

if legend_bool: # check there are actually any labels to be plotted
    
    any_legend_bool = False
    for key in all_dicts:
        any_legend_bool = any_legend_bool or any(all_dicts[key]["labels"])
        
    if not any_legend_bool:
        legend_bool = False
        print("No plot with label, setting legend_bool=False")

In [ ]:
zero_line_color = "grey"
zero_line_alpha = 1

In [ ]:
plt.rcParams["font.size"] = 22
ylabel_size = 24
legend_fontsize = 15.5

In [ ]:
figsize_x = 15

figsize_y = 1.41*figsize_x # aspect ratio of A4

figsize_x /= 2 if ncols == 1 else 1

In [ ]:
map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy","correlation","tilt_abs"]; map_list_string = "all"
# map_list = ["anisotropy","correlation","tilt_abs"]; map_list_string = "anicorrtilt"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy"]; map_list_string = "velmeanstd"
# map_list = ["mean_vx","mean_vy","std_vx","std_vy","anisotropy"]; map_list_string = "meanstdani"
# map_list = ["std_vx","std_vy","anisotropy"]; map_list_string = "stdani"
# map_list = ["mean_vx","mean_vy"]; map_list_string = "velmeans"

In [ ]:
filename_suffix = ""

In [ ]:
save_bool = True
# save_bool = False

In [ ]:
# plot

fig,axs = plt.subplots(figsize=(figsize_x,figsize_y),nrows=len(map_list)+1,ncols=ncols, facecolor='w',\
                       gridspec_kw={'hspace':0,'wspace':0,'height_ratios':[0.35]+len(map_list)*[1]})

if number_bool: # number and title
    bar_n_min,bar_n_max = [10**30],[0]

    for col in all_dicts:
        dic = all_dicts[col]

        axs[0,col].set_title(dic["title"])

        for i in range(len(dic["load_paths"])):
            if dic["number_bools"][i]:
                map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
                
                if dic["plot_ranges_str"][i] == "mean":
                    plot_range = PH.get_range_means(min_range,max_range)

                POPPLOT_number_bar(axs[0,col], plot_range, map_dict["number"],color=dic["colors"][i],alpha=dic["number_alphas"][i],\
                                   zorder=dic["zorderNs"][i], bar_width=dic["bar_width"])

                bar_n_min = min(bar_n_min, np.min(map_dict["number"]))
                bar_n_max = max(bar_n_max, np.max(map_dict["number"]))
    
    for col in all_dicts:
        POPPLOT_number_bar_axis_settings(axs[0,col],min_n=bar_n_min,max_n=bar_n_max,bar_log=bar_log,labels_on=col==ncols-1)        
else:
    for col in all_dicts:
        fig.delaxes(axs[0,col])
        
        axs[1,col].set_title(all_dicts[col])

for row,map_string in enumerate(map_list): # plot
    error_string = map_string+"_error"
    
    ymin,ymax = [float("inf")],[float("-inf")]
    
    for col in all_dicts:
        
        POPPLOT_yaxis_settings(axs[row+1,col],map_string,error_string,set_ylims=False,labels_on=col==0)
        axs[row+1,col].axhline(y=0,linestyle='--',color=zero_line_color,alpha=zero_line_alpha,zorder=0)
        
        dic = all_dicts[col]
        for i in range(len(dic["load_paths"])):
            
            map_dict,min_range,max_range,plot_range = load_values_and_plot_ranges(dic["load_paths"][i],full_map_string_list)
            plot_range = plot_range if dic["plot_ranges_str"][i]=="median" else PH.get_range_means(min_range,max_range)
            label = dic["labels"][i] if row == legend_row-1 else None
            
            if dic["surface_bools"][i]:
                POPPLOT_values_surface(axs[row+1,col],map_dict[map_string],map_dict[error_string],plot_range,color=dic["colors"][i],label=label,\
                                       line_alpha=dic["line_alphas"][i],surface_alpha=dic["surface_alphas"][i],zorder=dic["zorders"][i])
            else:
                POPPLOT_values_scatter(axs[row+1,col],map_dict[map_string],map_dict[error_string],plot_range,min_range,max_range,color=dic["colors"][i],label=label,\
                                       line_alpha=dic["line_alphas"][i],zorder=dic["zorders"][i],lines_bool=scatter_join_bool)
            
            ymin = min(ymin, np.nanmin(map_dict[map_string]-map_dict[error_string]))
            ymax = max(ymax, np.nanmax(map_dict[map_string]+map_dict[error_string]))
    
    if map_string in yshift_dict:
        ymin -= yshift_dict[map_string]
        ymax += yshift_dict[map_string]
    
    if hard_coded_ylims_bool and map_string in hard_coded_ylims:
        ymin,ymax = hard_coded_ylims[map_string]
    
    for col in all_dicts:
        axs[row+1,col].set_ylim(ymin,ymax)

if legend_bool: # legend
    for col in all_dicts:
        if col in legend_cols:
            axs[legend_row,col].legend(loc=legend_loc,fontsize=legend_fontsize,ncols=ncols_leg)#,loc="lower left")
    
if True: # x-axis
        
    for col in all_dicts:
        
        dic = all_dicts[col]
    
        for row in range(len(map_list)+1): # include number bars

            POPPLOT_xaxis_settings(axs[row,col],xmin=dic["x_lims"][0],xmax=dic["x_lims"][1],xlabel=dic["xaxis_label"],xticks=dic["x_ticks"],labels_on=row==len(map_list))

            if dic["invert_xaxis"]:
                axs[row,col].invert_xaxis()
    
    fig.align_labels()
    
if True: # save
    
    filename = "kinpop_" + map_list_string
    
    filename += "_noN" if not number_bool else ""
    
    filename += "_noNvar" if number_bool and not number_variations_bool else ""
    
    filename += "_noLeg" if not legend_bool else ""
    
    if not scatter_join_bool:
        filename += "_noLines"
    
    filename += filename_suffix
    
    print(filename)
    
    if save_bool:
        print("Saving in",save_path)
        for formatting in ['.png','.pdf']:
            plt.savefig(save_path+filename+formatting, bbox_inches='tight', dpi=300)
            print("Saved",formatting)
    plt.show()

#### Sim & data same ax

Victor didn't like this as the two x-axis are confusing (and they represent different things so a 1-to-1 comparison within the plot isn't appropriate

In [ ]:
fig, ax = plt.subplots()

ax1 = ax.twiny()
ax1.set_xlabel('[Fe/H]',labelpad=10)
ax1.plot(metal_midranges, o_vertex_values, linestyle='--',color='red')
ax1.fill_between(metal_midranges, o_vertex_values-o_vertex_errors, o_vertex_values+o_vertex_errors,alpha=0.4, color='red',label='Apogee') #edgecolor='#CC4F1B', facecolor='#FF9848'
ax1.set_xticks(np.linspace(metal_poor_lim,metal_rich_lim,5))
ax1.set_xlim(metal_rich_lim,metal_poor_lim)

ax.set_xlabel('Age [Gyr]')
ax.set_ylabel(r'$l_{\mathrm{v}} \:\: [^\circ]$')
#ax.tick_params(labelsize=15,length=4,width=1)
ax.set_ylim(-45,45)

invert_axis_bool = False #invert x-axis (from old to young)
if invert_axis_bool:
    ax.set_xlim(max(age_mid_ranges)+half_range/2, min(age_mid_ranges)-half_range/2)
else:
    ax.set_xticks(np.arange(min(age_mid_ranges)-0.5, max(age_mid_ranges)+0.5, 1.0))

vertex_values = np.array(vertex_values)
#ax.errorbar(xb, yb,
#            yerr=vertex_err,
#            fmt='-o')
ax.plot(age_mid_ranges, vertex_values, linestyle='--',color='blue')
ax.fill_between(age_mid_ranges, vertex_values-vertex_errors, vertex_values+vertex_errors,alpha=0.4, facecolor='blue',label='D17') #edgecolor='#CC4F1B', facecolor='#FF9848'

fig.legend(loc=(0.15,0.8))

l_text = fr"${lmin}<l$[{degree_symbol}]$<{lmax}$"
b_text = fr"${bmin}<|b|$[{degree_symbol}]$<{bmax}$"
x,y= 0.06,0.75
ax.text(x=x,y=y,s=l_text,transform=ax.transAxes)
ax.text(x=x,y=y-0.05,s=b_text,transform=ax.transAxes)

#file_name = 'rotangle' + str(rot_angle)+'_708_vertex_age_'+str(age_step_old)+'oldResolution_'+str(age_limit_oldyoung)+'ageLimit_window2x2_invertaxis'+str(invert_axis_bool)+'.png'

plt.savefig(save_path_pop+'age_metal.pdf',bbox_inches='tight')
plt.show()

#### Sim old subdivision

In [ ]:
if not data_bool:
    
    fig,axs = plt.subplots(figsize=(13,10),nrows=2,ncols=2, gridspec_kw={'wspace':0.15,'hspace':0,'width_ratios':[1,0.17],'height_ratios':[0.15,1]})

    barax = axs[0,0]
    barax1 = axs[0,1]
    ax = axs[1,0]
    ax1 = axs[1,1]

    if True: # left right array division 
        age_plot_left = pop_plot_range[:limit_index]
        var_left = map_dict[var][:limit_index]
        err_left = map_dict[err][:limit_index]

        age_plot_right = pop_plot_range[limit_index:]
        var_right = map_dict[var][limit_index:]
        err_right = map_dict[err][limit_index:]

    if True: # bar plots

        number_left = map_dict['number'][:limit_index]
        number_right = map_dict['number'][limit_index:]

        barax.hist(df_extra[df_extra["age"]>4]["age"],bins=50,log=True,histtype="step",color="k")

        barax.bar(age_plot_left,number_left,log=True,width=0.2,color="grey",alpha=0.8)
        barax1.bar(age_plot_right,number_right,log=True,width=0.01)

    #     barax.plot(age_plot_left,number_left)
    #     barax1.plot(age_plot_right,number_right)

        for bar in [barax,barax1]:
            bar.minorticks_off()
            bar.set_xticklabels([])

            min_barticks = 3
            max_bartick = 5
            bar.set_yticks([10**i for i in np.arange(min_barticks,max_bartick+1,1)])

        if old_subplots:
            barax.tick_params(labelleft=False)
            barax1.tick_params(labelleft=False,labelright=True)
        else:
            barax.tick_params(labelleft=False,labelright=True)

    if True: # plot
        ax.plot(age_plot_left,var_left,color=color)
        ax.fill_between(age_plot_left,var_left-err_left,var_left+err_left,label=symbol_dict[var],color=color,alpha=0.5,linewidth=0)
        ax1.plot(age_plot_right,var_right,color=color)
        ax1.fill_between(age_plot_right,var_right-err_right,var_right+err_right,label=symbol_dict[var],color=color,alpha=0.5,linewidth=0)

    if var1_bool: # var1, legend 
        var1_left = map_dict[var1][:limit_index]
        err1_left = map_dict[err1][:limit_index]

        var1_right = map_dict[var1][limit_index:]
        err1_right = map_dict[err1][limit_index:]

        ax.plot(age_plot_left,var1_left,color=color1)
        ax.fill_between(age_plot_left,var1_left-err1_left,var1_left+err1_left,label=r"$%s$"%only_symbol_dict[var1],color=color1,alpha=0.5,linewidth=0)
        ax1.plot(age_plot_right,var1_right,color=color1)
        ax1.fill_between(age_plot_right,var1_right-err1_right,var1_right+err1_right,label=r"$%s$"%only_symbol_dict[var1],color=color1,alpha=0.5,linewidth=0)

        ax.legend(loc='lower right')

    if True: # axhlines, ylim, ticks

        ax.axhline(y=0,linestyle='--',color='grey')
        ax1.axhline(y=0,linestyle='--',color='grey')

        ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))
        barax.xaxis.set_minor_locator(ticker.MultipleLocator(0.25))

        ax1.xaxis.set_minor_locator(ticker.MultipleLocator(0.025))
        barax1.xaxis.set_minor_locator(ticker.MultipleLocator(0.025))

        for axis in [ax,ax1]:

            if var == 'anisotropy' or var == 'correlation':
                ybottom, ytop = -0.5,0.2
                axis.set_ylim(ybottom,ytop); print(f"Lims {ybottom},{ytop}")
                axis.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
                axis.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

            elif var == 'tilt_abs':
                ybottom, ytop = -45, -10
                axis.set_ylim(ybottom,ytop); print(f"Lims {ybottom},{ytop}")
                axis.yaxis.set_major_locator(ticker.MultipleLocator(10))
                axis.yaxis.set_minor_locator(ticker.MultipleLocator(2.5))
            else:
                maxabs = max(np.abs([min(map_dict[var]-map_dict[err]),max(map_dict[var]+map_dict[err])]))

                if var1_bool:
                    maxabs1 = max(np.abs([min(map_dict[var1]-map_dict[err1]),max(map_dict[var1]+map_dict[err1])]))

                    maxabs = max(maxabs,maxabs1)

                axis.set_ylim(-maxabs-0.05,maxabs+0.05)

        ax1.set_yticklabels([])

    if True: # labels and text
        ax.set_xlabel('Age [Gyr]')
        if not var1_bool:
            ax.set_ylabel(symbol_dict[var]+units_dict[var])

    #         if var == 'vertex_abs':
    #             ax.text(x=0.05,y=0.6,s=fr"$|l|<{lmax}^\circ$"+'\n'+fr"${bmin}^\circ<|b|<{bmax}^\circ$"+'\n'+fr"${dmin}<d/\mathrm{{kpc}}<{dmax}$",
    #                     transform=ax.transAxes,fontsize=15)#,bbox={'facecolor':'w'})

        if not old_subplots:
            barax.set_ylabel(r"$N$",rotation=0,labelpad=20)
            barax.yaxis.set_label_position("right")

    if True: # xlim, broken axes params

        if old_subplots:
            left_age_transition = max_age#age_limit_oldyoung
            ax.set_xlim(min_age,left_age_transition)
            barax.set_xlim(min_age,left_age_transition)

            ax1.set_xlim(age_limit_old,max_age)
            barax1.set_xlim(age_limit_old,max_age)

        # hide the spines between ax and ax1
    #     ax.spines['right'].set_visible(False)
    #     ax1.spines['left'].set_visible(False)
    #     ax.tick_params(which='both',right=False)
    #     ax1.tick_params(which='both',left=False)

    #     d = .01
    #     kwargs = dict(transform=ax.transAxes, color='r', clip_on=False)
    #     ax.plot((1-d,1+d), (-d,+d), **kwargs)
    #     ax.plot((1-d,1+d),(1-d,1+d), **kwargs)

    #     # uncomment the lines below if wanting to leave a space between the axes
    #     kwargs.update(transform=ax1.transAxes)  # switch to the bottom axes
    #     d_x = .057
    #     d_y = 0.011
    #     ax1.plot((-d_x,+d_x), (1-d_y,1+d_y), **kwargs)
    #     ax1.plot((-d_x,+d_x), (-d_y,+d_y), **kwargs)

        else:
            ax.set_xlim(min(df["age"]),max(df["age"]))
            barax.set_xlim(min(df["age"]),max(df["age"]))

    if not old_subplots: # delete right column axes 
        fig.delaxes(ax1)
        fig.delaxes(barax1)

    if True: # save

        filename = var
        if var1_bool: filename += var1

        if filename == 'anisotropycorrelation': filename = 'anicorr'
        elif filename == 'mean_vxmean_vy': filename = 'vel'

        filename += f'_sim'
        print(filename)

        if save_bool:
            for f in ['.png','.pdf']:
                plt.savefig(save_path+filename+f,bbox_inches='tight',dpi=200)
            print(save_path+filename)
    plt.show()

# Distance

## Split

Check how wide in longitude we can go before the metal-poor develop a tilt

In [ ]:
lmax = 2
bmin = 0.5
bmax = 10

data_bulge = data[(np.abs(data['l'])<lmax)&(data['d']>6)&(data['d']<10)&(data['b']>bmin)&(data['b']<bmax)]

data_bulge_poor = data_bulge[data_bulge['FeH']<-0.5]

In [ ]:
vx = data_bulge_poor.vr.values
vy = data_bulge_poor.vl.values

# val = CV.calculate_tilt(vx,vy,absolute=True)
# err = CV.get_std_bootstrap(vx,vy,CV.calculate_tilt,tilt=True,absolute=True)
val = CV.calculate_correlation(vx,vy)
err = CV.get_std_bootstrap(vx,vy,CV.calculate_correlation)

print(r"%.2f +- %.2f"%(val,err))

Split in distance distribution for the simulation, check at what latitude it appears

In the literature they say the MW's one appears with $l=0$ and $b=-5$, but most of my data lives below $b=5$

4 to 7 Gyr: inner split visible at $b>2.5$, hard to see at $b>4$, where outer X-shape dominates

9.8 to 10 Gyr: visible at $b>3.5$, far end dissappears after $b>6$

4 to 10 Gyr: visible at $b>3$, far end dissapears after $b>6$

In [ ]:
lmax = 2
bmin = 6
bmax = 10

# agemin = 4
# agemax = 10
agemin = 4
agemax = 7
# agemin = 9.8
# agemax = 10

df_bulge = df0[(np.abs(df0['l'])<lmax)&(df0['d']>6)&(df0['d']<10)&(df0['b']>bmin)&(df0['b']<bmax)&(df0['age']>agemin)&(df0['age']<agemax)]

In [ ]:
fig,ax=plt.subplots()

ax.hist(df_bulge['d'],bins=50)
plt.show()

In [ ]:
lmax = 2
bmin = 2.5
bmax = 10

metalmin = data['FeH'].min()
metalmax = data['FeH'].max()

data_bulge = data[(np.abs(data['l'])<lmax)&(data['d']>6)&(data['d']<10)&(data['b']>bmin)&(data['b']<bmax)&(data['FeH']>metalmin)&(data['FeH']<metalmax)]

In [ ]:
fig,ax=plt.subplots()

ax.hist(data_bulge['d'],bins=30)
plt.show()

## Plot

In [ ]:
lmin = -1.5
lmax = 1.5
bmin = 2.5
bmax = 4.5

dmin = 6
dmax = 10

df_extra = df0[(df0["l"]>lmin)&(df0["l"]<lmax)&(df0["d"]>dmin)&(df0["d"]<dmax)&(df0["b"]>bmin)&(df0["b"]<bmax+b_step)]
df_ages = [df_extra[(df_extra["age"]>agelim[0])&(df_extra["age"]<agelim[1])] for agelim in age_limits]

poor_condition = (o_df_extra["FeH"] < metal_poor_highlim)&(o_df_extra["FeH"] > metal_poor_lowlim)
rich_condition = o_df_extra["FeH"] > metal_rich_lowlim if metal_rich_highlim is None else (o_df_extra["FeH"] < metal_rich_highlim)&(o_df_extra["FeH"] > metal_rich_lowlim)

df_metals = [o_df_extra[rich_condition], o_df_extra[poor_condition]]

In [ ]:
d_step = 1
d_range = np.arange(dmin,dmax,d_step)
d_range_plot = d_range+d_step/2
o_d_range_plot = d_range_plot
print("d_range is",d_range)
print("Plotting at:",d_range_plot)

In [ ]:
save_path = general_path + "708main_simulation/graphs/Oscar/Apogee/"
create_dir(save_path)

# save_path += "scaling_"+str(sim_scaling)+'/'
# MF.create_dir(save_path)

save_path += "individual_variable/"
MF.create_dir(save_path)

save_path += "distance/"
MF.create_dir(save_path)
    
save_path += f"{lmin}l{lmax}/"
create_dir(save_path)

save_path += f"{bmin}b{bmax}/"
create_dir(save_path)
    
save_path += f"{young_min}-{young_max}_{old_min}-{old_max}/"
create_dir(save_path)

#save_path += 'halo_metal/'
#create_dir(save_path)

if not galactocentric:
    save_path += 'LSR/'
    create_dir(save_path)

poor_condition = (o_df_extra["FeH"] < metal_poor_highlim)&(o_df_extra["FeH"] > metal_poor_lowlim)
label_poor = fr'${metal_poor_lowlim}<$[Fe/H]$<{metal_poor_highlim}$'

if metal_rich_highlim is None:
    rich_condition = o_df_extra["FeH"] > metal_rich_lowlim
    label_rich = fr'${metal_rich_lowlim}<$[Fe/H]'
    save_path += f"{metal_rich_lowlim}to{metal_rich_highlim}_{metal_poor_lowlim}to{metal_poor_highlim}/"
else:
    rich_condition = (o_df_extra["FeH"] < metal_rich_highlim)&(o_df_extra["FeH"] > metal_rich_lowlim)
    label_rich = fr'${metal_rich_lowlim}<$[Fe/H]$<{metal_rich_highlim}$'
    save_path += f"{metal_rich_lowlim}to{metal_rich_highlim}_{metal_poor_lowlim}to{metal_poor_highlim}/"

df_metals = [o_df_extra[rich_condition], o_df_extra[poor_condition]]
label_rich += f" ({len(df_metals[0])})"
label_poor += f" ({len(df_metals[1])})"

create_dir(save_path)

if halo_bool:
    df_metals.append(o_df_extra[o_df_extra["FeH"] < metal_halo_lim])    
    label_halo = fr'(%i) [Fe/H]$<{metal_halo_lim}$'%len(df_metals[2])
    print("Working with the halo population")

print("SAVING IN\n"+save_path)

fig, ax = plt.subplots()

alpha=0.7
bins = np.linspace(dmin,dmax,50)
if not halo_bool:
    ax.hist(o_df_extra['d'],bins=bins,label='(%i) All'%len(o_df_extra['d']),alpha=alpha,color='orange')
ax.hist(df_metals[1]['d'],bins=bins,label=label_poor,alpha=alpha,color='blue')
ax.hist(df_metals[0]['d'],bins=bins,label=label_rich,alpha=alpha,color='red')
if halo_bool:
    ax.hist(df_metals[2]['d'],bins=bins,label=label_halo,alpha=alpha*0.75,color='cyan')
ax.set_xlim(dmin,dmax)
ax.set_xticks(np.arange(dmin,dmax,1))
ax.set_xlabel(r'$d$ [kpc]')
ax.set_ylabel(r"$N$",rotation=0,labelpad=20)
ax.legend(loc='upper right')
ax.text(0.05,0.9,fr"${lmin} < l < {lmax}$",transform=ax.transAxes)
ax.text(0.07,0.85,fr"${bmin} < b < {bmax}$",transform=ax.transAxes)
plt.savefig(save_path+f"number_observations_{lmin}l{lmax}_{bmin}b{bmax}.png",bbox_inches='tight',dpi=150)
plt.show()

In [ ]:
all_arrays = np.zeros(shape=(len(full_map_string_list),len(d_range),len(df_ages)))
o_all_arrays = np.zeros(shape=(len(full_map_string_list),len(d_range),len(df_metals)))

for d_index,distance in enumerate(d_range):
    for age_index,df in enumerate(df_ages):
        vr = df[(df['d']>distance)&(df['d']<distance+d_step)].vr.values
        vl = df[(df['d']>distance)&(df['d']<distance+d_step)].vl.values
        
        values = get_all_variable_values_and_errors(vr,vl,bootstrap_repeat=100,min_number=min_number_sim)
        
        for index, val in enumerate(values):
            all_arrays[index, d_index, age_index] = val
    
    for metal_index, o_df in enumerate(df_metals):
        vr = o_df[(o_df['d']>distance)&(o_df['d']<distance+d_step)].vr.values
        vl = o_df[(o_df['d']>distance)&(o_df['d']<distance+d_step)].vl.values
        
        values = get_all_variable_values_and_errors(vr,vl,bootstrap_repeat=100,min_number=10)
        
        for index, val in enumerate(values):
            o_all_arrays[index, d_index, metal_index] = val
    
print("Done")

In [ ]:
map_dict = create_map_array_dict(full_map_string_list, all_arrays)
o_map_dict = create_map_array_dict(full_map_string_list, o_all_arrays)

In [ ]:
xsymbol = d

In [ ]:
map_string = "vertex_abs"

In [ ]:
map_array = map_dict[map_string]
error_array = map_dict[error_string]
map_symbol = symbol_dict[map_string]
map_title = title_dict[map_string]
yticks = get_variable_ticks(map_string, map_array)
displacement = max(yticks) - min(yticks)

color_y, color_o = color_dict['vertex_abs'][0], color_dict['vertex_abs'][1]

transparency = 0.5
alpha_area = 0.4

fig, ax = plt.subplots()

divider = make_axes_locatable(ax)
ax_histx = divider.append_axes("top", size=1.2, pad=0, sharex=ax)
ax_histx.xaxis.set_tick_params(labelbottom=False)

alpha=0.7
ax_histx.hist(df_metals[1]['d'],bins=bins,alpha=1,color=color_o)
ax_histx.hist(df_metals[0]['d'],bins=bins,alpha=alpha,color=color_y)
if halo_bool:
    ax_histx.hist(df_metals[2]['d'],bins=bins,alpha=alpha,color='cyan')

bar_width=0.2
ax_histx.bar(d_range_plot-bar_width/2, map_dict["number"][:,0], width=bar_width,alpha=transparency,log=True,color=color_y)
ax_histx.bar(d_range_plot+bar_width/2, map_dict["number"][:,1], width=bar_width,alpha=transparency,log=True,color=color_o)
ax_histx.yaxis.set_tick_params(labelleft=False,labelright=True)
ax_histx.set_yticks([1,10,100,1000,10000,100000])
ax_histx.set_ylabel(r"$N$",labelpad=15,rotation=0)
ax_histx.yaxis.set_label_position("right")

ax.plot(d_range_plot, map_array[:,1] , color=color_o, linestyle='--', lw=1)
ax.plot(d_range_plot, map_array[:,0], color=color_y, linestyle='--', lw=1)


ax.fill_between(d_range_plot, map_array[:,1]-error_array[:,1], map_array[:,1]+error_array[:,1],alpha=alpha_area, facecolor=color_o, label=label_o)
ax.fill_between(d_range_plot, map_array[:,0]-error_array[:,0], map_array[:,0]+error_array[:,0],alpha=alpha_area, facecolor=color_y, label=label_y)

if not absolute:
    ax.fill_between(d_range_plot, map_array[:,1]-error_array[:,1]+displacement, map_array[:,1]+error_array[:,1]+displacement,alpha=alpha_area, facecolor=color_o)
    ax.fill_between(d_range_plot, map_array[:,1]-error_array[:,1]-displacement, map_array[:,1]+error_array[:,1]-displacement,alpha=alpha_area, facecolor=color_o)
    ax.fill_between(d_range_plot, map_array[:,0]-error_array[:,0]+displacement, map_array[:,0]+error_array[:,0]+displacement,alpha=alpha_area, facecolor=color_y)
    ax.fill_between(d_range_plot, map_array[:,0]-error_array[:,0]-displacement, map_array[:,0]+error_array[:,0]-displacement,alpha=alpha_area, facecolor=color_y)

#OBSERVATIONS-----------------------------------------------------------------------------------------------
o_map_array = o_map_dict[map_string]
o_error_array = o_map_dict[error_string]

ax.errorbar(o_d_range_plot, o_map_array[:,1] , yerr= o_error_array[:,1], color=color_o, label=label_poor,fmt='o',marker="$\u25A1$")
ax.errorbar(o_d_range_plot, o_map_array[:,0], yerr= o_error_array[:,0], color=color_y, alpha=0.8,label=label_rich,fmt='s',marker="$\u25EF$")
if halo_bool:
    ax.errorbar(o_d_range_plot, o_map_array[:,2] , yerr=o_error_array[:,2], color='cyan', label=label_halo,fmt='o',markersize=8,marker="$\u25B3$")

#------------------------------------------------------------------------------------------------------------
    
ax.legend(fontsize=15,loc="best")

ax.set_xlim(dmin,dmax)
ax.set_xticks(np.arange(dmin,dmax,1))
ax.set_yticks(yticks)
ax.set_ylim(min(yticks),max(yticks))

ax.set_xlabel(r"$%s$ [°]"%xsymbol)
ax.set_ylabel(map_symbol)

title_string = 'Vertex deviation (absolute value equation)' if absolute else 'Vertex deviation'
ax_histx.set_title(title_string,fontsize=18,pad=10)

#ax.set_aspect(0.082 if absolute else 0.04)

ax.text(x=-0.14,y=1.24,s='Sim scaling '+str(sim_scaling),size=13, transform=ax.transAxes)
l_string = r"$%i < l [%s] < {%i},$"%(lmin,degree_symbol,lmax)
d_string = r"${%i}<d [\mathrm{%s}]<{%i}$"%(dmin,'kpc',dmax)
text_y = -0.09
ax.text(x=0.79,y=text_y,s=l_string,size=13,transform=ax.transAxes)
ax.text(x=0.94,y=text_y,s=d_string,size=13,transform=ax.transAxes)
plt.savefig(save_path+map_string+'_distance.pdf',bbox_inches='tight')
print(save_path+map_string)
plt.show()

In [ ]:
age_min = 9.
age_max = 10.

dmin = 6
dmax = 10

In [ ]:
d_step = 0.5
distance_range = np.arange(dmin,dmax,d_step)

In [ ]:
lb_step = 1.5

lmin = -lb_step/2
lmax = lb_step/2
bmin = -lb_step/2
bmax = lb_step/2

min_star_number = 10

limit_vertex = -40

In [ ]:
longitude_range = np.arange(lmin, lmax, lb_step)
latitude_range = np.arange(bmin, bmax, lb_step)

print("Longitude range:",longitude_range)
print("Latitude range:",latitude_range)

In [ ]:
for i in latitude_range:
    print(i,i+lb_step)

In [ ]:
save_path = "708main_simulation/graphs/vertex_distance/"

In [ ]:
#No division in distance intervals
theta_across_d = []

df_dist_age = df0[(df0.age <= age_max)&(df0.age >= age_min)&(df0.d <= dmax)&(df0.d>=dmin)]

for longitude in longitude_range:
        df_long = df_dist_age[(df_dist_age.l >= longitude)&(df_dist_age.l < longitude + lb_step)]

        for latitude in latitude_range:
            df_lat = df_long[(df_long.b >= latitude)&(df_long.b < latitude + lb_step)]

            cov = np.cov(df_lat.vr.values, df_lat.pml.values)
            varr = cov[0,0]
            varl = cov[1,1]
            covrl = cov[0,1]

            if len(df_lat) > min_star_number:
                theta = np.degrees(np.arctan2(2.*covrl, np.abs(varr - varl))/2.)
            else:
                continue
            
            print(theta)
            theta_across_d.append(theta)
del df_dist_age, df_long, df_lat

In [ ]:
with open(save_path + "total_vertex_"+str(lmin)+"l"+str(lmax)+"_"+str(bmin)+"b"+str(bmax)+".txt", 'w') as f:
    f.write("The total vertex deviation across the line of sight (6 < d < 10)kpc is: "+str(theta_across_d[0])+'°'\
           "\n\nWorking with a "+str(lb_step)+"×"+str(lb_step)+"° window in: \n"+str(lmin)+" <= l < "+str(lmax)+
           "\n"+str(bmin)+" <= b < "+str(bmax))

In [ ]:
df_age = df0[(df0.age <= age_max)&(df0.age >= age_min)]

x_positions, y_positions, x_velocities, y_velocities, r_vel, t_vel = [], [], [], [], [], []
theta_values, number_points = [], []

i = 0

for distance in distance_range:
    df_dist = df_age[(df_age.d >= distance)&(df_age.d < distance + d_step)]

    for longitude in longitude_range:
        df_long = df_dist[(df_dist.l >= longitude)&(df_dist.l < longitude + lb_step)]

        for latitude in latitude_range:
            df_lat = df_long[(df_long.b >= latitude)&(df_long.b < latitude + lb_step)]

            cov = np.cov(df_lat.vr.values, df_lat.pml.values)
            varr = cov[0,0]
            varl = cov[1,1]
            covrl = cov[0,1]

            if len(df_lat) > min_star_number:
                theta = np.degrees(np.arctan2(2.*covrl, np.abs(varr - varl))/2.)
            else:
                continue
            
            print(theta)
            #if theta < limit_vertex:
            number_points.append(len(df_lat.x.values))
            theta_values.append(theta)
            x_positions.append(df_lat.x.values)
            y_positions.append(df_lat.y.values)
            x_velocities.append(df_lat.vx.values)
            y_velocities.append(df_lat.vy.values)
            r_vel.append(df_lat.vr.values)
            t_vel.append(df_lat.pml.values)
        
        i+=1
                        
print("There were",i,"intervals")
print("There are",len(distance_range),"distance bins")

del df_dist, df_long, df_lat

In [ ]:
print("The max number of points is",np.max(number_points))

In [ ]:
fig, ax = plt.subplots()

n_points_frac = 0.5
l1= ax.scatter(distance_range+0.25, theta_values, s=n_points_frac*np.array(number_points), alpha=0.7)
ax.plot(distance_range+0.25, theta_values)#, s=number_points, alpha=0.7)

ax.set_xlim(6,10)


#handles, labels = l1.legend_elements(prop="sizes", num = [500,1000,1500], alpha=0.7, color='blue')
#leg = ax.legend(handles, labels, loc="lower right", title="#Datapoints", numpoints = 1, fontsize=13, labelspacing=1)
#leg.get_title().set_fontsize('13')
size1, size2, size3 = 500, 1500, 2500
leg_colour = "tab:blue"
leg1 = ax.scatter([],[],s = n_points_frac*size1, color=leg_colour, alpha=0.7)
leg2 = ax.scatter([],[],s = n_points_frac*size2, color=leg_colour, alpha=0.7)
leg3 = ax.scatter([],[],s = n_points_frac*size3, color=leg_colour, alpha=0.7)

leg = ax.legend((leg1, leg2, leg3),
             (str(size1),str(size2),str(size3)),
             scatterpoints=1,
             loc="lower right",
             ncol=1,
             fontsize=13,
             title = "# datapoints",
             labelspacing=1.3)
leg.get_title().set_fontsize('14')


ax.set_xlabel(r"Distance (kpc)")
ax.set_ylabel(r"$\theta_v \hspace{0.4}[°]$")

filename = "vertexdistance_"+str(lmin)+"l"+str(lmax)+"_"+str(bmin)+"b"+str(bmax)+"_"+str(dmin)+"d"+str(dmax)+".png"
plt.savefig(save_path + filename, bbox='tight')

In [ ]:
print(save_path)

In [ ]:
save_path_vel = save_path + "velocities_"+str(lmin)+"l"+str(lmax)+"_"+str(bmin)+"b"+str(bmax)+'/'

if not os.path.isdir(save_path_vel):
    os.mkdir(save_path_vel)

print(save_path_vel)

In [ ]:
for index in range(len(theta_values)):
    fig, ax = plt.subplots()

    ax.scatter(r_vel[index], t_vel[index], marker='.', color='grey')

    cmap2 = 'coolwarm'

    level_n = 10

    sns.kdeplot(r_vel[index], t_vel[index], cmap=cmap2, cut=1, n_levels=level_n, fill=True, shade_lowest=False, \
                alpha=0.4, cbar=False, aspect='equal', extend='both')

    mec = sns.kdeplot(r_vel[index], t_vel[index], cmap=cmap2, cut=1, n_levels=level_n, fill=False, shade_lowest=False, \
                alpha=1, cbar=True, aspect='equal', extend='both', linewidths=2, \
                      cbar_kws={'label': r'Probability density [$\rm s^{2} \hspace{0.3} km^{-2}$]'})

    ax.set_xlabel(r"$v_r$ [km $\rm s^{-1}$]")
    ax.set_ylabel(r"$\mu_l$ [km $\rm s^{-1}$]")

    text_box = dict(boxstyle='round', facecolor='wheat', alpha=1)
    theta = theta_values[index]
    ax.text(0.6, 0.95, r"$\theta_v=$"+(r'$-$' if abs(theta) != theta else '')\
            +str(np.float16(abs(theta)))+'°', transform=ax.transAxes, fontsize=20, verticalalignment='top', bbox=text_box)
    ax.set_aspect('equal')
    
    dmin = distance_range[index]
    dmax = dmin + d_step
    
    ax.set_title(str(dmin)+r" $\leq$ Distance [kpc] $<$ "+str(dmax), fontsize=18)
    
    filename = "velocity_"+str(lmin)+"l"+str(lmax)+"_"+str(bmin)+"b"+str(bmax)+"_"+str(dmin)+"d"+str(dmax)+".png"
    plt.savefig(save_path_vel+filename, bbox_inches='tight')